    "اول کلاس کد مربوط به بخش انکدر شبکه ماشین ترجمه عصبی است. همانطور که در فایل متنی به تفصیل ارائه شده است، انکدر یک شبکه عصبی دو طرفه ال اس تی ام است"


In [ ]:
import sys
from chainer import *
from utilities import *
import random
import copy

class Encoder(Chain):
    #Bi-directional LSTM (forward + backward)
    def __init__(self, source_vocabulary_size, embed_size, hidden_size, source_vocabulary, source_word2vec, use_dropout, dropout_rate, library):
        super(Encoder, self).__init__(
            word2embed = links.EmbedID(source_vocabulary_size, embed_size, ignore_label = -1),
            embed2hidden_forward = links.LSTM(embed_size, hidden_size),
            embed2hidden_backward = links.LSTM(embed_size, hidden_size),
        )
        if source_word2vec is not None:
            for i in range(source_vocabulary_size):
                word = source_vocabulary.id2word[i]
                if word in source_word2vec:
                    self.word2embed.W.data[i] = source_word2vec[word]
        self.vocabulary_size = source_vocabulary_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.use_dropout = use_dropout
        self.dropout_rate = dropout_rate
        self.library = library

    def __call__(self, sentence):
        return self.forward(sentence)[0]
    
    def forward(self, sentence):
        embed_states = list()
        hidden_backward_states = list()
        hidden_states = list()
        for word in sentence:
            embed_states.append(functions.dropout(functions.tanh(self.word2embed(word)), ratio = self.dropout_rate))
        for embed in embed_states[::-1]:
            hidden_backward_states.insert(0, functions.dropout(functions.tanh(self.embed2hidden_backward(embed)), ratio = 0.0)) #False
        for embed, hidden_backward in zip(embed_states, hidden_backward_states):
            plus = functions.dropout(functions.tanh(self.embed2hidden_forward(embed)), ratio = 0.0) + hidden_backward #False
            hidden_states.append(plus)
        return hidden_states, embed_states

    def reset_states(self):
        self.embed2hidden_forward.reset_state()
        self.embed2hidden_backward.reset_state()

    "کلاس بعدی اقدام به ایجاد شبکه دیکدر می نماید. شبکه دیکدر در نظر گرفته شده یک شبکه عصبی یک لایه یک طرفه ال اس تی ام است. همچنین برای بالا بردن دقت عملکردی برنامه به شبکه یک لایه اتنشن نیز اضافه شده است"


In [ ]:
class Decoder(Chain):
	#Luong Global-Attention (dot)
    def __init__(self, target_vocabulary_size, embed_size, hidden_size, target_vocabulary, target_word2vec, use_dropout, dropout_rate, generation_limit, use_beamsearch, beam_size, library):
        super(Decoder, self).__init__(
            encoder2decoder_init = links.LSTM(hidden_size, hidden_size),
            word2embed = links.EmbedID(target_vocabulary_size, embed_size, ignore_label = -1),
            embed_hidden_tilde2hidden = links.LSTM(embed_size + hidden_size, hidden_size),
            attention_hidden2hidden_tilde = links.Linear(2 * hidden_size, hidden_size),
            hidden_tilde2predict = links.Linear(hidden_size, target_vocabulary_size),
        )
        if target_word2vec is not None:
            for i in range(target_vocabulary_size):
                word = target_vocabulary.id2word[i]
                if word in target_word2vec:
                    self.word2embed.W.data[i] = target_word2vec[word]
        self.vocabulary_size = target_vocabulary_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.use_dropout = use_dropout
        self.dropout_rate = dropout_rate
        self.generation_limit = generation_limit
        self.use_beamsearch = use_beamsearch
        self.beam_size = beam_size
        self.library = library

    def __call__(self, encoder_hidden_states, sentence):
        return self.forward(encoder_hidden_states, sentence)[:2]

    def forward(self, encoder_hidden_states, sentence):
        predicts = list()
        target_embed_states = list()
        predict_embed_states = list()
        hidden_states = list()
        attention_weights_matrix = list()
        if sentence is not None:
            loss = Variable(self.library.zeros((), dtype = self.library.float32))
            for i, word in enumerate(sentence):
                if i == 0:
                    hidden = functions.dropout(functions.tanh(self.encoder2decoder_init(encoder_hidden_states[0])), ratio = 0.0) #False
                    encoder_hidden_states = functions.dstack(encoder_hidden_states)
                    self.copy_states()
                else:
                    previous_embed = functions.dropout(functions.tanh(self.word2embed(sentence[i - 1])), ratio = self.dropout_rate)
                    target_embed_states.append(previous_embed)
                    hidden = functions.dropout(functions.tanh(self.embed_hidden_tilde2hidden(functions.concat((previous_embed, hidden_tilde)))), ratio = 0.0) #False
                attention_weights = functions.softmax(functions.batch_matmul(encoder_hidden_states, hidden, transa = True))
                attention = functions.reshape(functions.batch_matmul(encoder_hidden_states, attention_weights), (encoder_hidden_states.shape[0], encoder_hidden_states.shape[1]))
                hidden_tilde = functions.dropout(functions.tanh(self.attention_hidden2hidden_tilde(functions.concat((attention, hidden)))), ratio = self.dropout_rate)
                hidden_states.append(hidden_tilde)
                score = self.hidden_tilde2predict(hidden_tilde)
                predict = functions.argmax(score, axis = 1)
                loss += functions.softmax_cross_entropy(score, word, ignore_label = -1)
                predict = functions.where(word.data == -1, word, predict)
                predicts.append(predict.data)
                predict_embed_states.append(functions.dropout(functions.tanh(self.word2embed(predict)), ratio = self.dropout_rate))
            target_embed_states.append(functions.dropout(functions.tanh(self.word2embed(sentence[-1])), ratio = self.dropout_rate))
            return loss, predicts, target_embed_states, predict_embed_states, hidden_states, None

        elif not self.use_beamsearch:
            while len(predicts) < self.generation_limit:
                if len(predicts) == 0:
                    hidden = functions.tanh(self.encoder2decoder_init(encoder_hidden_states[0]))
                    encoder_hidden_states = functions.dstack(encoder_hidden_states)
                    self.copy_states()
                else:
                    previous_embed = functions.tanh(self.word2embed(predict))
                    predict_embed_states.append(previous_embed)
                    hidden = functions.tanh(self.embed_hidden_tilde2hidden(functions.concat((previous_embed, hidden_tilde))))
                attention_weights = functions.softmax(functions.batch_matmul(encoder_hidden_states, hidden, transa = True))
                attention_weights_matrix.append(functions.reshape(attention_weights, (attention_weights.shape[0], attention_weights.shape[1])))
                attention = functions.reshape(functions.batch_matmul(encoder_hidden_states, attention_weights), (encoder_hidden_states.shape[0], encoder_hidden_states.shape[1]))
                hidden_tilde = functions.tanh(self.attention_hidden2hidden_tilde(functions.concat((attention, hidden))))
                hidden_states.append(hidden_tilde)
                score = self.hidden_tilde2predict(hidden_tilde)
                predict = functions.argmax(score, axis = 1)
                predicts.append(predict.data)
                if predict.data[0] == 1:
                    break
            predict_embed_states.append(functions.tanh(self.word2embed(predict)))
            attention_weights_matrix = functions.stack(attention_weights_matrix, axis = 1)
            return None, predicts, None, predict_embed_states, hidden_states, attention_weights_matrix

        else:
            initial_beam = [(0, None, list(), encoder_hidden_states, list(), list(), list())]
            for _, _, sentence, _, predict_embed_states, hidden_states, attention_weights_matrix in sorted(self.n_forwards(initial_beam), key = lambda x: x[0].data / len(x[2]), reverse = True):
                for word in sentence:
                    predicts.append(word.data)
                attention_weights_matrix = functions.stack(attention_weights_matrix, axis = 1)
                break
            return None, predicts, None, predict_embed_states, hidden_states, attention_weights_matrix

    def n_forwards(self, initial_beam):
        beam = [0] * self.generation_limit
        for i in range(self.generation_limit):
            beam[i] = list()
            if i == 0:
                new_beam = list()
                for logprob, states, sentence, encoder_hidden_states, embed_states, hidden_states, attention_weights_matrix in initial_beam:
                    hidden = functions.tanh(self.encoder2decoder_init(encoder_hidden_states[0]))
                    encoder_hidden_states = functions.dstack(encoder_hidden_states)
                    self.copy_states()
                    attention_weights = functions.softmax(functions.batch_matmul(encoder_hidden_states, hidden, transa = True))
                    attention = functions.reshape(functions.batch_matmul(encoder_hidden_states, attention_weights), (encoder_hidden_states.shape[0], encoder_hidden_states.shape[1]))
                    hidden_tilde = functions.tanh(self.attention_hidden2hidden_tilde(functions.concat((attention, hidden))))
                    prob = functions.softmax(self.hidden_tilde2predict(hidden_tilde))
                    cell, hidden = self.get_states()
                    for predict in numpy.argsort(cuda.to_cpu(prob.data)[0])[-1:-self.beam_size-1:-1]:
                        predict_variable = Variable(self.library.array([predict], dtype = self.library.int32))
                        if predict == 1:
                            new_beam.append((logprob + functions.log(prob[0][predict]), None, sentence + [predict_variable], encoder_hidden_states, embed_states + [functions.tanh(self.word2embed(predict_variable))], hidden_states + [hidden_tilde], attention_weights_matrix + [functions.reshape(attention_weights, (attention_weights.shape[0], attention_weights.shape[1]))])) 
                        else:
                            new_beam.append((logprob + functions.log(prob[0][predict]), (cell, hidden, hidden_tilde), sentence + [predict_variable], encoder_hidden_states, embed_states, hidden_states + [hidden_tilde], attention_weights_matrix + [functions.reshape(attention_weights, (attention_weights.shape[0], attention_weights.shape[1]))]))
                for _, (logprob, states, sentence, encoder_hidden_states, embed_states, hidden_states, attention_weights_matrix) in zip(range(self.beam_size), sorted(new_beam, key = lambda x: x[0].data / len(x[2]), reverse = True)):
                    beam[i].append((logprob, states, sentence, encoder_hidden_states, embed_states, hidden_states, attention_weights_matrix))
            else:
                new_beam = list()
                for logprob, states, sentence, encoder_hidden_states, embed_states, hidden_states, attention_weights_matrix in beam[i - 1]:
                    if states is not None:
                        previous_cell, previous_hidden, previous_hidden_tilde = states
                        self.embed_hidden_tilde2hidden.set_state(previous_cell, previous_hidden)
                        previous_embed = functions.tanh(self.word2embed(sentence[-1]))
                        hidden = functions.tanh(self.embed_hidden_tilde2hidden(functions.concat((previous_embed, previous_hidden_tilde))))
                        attention_weights = functions.softmax(functions.batch_matmul(encoder_hidden_states, hidden, transa = True))
                        attention = functions.reshape(functions.batch_matmul(encoder_hidden_states, attention_weights), (encoder_hidden_states.shape[0], encoder_hidden_states.shape[1]))
                        hidden_tilde = functions.tanh(self.attention_hidden2hidden_tilde(functions.concat((attention, hidden))))
                        prob = functions.softmax(self.hidden_tilde2predict(hidden_tilde))
                        cell, hidden = self.get_states()
                        for predict in numpy.argsort(cuda.to_cpu(prob.data)[0])[-1:-self.beam_size-1:-1]:
                            predict_variable = Variable(self.library.array([predict], dtype = self.library.int32))
                            if predict == 1:
                                new_beam.append((logprob + functions.log(prob[0][predict]), None, sentence + [predict_variable], encoder_hidden_states, embed_states + [previous_embed, functions.tanh(self.word2embed(predict_variable))], hidden_states + [hidden_tilde], attention_weights_matrix + [functions.reshape(attention_weights, (attention_weights.shape[0], attention_weights.shape[1]))])) 
                            else:
                                new_beam.append((logprob + functions.log(prob[0][predict]), (cell, hidden, hidden_tilde), sentence + [predict_variable], encoder_hidden_states, embed_states + [previous_embed], hidden_states + [hidden_tilde], attention_weights_matrix + [functions.reshape(attention_weights, (attention_weights.shape[0], attention_weights.shape[1]))]))
                    else:
                        new_beam.append((logprob, None, sentence, encoder_hidden_states, embed_states, hidden_states, attention_weights_matrix))
                for _, (logprob, states, sentence, encoder_hidden_states, embed_states, hidden_states, attention_weights_matrix) in zip(range(self.beam_size), sorted(new_beam, key = lambda x: x[0].data / len(x[2]), reverse = True)):
                    beam[i].append((logprob, states, sentence, encoder_hidden_states, embed_states, hidden_states, attention_weights_matrix))
        return beam[-1]


    def get_states(self):
        cell = copy.deepcopy(self.embed_hidden_tilde2hidden.c)
        hidden = copy.deepcopy(self.embed_hidden_tilde2hidden.h)
        return cell, hidden

    def copy_states(self):
        cell = self.encoder2decoder_init.c
        hidden = self.encoder2decoder_init.h
        self.embed_hidden_tilde2hidden.set_state(cell, hidden)

    def reset_states(self):
        self.encoder2decoder_init.reset_state()
        self.embed_hidden_tilde2hidden.reset_state()

    "کلاس زیر نیز به تعریف ماشین ترجمه عصبی مبتنی بر اتنشن می پردازد. برای تعیین وضعیت های مورد توجه مقادیر وضعیت ها از شبکه های انکدر و دیکدر دریافت می شود و با بررسی پیش بینی و خطای شبکه حالت ها مورد توجه برای شبکه تعیین می گردد"


In [ ]:
class AttentionalNMT(Chain):
    def __init__(self, source_vocabulary_size, target_vocabulary_size, embed_size, hidden_size, source_vocabulary, target_vocabulary, source_word2vec, target_word2vec, use_dropout, dropout_rate, generation_limit, use_beamsearch, beam_size, library):
        super(AttentionalNMT, self).__init__(
            encoder = Encoder(source_vocabulary_size, embed_size, hidden_size, source_vocabulary, source_word2vec, use_dropout, dropout_rate, library),
            decoder = Decoder(target_vocabulary_size, embed_size, hidden_size, target_vocabulary, target_word2vec, use_dropout, dropout_rate, generation_limit, use_beamsearch, beam_size, library),
		)

    def __call__(self, batch_source, batch_target):
        self.reset_states()
        encoder_hidden_states = self.encoder(batch_source)
        loss, predicts = self.decoder(encoder_hidden_states, batch_target)
        return loss, predicts

    def forward(self, batch_source, batch_target):
        self.reset_states()
        encoder_hidden_states, source_embed_states = self.encoder.forward(batch_source)
        loss, predicts, target_embed_states, predict_embed_states, decoder_hidden_states, attention_matrix = self.decoder.forward(encoder_hidden_states, batch_target)
        return loss, predicts, source_embed_states, target_embed_states, predict_embed_states, encoder_hidden_states, decoder_hidden_states, attention_matrix
    
    def reset_states(self):
        self.encoder.reset_states()
        self.decoder.reset_states()

    "تابع زیر اقدام به استخراج کانفیگ های تعریف شده برای پارامترهای شبکه ماشین ترجمه عصبی می نماید. همچنین فایل های مرتبط برای برنامه شامل فایل لغات، بردارهای جاسازی لازم ساخته و یا بارگزاری می شود. در نهایت خروجی شبکه شامل خطا، مدل، وزن و مدل بهینه شبکه ذخیره می شود"


In [ ]:
def train(config):
    if len(sys.argv) == 4:
        start = int(sys.argv[3]) - 1
        trace("Start Re-Training ...")
        trace("Loading Vocabulary ...")
        source_vocabulary = Vocabulary.load("{}.{:03d}.source_vocabulary".format(config.model, start))
        target_vocabulary = Vocabulary.load("{}.{:03d}.target_vocabulary".format(config.model, start))
        source_word2vec = None
        target_word2vec = None
    else:
        start = 0
        trace("Start Training ...")
        trace("Making Vocabulary ...")
        source_vocabulary = Vocabulary.make(config.source_train, config.source_vocabulary_size)
        target_vocabulary = Vocabulary.make(config.target_train, config.target_vocabulary_size)

        if config.use_word2vec == "Load":
            trace("Loading Word2vec ...")
            source_word2vec = load_word2vec(config.source_word2vec_file)
            target_word2vec = load_word2vec(config.target_word2vec_file)
            save_word2vec(source_word2vec, "{}.source_word2vec".format(config.model))
            save_word2vec(target_word2vec, "{}.target_word2vec".format(config.model))
        elif config.use_word2vec == "Make":
            trace("Making Word2vec ...")
            source_word2vec = make_word2vec(config.source_train, config.embed_size)
            target_word2vec = make_word2vec(config.target_train, config.embed_size)
            save_word2vec(source_word2vec, "{}.source_word2vec".format(config.model))
            save_word2vec(target_word2vec, "{}.target_word2vec".format(config.model))
        else:
            source_word2vec = None
            target_word2vec = None

    config.source_vocabulary_size = source_vocabulary.size
    config.target_vocabulary_size = target_vocabulary.size
    
    trace("Making Model ...")
    nmt = AttentionalNMT(config.source_vocabulary_size, config.target_vocabulary_size, config.embed_size, config.hidden_size, source_vocabulary, target_vocabulary, source_word2vec, target_word2vec, config.use_dropout, config.dropout_rate, None, False, None, config.library)
    if config.use_gpu:
        cuda.get_device(config.gpu_device).use()
        nmt.to_gpu()

    opt = config.optimizer
    opt.setup(nmt)
    opt.add_hook(optimizer.GradientClipping(5))


    if start != 0:
        serializers.load_hdf5("{}.{:03d}.weights".format(config.model, start), nmt)
        serializers.load_hdf5("{}.{:03d}.optimizer".format(config.model, start), opt)

    for epoch in range(start, config.epoch):
        trace("Epoch {}/{}".format(epoch + 1, config.epoch))
        accum_loss = 0.0
        finished = 0
        random.seed(epoch)
        for batch_source, batch_target in random_sorted_parallel_batch(config.source_train, config.target_train, source_vocabulary, target_vocabulary, config.batch_size, config.pooling, config.library):
            nmt.zerograds()
            loss, batch_predict = nmt(batch_source, batch_target)
            accum_loss += loss.data
            loss.backward()
            opt.update()

            for source, target, predict in zip(convert_wordlist(batch_source, source_vocabulary), convert_wordlist(batch_target, target_vocabulary), convert_wordlist(batch_predict, target_vocabulary)):
                trace("Epoch {}/{}, Sample {}".format(epoch + 1, config.epoch, finished + 1))
                trace("Source  = {}".format(source))
                trace("Target  = {}".format(target))
                trace("Predict = {}".format(predict))
                finished += 1

        trace("accum_loss = {}".format(accum_loss))
        trace("Saving Model ...")
        model = "{}.{:03d}".format(config.model, epoch + 1)
        source_vocabulary.save("{}.source_vocabulary".format(model))
        target_vocabulary.save("{}.target_vocabulary".format(model))
        serializers.save_hdf5("{}.weights".format(model), nmt)
        serializers.save_hdf5("{}.optimizer".format(model), opt)

    trace("Finished.")

    "در صورتی که قصد تست شبکه را داشته باشیم تابع زیر فراخوانی می شود. با توجه به تابع مقادیر پارامترهای متناسب و کلمات ورودی و خروجی شبکه خواهنده شده. ترجمه شبکه تهیه می شود و مقدار خطای شبکه با توحه به مقدار پارامتر جستجو مشخص می گردد"


In [ ]:
def test(config):
    trace("Loading Vocabulary ...")
    source_vocabulary = Vocabulary.load("{}.source_vocabulary".format(config.model))
    target_vocabulary = Vocabulary.load("{}.target_vocabulary".format(config.model))
    config.source_vocabulary_size = source_vocabulary.size
    config.target_vocabulary_size = target_vocabulary.size

    trace("Loading Model ...")
    nmt = AttentionalNMT(config.source_vocabulary_size, config.target_vocabulary_size, config.embed_size, config.hidden_size, source_vocabulary, target_vocabulary, None, None, False, 0.0, config.generation_limit, config.use_beamsearch, config.beam_size, config.library)
    if config.use_gpu:
        cuda.get_device(config.gpu_device).use()
        nmt.to_gpu()
    serializers.load_hdf5("{}.weights".format(config.model), nmt)

    trace("Generating Translation ...")
    finished = 0
    
    with open(config.predict_file, 'w') as ft:
        for batch_source in mono_batch(config.source_file, source_vocabulary, 1, config.library):
            trace("Sample {} ...".format(finished + 1))
            _, batch_predict, _, _, _, _, _, batch_attention = nmt.forward(batch_source, None)
            for source, predict, attention in zip(convert_wordlist(batch_source, source_vocabulary), convert_wordlist(batch_predict, target_vocabulary), batch_attention.data):
                ft.write("{}\n".format(predict))
                finished += 1

if __name__ == "__main__":
    config = Configuration(sys.argv[1], sys.argv[2])
    if config.mode == "train":
        train(config)
    elif config.mode == "test":
        trace("Start Testing ...")
        config.source_file = config.source_test
        config.predict_file = "{}.test_result.beam{}".format(config.model, config.beam_size)
        config.model = "{}.{:03d}".format(config.model, int(sys.argv[3]))
        test(config)
        trace("Finished.")
    elif config.mode == "dev":
        trace("Start Developing ...")
        config.source_file = config.source_dev
        model = config.model
        if len(sys.argv) == 5:
            start = int(sys.argv[3]) - 1
            end = int(sys.argv[4])
        else:
            start = 0
            end = config.epoch
        for i in range(start, end):
            config.model = "{}.{:03d}".format(model, i + 1)
            trace("Model {}/{}".format(i + 1, config.epoch))
            config.predict_file = "{}.dev_result".format(config.model)
            test(config)
        trace("Finished.")

    "در ادامه نمونه از خروجی شبکه بر روی داده ای محدود از ترجمه زبان انگلیسی به ژاپنی را مشاهده می نماییم. خروجی زیر مربوط به زمانی است که قصد آموزش شبکه ماشین ترجمه عصبی را داشته ایم. خروجی حاصل مربوط به دور 10 اجرا و در بخش انتهایی از مرحله آموزش است."


2019-09-17 05:21:33.679009 ... Epoch 10/10, Sample 49250
2019-09-17 05:21:33.679022 ... Source  = 彼女 は 富豪 と 結婚 する 。
2019-09-17 05:21:33.679032 ... Target  = she will get married to a rich man .
2019-09-17 05:21:33.679041 ... Predict = she is marry married a a man man .
2019-09-17 05:21:33.679174 ... Epoch 10/10, Sample 49251
2019-09-17 05:21:33.679186 ... Source  = よく 悪夢 を 見 ま す 。
2019-09-17 05:21:33.679196 ... Target  = i often have nightmares .
2019-09-17 05:21:33.679206 ... Predict = i 'm see nightmares .
2019-09-17 05:21:33.679337 ... Epoch 10/10, Sample 49252
2019-09-17 05:21:33.679350 ... Source  = コーヒー 飲 み ま す か 。
2019-09-17 05:21:33.679360 ... Target  = have some coffee ?
2019-09-17 05:21:33.679369 ... Predict = would a coffee ,
2019-09-17 05:21:33.679500 ... Epoch 10/10, Sample 49253
2019-09-17 05:21:33.679513 ... Source  = 待 っ て も むだ だ 。
2019-09-17 05:21:33.679523 ... Target  = there 's no point in waiting .
2019-09-17 05:21:33.679533 ... Predict = it is no use for waiting .
2019-09-17 05:21:33.679664 ... Epoch 10/10, Sample 49254
2019-09-17 05:21:33.679677 ... Source  = 今晩 は 予定 が あ る ？
2019-09-17 05:21:33.679687 ... Target  = what are you doing tonight ?
2019-09-17 05:21:33.679697 ... Predict = what 's you going this ?
2019-09-17 05:21:33.679829 ... Epoch 10/10, Sample 49255
2019-09-17 05:21:33.679842 ... Source  = 私 だけ が 残 っ た 。
2019-09-17 05:21:33.679852 ... Target  = there was no one left but me .
2019-09-17 05:21:33.679861 ... Predict = i was a one left . i .
2019-09-17 05:21:33.680001 ... Epoch 10/10, Sample 49256
2019-09-17 05:21:33.680015 ... Source  = 私 は 遅刻 で す ね 。
2019-09-17 05:21:33.680026 ... Target  = i 'm late , aren 't i ?
2019-09-17 05:21:33.680044 ... Predict = i am late , aren 't i ?
2019-09-17 05:21:33.680176 ... Epoch 10/10, Sample 49257
2019-09-17 05:21:33.680189 ... Source  = 私 は 猫 が 好き だ 。
2019-09-17 05:21:33.680199 ... Target  = i like cats .
2019-09-17 05:21:33.680208 ... Predict = i like cats .
2019-09-17 05:21:33.680339 ... Epoch 10/10, Sample 49258
2019-09-17 05:21:33.680352 ... Source  = 彼 は 頭 が い い 。
2019-09-17 05:21:33.680362 ... Target  = he has a good brain .
2019-09-17 05:21:33.680371 ... Predict = he has a good brain .
2019-09-17 05:21:33.680503 ... Epoch 10/10, Sample 49259
2019-09-17 05:21:33.680516 ... Source  = この 教訓 を 忘れ る な 。
2019-09-17 05:21:33.680526 ... Target  = keep this lesson in mind .
2019-09-17 05:21:33.680535 ... Predict = don this lesson to order .
2019-09-17 05:21:33.680667 ... Epoch 10/10, Sample 49260
2019-09-17 05:21:33.680680 ... Source  = 脅か さ な い で よ 。
2019-09-17 05:21:33.680690 ... Target  = don 't scare me like that !
2019-09-17 05:21:33.680699 ... Predict = don 't scare me like it !
2019-09-17 05:21:33.680830 ... Epoch 10/10, Sample 49261
2019-09-17 05:21:33.680843 ... Source  = 彼 の 考え は 新し い 。
2019-09-17 05:21:33.680853 ... Target  = his ideas are up to date .
2019-09-17 05:21:33.680863 ... Predict = his ideas are new . his .
2019-09-17 05:21:33.680993 ... Epoch 10/10, Sample 49262
2019-09-17 05:21:33.681006 ... Source  = 彼女 は 歌 が うま い 。
2019-09-17 05:21:33.681016 ... Target  = she sings well .
2019-09-17 05:21:33.681025 ... Predict = she sings well .
2019-09-17 05:21:33.681157 ... Epoch 10/10, Sample 49263
2019-09-17 05:21:33.681170 ... Source  = これ は お 土産 で す 。
2019-09-17 05:21:33.681180 ... Target  = this is a little gift for you .
2019-09-17 05:21:33.681189 ... Predict = this is a little present . you .
2019-09-17 05:21:33.681320 ... Epoch 10/10, Sample 49264
2019-09-17 05:21:33.681333 ... Source  = 景気 は どう で す か 。
2019-09-17 05:21:33.681343 ... Target  = how is the economy ?
2019-09-17 05:21:33.681352 ... Predict = how do the economy ?
2019-09-17 05:21:33.681484 ... Epoch 10/10, Sample 49265
2019-09-17 05:21:33.681497 ... Source  = 考え て 返事 し ま す 。
2019-09-17 05:21:33.681507 ... Target  = i 'll think about it and get back to you .
2019-09-17 05:21:33.681516 ... Predict = i 'll answer for you . answer out to you .
2019-09-17 05:21:33.681648 ... Epoch 10/10, Sample 49266
2019-09-17 05:21:33.681661 ... Source  = 彼女 は 毎日 テニス を する 。
2019-09-17 05:21:33.681671 ... Target  = she plays tennis every day .
2019-09-17 05:21:33.681680 ... Predict = she plays tennis every day .
2019-09-17 05:21:33.681811 ... Epoch 10/10, Sample 49267
2019-09-17 05:21:33.681824 ... Source  = 私 は 学校 で 勉強 する 。
2019-09-17 05:21:33.681834 ... Target  = i study at school .
2019-09-17 05:21:33.681843 ... Predict = i study at school .
2019-09-17 05:21:33.681976 ... Epoch 10/10, Sample 49268
2019-09-17 05:21:33.681989 ... Source  = そんな 言葉 じゃ 言い表せ な い 。
2019-09-17 05:21:33.681998 ... Target  = like is hardly the word .
2019-09-17 05:21:33.682008 ... Predict = that nothing not the word .
2019-09-17 05:21:33.682141 ... Epoch 10/10, Sample 49269
2019-09-17 05:21:33.682154 ... Source  = ここ から タクシー で まにあ う 。
2019-09-17 05:21:33.682164 ... Target  = we can make it from here by taxi in time .
2019-09-17 05:21:33.682173 ... Predict = it 're catch it from here a taxi . time .
2019-09-17 05:21:33.682304 ... Epoch 10/10, Sample 49270
2019-09-17 05:21:33.682317 ... Source  = 紹介 さ せ て くださ い 。
2019-09-17 05:21:33.682327 ... Target  = let me introduce myself .
2019-09-17 05:21:33.682336 ... Predict = let me introduce you to
2019-09-17 05:21:33.682466 ... Epoch 10/10, Sample 49271
2019-09-17 05:21:33.682479 ... Source  = 彼 は 医者 で す か 。
2019-09-17 05:21:33.682489 ... Target  = is he a doctor ?
2019-09-17 05:21:33.682498 ... Predict = is he a doctor ?
2019-09-17 05:21:33.682640 ... Epoch 10/10, Sample 49272
2019-09-17 05:21:33.682654 ... Source  = 紅茶 また は コーヒー は いかが ？
2019-09-17 05:21:33.682664 ... Target  = would you like some tea or some coffee ?
2019-09-17 05:21:33.682673 ... Predict = how you like coffee tea ? ? coffee ?
2019-09-17 05:21:33.682806 ... Epoch 10/10, Sample 49273
2019-09-17 05:21:33.682819 ... Source  = 今夜 は 非番 で す か 。
2019-09-17 05:21:33.682829 ... Target  = are you off duty tonight ?
2019-09-17 05:21:33.682838 ... Predict = are you off duty tonight ?
2019-09-17 05:21:33.682969 ... Epoch 10/10, Sample 49274
2019-09-17 05:21:33.682982 ... Source  = すぐ に 行 き ま す 。
2019-09-17 05:21:33.682992 ... Target  = i 'm coming at once .
2019-09-17 05:21:33.683001 ... Predict = i 'll going at once .
2019-09-17 05:21:33.683134 ... Epoch 10/10, Sample 49275
2019-09-17 05:21:33.683155 ... Source  = 母子 とも に 健全 で す 。
2019-09-17 05:21:33.683166 ... Target  = mother and child are both doing well .
2019-09-17 05:21:33.683175 ... Predict = my and be are very with . .
2019-09-17 05:21:33.683365 ... Epoch 10/10, Sample 49276
2019-09-17 05:21:33.683378 ... Source  = あれ は 成り行き で し た 。
2019-09-17 05:21:33.683388 ... Target  = one thing led to another .
2019-09-17 05:21:33.683397 ... Predict = that is made . one .
2019-09-17 05:21:33.683538 ... Epoch 10/10, Sample 49277
2019-09-17 05:21:33.683551 ... Source  = 決して 約束 を 破 る な 。
2019-09-17 05:21:33.683562 ... Target  = never break your promise .
2019-09-17 05:21:33.683571 ... Predict = never break the promise .
2019-09-17 05:21:33.683707 ... Epoch 10/10, Sample 49278
2019-09-17 05:21:33.683720 ... Source  = 彼 は 背 を 曲げ た 。
2019-09-17 05:21:33.683730 ... Target  = he bent his back .
2019-09-17 05:21:33.683740 ... Predict = he died his back .
2019-09-17 05:21:33.683876 ... Epoch 10/10, Sample 49279
2019-09-17 05:21:33.683890 ... Source  = 心 から 感謝 し ま す 。
2019-09-17 05:21:33.683900 ... Target  = i really appreciate it .
2019-09-17 05:21:33.683910 ... Predict = i appreciate thank my .
2019-09-17 05:21:33.684046 ... Epoch 10/10, Sample 49280
2019-09-17 05:21:33.684059 ... Source  = 彼女 は 本当 に 美人 だ 。
2019-09-17 05:21:33.684069 ... Target  = she is a real beauty .
2019-09-17 05:21:33.684079 ... Predict = she is a beauty beauty .
2019-09-17 05:21:33.684216 ... Epoch 10/10, Sample 49281
2019-09-17 05:21:33.684229 ... Source  = 彼 は 速 く 走れ る 。
2019-09-17 05:21:33.684239 ... Target  = he can run fast .
2019-09-17 05:21:33.684249 ... Predict = he can run fast .
2019-09-17 05:21:33.684384 ... Epoch 10/10, Sample 49282
2019-09-17 05:21:33.684398 ... Source  = 彼女 は 明らか に 病気 だ 。
2019-09-17 05:21:33.684408 ... Target  = she is evidently sick .
2019-09-17 05:21:33.684418 ... Predict = she is ill ill .
2019-09-17 05:21:33.684554 ... Epoch 10/10, Sample 49283
2019-09-17 05:21:33.684567 ... Source  = どの バス も 満員 で す 。
2019-09-17 05:21:33.684577 ... Target  = every bus is full .
2019-09-17 05:21:33.684587 ... Predict = every bus is full .
2019-09-17 05:21:33.684722 ... Epoch 10/10, Sample 49284
2019-09-17 05:21:33.684735 ... Source  = 英語 を 話せ ま す か 。
2019-09-17 05:21:33.684745 ... Target  = can you speak english ?
2019-09-17 05:21:33.684755 ... Predict = can speak speak english ?
2019-09-17 05:21:33.684890 ... Epoch 10/10, Sample 49285
2019-09-17 05:21:33.684904 ... Source  = 彼女 は 私 に 冷淡 だ 。
2019-09-17 05:21:33.684914 ... Target  = she is cold to me .
2019-09-17 05:21:33.684924 ... Predict = she is a to me .
2019-09-17 05:21:33.685059 ... Epoch 10/10, Sample 49286
2019-09-17 05:21:33.685081 ... Source  = 一雨 き そう で す ね 。
2019-09-17 05:21:33.685092 ... Target  = the sky looks threatening .
2019-09-17 05:21:33.685110 ... Predict = it rain is threatening .
2019-09-17 05:21:33.685246 ... Epoch 10/10, Sample 49287
2019-09-17 05:21:33.685259 ... Source  = 学校 は 夏休み に な る 。
2019-09-17 05:21:33.685269 ... Target  = school will break for the summer .
2019-09-17 05:21:33.685279 ... Predict = school will be on the summer .
2019-09-17 05:21:33.685413 ... Epoch 10/10, Sample 49288
2019-09-17 05:21:33.685427 ... Source  = 彼 の 邪魔 を し た 。
2019-09-17 05:21:33.685437 ... Target  = we disturbed him .
2019-09-17 05:21:33.685447 ... Predict = i disturbed him .
2019-09-17 05:21:33.685581 ... Epoch 10/10, Sample 49289
2019-09-17 05:21:33.685595 ... Source  = 彼 は フランス 人 で す 。
2019-09-17 05:21:33.685605 ... Target  = he is french .
2019-09-17 05:21:33.685614 ... Predict = he is french .
2019-09-17 05:21:33.685749 ... Epoch 10/10, Sample 49290
2019-09-17 05:21:33.685763 ... Source  = テニス を し ま す か 。
2019-09-17 05:21:33.685773 ... Target  = do you play tennis ?
2019-09-17 05:21:33.685783 ... Predict = what you play tennis ?
2019-09-17 05:21:33.685918 ... Epoch 10/10, Sample 49291
2019-09-17 05:21:33.685932 ... Source  = 私 の 妹 は 有名 だ 。
2019-09-17 05:21:33.685943 ... Target  = my sister is famous .
2019-09-17 05:21:33.685952 ... Predict = my sister is famous .
2019-09-17 05:21:33.686088 ... Epoch 10/10, Sample 49292
2019-09-17 05:21:33.686102 ... Source  = 私 は 同じ 年齢 で す 。
2019-09-17 05:21:33.686112 ... Target  = i am the same age .
2019-09-17 05:21:33.686122 ... Predict = i am the same age .
2019-09-17 05:21:33.686258 ... Epoch 10/10, Sample 49293
2019-09-17 05:21:33.686271 ... Source  = 彼女 は 非常 に 忙し い 。
2019-09-17 05:21:33.686281 ... Target  = she is as busy as a bee .
2019-09-17 05:21:33.686291 ... Predict = she is very busy as she bee .
2019-09-17 05:21:33.686427 ... Epoch 10/10, Sample 49294
2019-09-17 05:21:33.686440 ... Source  = 彼 の 話 は おかし い 。
2019-09-17 05:21:33.686450 ... Target  = his story sounds strange .
2019-09-17 05:21:33.686460 ... Predict = his story is true .
2019-09-17 05:21:33.686602 ... Epoch 10/10, Sample 49295
2019-09-17 05:21:33.686616 ... Source  = 成功 の 望み は な い 。
2019-09-17 05:21:33.686627 ... Target  = there is no hope of success .
2019-09-17 05:21:33.686636 ... Predict = there is no hope of success .
2019-09-17 05:21:33.686772 ... Epoch 10/10, Sample 49296
2019-09-17 05:21:33.686785 ... Source  = 私 は 勉強 に 疲れ た 。
2019-09-17 05:21:33.686795 ... Target  = i was tired from studying .
2019-09-17 05:21:33.686805 ... Predict = i was tired from studying .
2019-09-17 05:21:33.686949 ... Epoch 10/10, Sample 49297
2019-09-17 05:21:33.686962 ... Source  = 何 か 飲物 を くださ い 。
2019-09-17 05:21:33.686972 ... Target  = please give me something to drink .
2019-09-17 05:21:33.686981 ... Predict = give give me something to drink .
2019-09-17 05:21:33.687112 ... Epoch 10/10, Sample 49298
2019-09-17 05:21:33.687125 ... Source  = あなた に すべて 任せ ま す 。
2019-09-17 05:21:33.687135 ... Target  = i 'll leave everything to you .
2019-09-17 05:21:33.687145 ... Predict = all 'll leave everything . you .
2019-09-17 05:21:33.687276 ... Epoch 10/10, Sample 49299
2019-09-17 05:21:33.687289 ... Source  = 彼女 は とても お しとやか だ 。
2019-09-17 05:21:33.687299 ... Target  = she is as modest as anything .
2019-09-17 05:21:33.687309 ... Predict = she is as modest as anything .
2019-09-17 05:21:33.687440 ... Epoch 10/10, Sample 49300
2019-09-17 05:21:33.687453 ... Source  = 大変 寒 く な っ た 。
2019-09-17 05:21:33.687463 ... Target  = it has become very cold .
2019-09-17 05:21:33.687472 ... Predict = it was become very cold .
2019-09-17 05:21:33.687603 ... Epoch 10/10, Sample 49301
2019-09-17 05:21:33.687616 ... Source  = 大変 、 忙し い で す 。
2019-09-17 05:21:33.687626 ... Target  = i 'm busy as a bee .
2019-09-17 05:21:33.687635 ... Predict = i 'm busy , busy bee .
2019-09-17 05:21:33.687766 ... Epoch 10/10, Sample 49302
2019-09-17 05:21:33.687779 ... Source  = 彼 は 尻 が 重 い 。
2019-09-17 05:21:33.687789 ... Target  = he is lazy .
2019-09-17 05:21:33.687799 ... Predict = he is heavy .
2019-09-17 05:21:33.687929 ... Epoch 10/10, Sample 49303
2019-09-17 05:21:33.687942 ... Source  = 彼 ら は 健康 そう だ 。
2019-09-17 05:21:33.687952 ... Target  = they look healthy .
2019-09-17 05:21:33.687961 ... Predict = they are in .
2019-09-17 05:21:33.688091 ... Epoch 10/10, Sample 49304
2019-09-17 05:21:33.688104 ... Source  = 部屋 を 片付け な さ い 。
2019-09-17 05:21:33.688114 ... Target  = put your room in order .
2019-09-17 05:21:33.688123 ... Predict = put your room off order .
2019-09-17 05:21:33.688276 ... Epoch 10/10, Sample 49305
2019-09-17 05:21:33.688290 ... Source  = 私 は ノート が ほし い 。
2019-09-17 05:21:33.688301 ... Target  = i want a notebook .
2019-09-17 05:21:33.688311 ... Predict = i want a notebook .
2019-09-17 05:21:33.688462 ... Epoch 10/10, Sample 49306
2019-09-17 05:21:33.688475 ... Source  = 腕 を 放 し て くれ 。
2019-09-17 05:21:33.688485 ... Target  = let go of my arm .
2019-09-17 05:21:33.688494 ... Predict = let me out my arm .
2019-09-17 05:21:33.688626 ... Epoch 10/10, Sample 49307
2019-09-17 05:21:33.688639 ... Source  = 彼 は 必ず 成功 する よ 。
2019-09-17 05:21:33.688649 ... Target  = he will succeed without fail .
2019-09-17 05:21:33.688658 ... Predict = he is succeed without fail .
2019-09-17 05:21:33.688789 ... Epoch 10/10, Sample 49308
2019-09-17 05:21:33.688802 ... Source  = 退屈 で 死 に そう だ 。
2019-09-17 05:21:33.688812 ... Target  = i am bored to death .
2019-09-17 05:21:33.688821 ... Predict = i think bored to death .
2019-09-17 05:21:33.688954 ... Epoch 10/10, Sample 49309
2019-09-17 05:21:33.688967 ... Source  = 結果 は 気 に する な 。
2019-09-17 05:21:33.688977 ... Target  = don 't worry about the results .
2019-09-17 05:21:33.688986 ... Predict = don 't worry about the result .
2019-09-17 05:21:33.689117 ... Epoch 10/10, Sample 49310
2019-09-17 05:21:33.689130 ... Source  = マッチ あ り ま す か 。
2019-09-17 05:21:33.689140 ... Target  = do you have a match ?
2019-09-17 05:21:33.689149 ... Predict = do you have a match ?
2019-09-17 05:21:33.689281 ... Epoch 10/10, Sample 49311
2019-09-17 05:21:33.689294 ... Source  = わあ 、 い い 天気 だなー 。
2019-09-17 05:21:33.689304 ... Target  = it 's a beautiful day !
2019-09-17 05:21:33.689313 ... Predict = it 's a nice day !
2019-09-17 05:21:33.689444 ... Epoch 10/10, Sample 49312
2019-09-17 05:21:33.689457 ... Source  = 馬鹿 な 真似 は する な 。
2019-09-17 05:21:33.689467 ... Target  = stop making a fool of yourself .
2019-09-17 05:21:33.689477 ... Predict = don a a fool of fun .
2019-09-17 05:21:33.689609 ... Epoch 10/10, Sample 49313
2019-09-17 05:21:33.689622 ... Source  = 彼 は 床 に 倒れ た 。
2019-09-17 05:21:33.689632 ... Target  = he fell down on the floor .
2019-09-17 05:21:33.689642 ... Predict = he fell against the the floor .
2019-09-17 05:21:33.689773 ... Epoch 10/10, Sample 49314
2019-09-17 05:21:33.689786 ... Source  = トミー 、 聞こえ る か い ？
2019-09-17 05:21:33.689796 ... Target  = tommy , can you hear me ?
2019-09-17 05:21:33.689806 ... Predict = do , will 't hear ? ?
2019-09-17 05:21:33.689938 ... Epoch 10/10, Sample 49315
2019-09-17 05:21:33.689951 ... Source  = 紙 を 一 枚 下さ い 。
2019-09-17 05:21:33.689961 ... Target  = give me a piece of paper .
2019-09-17 05:21:33.689971 ... Predict = give me a glass of paper .
2019-09-17 05:21:33.690101 ... Epoch 10/10, Sample 49316
2019-09-17 05:21:33.690115 ... Source  = 彼 は いつ も 夜更かし する 。
2019-09-17 05:21:33.690125 ... Target  = he always sits up late at night .
2019-09-17 05:21:33.690134 ... Predict = he always sits up late at night .
2019-09-17 05:21:33.690285 ... Epoch 10/10, Sample 49317
2019-09-17 05:21:33.690298 ... Source  = 窓 を 閉め て 下さ い 。
2019-09-17 05:21:33.690308 ... Target  = please close the window .
2019-09-17 05:21:33.690317 ... Predict = please close the window .
2019-09-17 05:21:33.690449 ... Epoch 10/10, Sample 49318
2019-09-17 05:21:33.690462 ... Source  = その 警官 は 非番 だっ た 。
2019-09-17 05:21:33.690472 ... Target  = the policeman was off duty .
2019-09-17 05:21:33.690481 ... Predict = the policeman was off duty .
2019-09-17 05:21:33.690620 ... Epoch 10/10, Sample 49319
2019-09-17 05:21:33.690633 ... Source  = 彼 に 来 て もら う 。
2019-09-17 05:21:33.690643 ... Target  = i will have him come .
2019-09-17 05:21:33.690653 ... Predict = i 'll come him come .
2019-09-17 05:21:33.690784 ... Epoch 10/10, Sample 49320
2019-09-17 05:21:33.690797 ... Source  = 彼 は いつ も そう だ 。
2019-09-17 05:21:33.690807 ... Target  = it is always the case with him .
2019-09-17 05:21:33.690816 ... Predict = he is always that case with him .
2019-09-17 05:21:33.690947 ... Epoch 10/10, Sample 49321
2019-09-17 05:21:33.690960 ... Source  = お 昼 に し よ う 。
2019-09-17 05:21:33.690970 ... Target  = let 's have lunch .
2019-09-17 05:21:33.690979 ... Predict = let 's visit lunch .
2019-09-17 05:21:33.691110 ... Epoch 10/10, Sample 49322
2019-09-17 05:21:33.691123 ... Source  = 彼 の 英語 は まあまあ だ 。
2019-09-17 05:21:33.691133 ... Target  = his english is quite good .
2019-09-17 05:21:33.691142 ... Predict = his english is well good .
2019-09-17 05:21:33.691275 ... Epoch 10/10, Sample 49323
2019-09-17 05:21:33.691288 ... Source  = ちょっと それ は やめ よ う 。
2019-09-17 05:21:33.691298 ... Target  = let 's drop the subject .
2019-09-17 05:21:33.691307 ... Predict = let 's give it rest .
2019-09-17 05:21:33.691439 ... Epoch 10/10, Sample 49324
2019-09-17 05:21:33.691452 ... Source  = 今回 が 最後 の 試合 だ 。
2019-09-17 05:21:33.691462 ... Target  = this is the last game .
2019-09-17 05:21:33.691471 ... Predict = this is the last game .
2019-09-17 05:21:33.691603 ... Epoch 10/10, Sample 49325
2019-09-17 05:21:33.691616 ... Source  = 全く その とおり で す ね 。
2019-09-17 05:21:33.691626 ... Target  = that 's absolutely right .
2019-09-17 05:21:33.691635 ... Predict = that 's all right .
2019-09-17 05:21:33.691765 ... Epoch 10/10, Sample 49326
2019-09-17 05:21:33.691778 ... Source  = 私 は 君 に 同意 する 。
2019-09-17 05:21:33.691788 ... Target  = i agree with you .
2019-09-17 05:21:33.691797 ... Predict = i agree with you .
2019-09-17 05:21:33.691928 ... Epoch 10/10, Sample 49327
2019-09-17 05:21:33.691941 ... Source  = これ は 何 で す か 。
2019-09-17 05:21:33.691951 ... Target  = what 's this ?
2019-09-17 05:21:33.691960 ... Predict = what is this ?
2019-09-17 05:21:33.692091 ... Epoch 10/10, Sample 49328
2019-09-17 05:21:33.692104 ... Source  = 時々 彼女 は テニス を する 。
2019-09-17 05:21:33.692114 ... Target  = now and then she plays tennis .
2019-09-17 05:21:33.692124 ... Predict = once she then she plays tennis .
2019-09-17 05:21:33.692257 ... Epoch 10/10, Sample 49329
2019-09-17 05:21:33.692270 ... Source  = 彼 は 過労 で 倒れ た 。
2019-09-17 05:21:33.692280 ... Target  = my father fell ill because he worked too hard .
2019-09-17 05:21:33.692289 ... Predict = he overwork fell hard in he fell . hard .
2019-09-17 05:21:33.692420 ... Epoch 10/10, Sample 49330
2019-09-17 05:21:33.692433 ... Source  = ８ 時 に 起こ し て 。
2019-09-17 05:21:33.692443 ... Target  = get me up at eight .
2019-09-17 05:21:33.692452 ... Predict = come it up at eight .
2019-09-17 05:21:33.692584 ... Epoch 10/10, Sample 49331
2019-09-17 05:21:33.692597 ... Source  = 少々 お ま ち くださ い 。
2019-09-17 05:21:33.692607 ... Target  = hold on a minute , please .
2019-09-17 05:21:33.692616 ... Predict = please on , minute , please .
2019-09-17 05:21:33.692749 ... Epoch 10/10, Sample 49332
2019-09-17 05:21:33.692761 ... Source  = 熱 い ココア を 下さ い 。
2019-09-17 05:21:33.692771 ... Target  = i 'd like to have some hot chocolate .
2019-09-17 05:21:33.692781 ... Predict = please 'd like to have some hot chocolate .
2019-09-17 05:21:33.692912 ... Epoch 10/10, Sample 49333
2019-09-17 05:21:33.692925 ... Source  = 私 は 非常 に 疲れ た 。
2019-09-17 05:21:33.692934 ... Target  = i was very tired .
2019-09-17 05:21:33.692944 ... Predict = i was very tired .
2019-09-17 05:21:33.693075 ... Epoch 10/10, Sample 49334
2019-09-17 05:21:33.693088 ... Source  = 目 を 開け な さ い 。
2019-09-17 05:21:33.693098 ... Target  = open your eyes .
2019-09-17 05:21:33.693107 ... Predict = open your eyes .
2019-09-17 05:21:33.693239 ... Epoch 10/10, Sample 49335
2019-09-17 05:21:33.693252 ... Source  = たん が たくさん 出 ま す 。
2019-09-17 05:21:33.693262 ... Target  = i have a lot of phlegm .
2019-09-17 05:21:33.693272 ... Predict = i have a lot of phlegm .
2019-09-17 05:21:33.693435 ... Epoch 10/10, Sample 49336
2019-09-17 05:21:33.693449 ... Source  = もう これ 以上 歩け な い 。
2019-09-17 05:21:33.693459 ... Target  = i can 't walk any farther .
2019-09-17 05:21:33.693469 ... Predict = i can 't walk any more .
2019-09-17 05:21:33.693609 ... Epoch 10/10, Sample 49337
2019-09-17 05:21:33.693622 ... Source  = あなた は 仕事 を 続け て 。
2019-09-17 05:21:33.693632 ... Target  = carry on with your work .
2019-09-17 05:21:33.693641 ... Predict = go on with your work .
2019-09-17 05:21:33.693773 ... Epoch 10/10, Sample 49338
2019-09-17 05:21:33.693786 ... Source  = 彼女 は 僕 と 同い年 だ 。
2019-09-17 05:21:33.693796 ... Target  = she and i are of an age .
2019-09-17 05:21:33.693805 ... Predict = she is i are the the age .
2019-09-17 05:21:33.693937 ... Epoch 10/10, Sample 49339
2019-09-17 05:21:33.693950 ... Source  = 健 は 泳げ ま す ね 。
2019-09-17 05:21:33.693960 ... Target  = ken can swim , can 't he ?
2019-09-17 05:21:33.693969 ... Predict = ken can swim . can 't he ?
2019-09-17 05:21:33.694102 ... Epoch 10/10, Sample 49340
2019-09-17 05:21:33.694115 ... Source  = ニューヨーク は 巨大 な 都市 だ 。
2019-09-17 05:21:33.694125 ... Target  = new york is a huge city .
2019-09-17 05:21:33.694134 ... Predict = new york is a huge city .
2019-09-17 05:21:33.694266 ... Epoch 10/10, Sample 49341
2019-09-17 05:21:33.694279 ... Source  = 初め の うち は 難し い 。
2019-09-17 05:21:33.694289 ... Target  = at first , it is difficult .
2019-09-17 05:21:33.694298 ... Predict = the all , it is difficult .
2019-09-17 05:21:33.694430 ... Epoch 10/10, Sample 49342
2019-09-17 05:21:33.694443 ... Source  = いっしょ に 昼食 で も どう ？
2019-09-17 05:21:33.694453 ... Target  = why don 't we have lunch together ?
2019-09-17 05:21:33.694462 ... Predict = how don 't you join lunch for ?
2019-09-17 05:21:33.694602 ... Epoch 10/10, Sample 49343
2019-09-17 05:21:33.694616 ... Source  = 吉田 、 ゆっくり や れ よ 。
2019-09-17 05:21:33.694626 ... Target  = take your time , yoshida .
2019-09-17 05:21:33.694635 ... Predict = take your time , and .
2019-09-17 05:21:33.694767 ... Epoch 10/10, Sample 49344
2019-09-17 05:21:33.694780 ... Source  = 彼 は 出張 中 で す 。
2019-09-17 05:21:33.694790 ... Target  = he 's out of town on business .
2019-09-17 05:21:33.694799 ... Predict = he is a of business . business .
2019-09-17 05:21:33.694931 ... Epoch 10/10, Sample 49345
2019-09-17 05:21:33.694944 ... Source  = 彼 は 老け て 見え る 。
2019-09-17 05:21:33.694954 ... Target  = he looks old for his age .
2019-09-17 05:21:33.694963 ... Predict = he looks old for his age .
2019-09-17 05:21:33.695094 ... Epoch 10/10, Sample 49346
2019-09-17 05:21:33.695107 ... Source  = 何 か 食べ物 が 欲し い 。
2019-09-17 05:21:33.695117 ... Target  = i want something to eat .
2019-09-17 05:21:33.695126 ... Predict = i want something to eat .
2019-09-17 05:21:33.695276 ... Epoch 10/10, Sample 49347
2019-09-17 05:21:33.695289 ... Source  = 星 が 出 て き た 。
2019-09-17 05:21:33.695299 ... Target  = the stars came out .
2019-09-17 05:21:33.695308 ... Predict = the stars came out .
2019-09-17 05:21:33.695439 ... Epoch 10/10, Sample 49348
2019-09-17 05:21:33.695452 ... Source  = シアトル に 行 き た い 。
2019-09-17 05:21:33.695462 ... Target  = i want to go to seattle .
2019-09-17 05:21:33.695471 ... Predict = i want to go to seattle .
2019-09-17 05:21:33.695602 ... Epoch 10/10, Sample 49349
2019-09-17 05:21:33.695615 ... Source  = 浣腸 を し ま し た 。
2019-09-17 05:21:33.695625 ... Target  = i gave him an enema .
2019-09-17 05:21:33.695634 ... Predict = i gave him an enema .
2019-09-17 05:21:33.695764 ... Epoch 10/10, Sample 49350
2019-09-17 05:21:33.695777 ... Source  = その 家 は よさ そう だ 。
2019-09-17 05:21:33.695787 ... Target  = that house looks nice .
2019-09-17 05:21:33.695797 ... Predict = the house seems well .
2019-09-17 05:21:33.695928 ... Epoch 10/10, Sample 49351
2019-09-17 05:21:33.695941 ... Source  = 彼 は 泳ぎ が うま い 。
2019-09-17 05:21:33.695951 ... Target  = he can swim well .
2019-09-17 05:21:33.695961 ... Predict = he is swim well .
2019-09-17 05:21:33.696091 ... Epoch 10/10, Sample 49352
2019-09-17 05:21:33.696103 ... Source  = 私 自身 が や っ た 。
2019-09-17 05:21:33.696113 ... Target  = i myself did it .
2019-09-17 05:21:33.696123 ... Predict = i am gave . .
2019-09-17 05:21:33.696254 ... Epoch 10/10, Sample 49353
2019-09-17 05:21:33.696267 ... Source  = ネクタイ は 必要 で す か 。
2019-09-17 05:21:33.696277 ... Target  = do i need a tie ?
2019-09-17 05:21:33.696286 ... Predict = do i need a tie ?
2019-09-17 05:21:33.696417 ... Epoch 10/10, Sample 49354
2019-09-17 05:21:33.696430 ... Source  = 相 変わ ら ず 忙し い 。
2019-09-17 05:21:33.696440 ... Target  = i 'm as busy as ever .
2019-09-17 05:21:33.696449 ... Predict = i 'm all busy as ever .
2019-09-17 05:21:33.696580 ... Epoch 10/10, Sample 49355
2019-09-17 05:21:33.696593 ... Source  = 帰 り た く な い 。
2019-09-17 05:21:33.696603 ... Target  = i don 't wanna go back .
2019-09-17 05:21:33.696612 ... Predict = i don 't want go back .
2019-09-17 05:21:33.696744 ... Epoch 10/10, Sample 49356
2019-09-17 05:21:33.696757 ... Source  = タバコ は 健康 に 悪 い 。
2019-09-17 05:21:33.696767 ... Target  = smoking is harmful to your health .
2019-09-17 05:21:33.696776 ... Predict = smoking is bad to your health .
2019-09-17 05:21:33.696908 ... Epoch 10/10, Sample 49357
2019-09-17 05:21:33.696920 ... Source  = 彼 は それ に 決め た 。
2019-09-17 05:21:33.696930 ... Target  = he decided on that .
2019-09-17 05:21:33.696940 ... Predict = he decided on it .
2019-09-17 05:21:33.697071 ... Epoch 10/10, Sample 49358
2019-09-17 05:21:33.697084 ... Source  = 寝 た 方 が 良 い 。
2019-09-17 05:21:33.697094 ... Target  = you may as well as go to bed now .
2019-09-17 05:21:33.697103 ... Predict = you had as well go go to bed . .
2019-09-17 05:21:33.697235 ... Epoch 10/10, Sample 49359
2019-09-17 05:21:33.697248 ... Source  = それ は 高 く な い 。
2019-09-17 05:21:33.697257 ... Target  = it isn 't expensive .
2019-09-17 05:21:33.697267 ... Predict = it is 't expensive .
2019-09-17 05:21:33.697398 ... Epoch 10/10, Sample 49360
2019-09-17 05:21:33.697411 ... Source  = いいえ 、 眠 い で す 。
2019-09-17 05:21:33.697421 ... Target  = no , i 'm tired .
2019-09-17 05:21:33.697430 ... Predict = no , it 'm sleepy .
2019-09-17 05:21:37.163839 ... Epoch 10/10, Sample 49361
2019-09-17 05:21:37.163882 ... Source  = しかし 我々 は 彼 なし で 続行 し た 。
2019-09-17 05:21:37.163895 ... Target  = but we carried on without him .
2019-09-17 05:21:37.163906 ... Predict = but we carried on without him .
2019-09-17 05:21:37.164067 ... Epoch 10/10, Sample 49362
2019-09-17 05:21:37.164082 ... Source  = 彼女 は ドア を 引 い て 開け た 。
2019-09-17 05:21:37.164092 ... Target  = she pulled the door open .
2019-09-17 05:21:37.164102 ... Predict = she opened the door open .
2019-09-17 05:21:37.164263 ... Epoch 10/10, Sample 49363
2019-09-17 05:21:37.164277 ... Source  = 私 達 は 偶然 同じ バス に 乗り合わせ た 。
2019-09-17 05:21:37.164287 ... Target  = we happened to get on the same bus .
2019-09-17 05:21:37.164296 ... Predict = we came to be on the same bus .
2019-09-17 05:21:37.164451 ... Epoch 10/10, Sample 49364
2019-09-17 05:21:37.164465 ... Source  = 私 は この 辞書 を 買 い た い 。
2019-09-17 05:21:37.164475 ... Target  = i want to buy this dictionary .
2019-09-17 05:21:37.164485 ... Predict = i want to buy this dictionary .
2019-09-17 05:21:37.164642 ... Epoch 10/10, Sample 49365
2019-09-17 05:21:37.164656 ... Source  = 暑 い 天気 が 長 く 続 い た 。
2019-09-17 05:21:37.164666 ... Target  = we have had a long spell of hot weather .
2019-09-17 05:21:37.164675 ... Predict = we had kept a long weather of hot weather .
2019-09-17 05:21:37.164831 ... Epoch 10/10, Sample 49366
2019-09-17 05:21:37.164844 ... Source  = 彼 は もう ここ に 来 て い る 。
2019-09-17 05:21:37.164854 ... Target  = he is already here .
2019-09-17 05:21:37.164863 ... Predict = he is already here .
2019-09-17 05:21:37.165017 ... Epoch 10/10, Sample 49367
2019-09-17 05:21:37.165031 ... Source  = 私 は そこ に い て よ かっ た 。
2019-09-17 05:21:37.165041 ... Target  = i 'm glad i was there .
2019-09-17 05:21:37.165050 ... Predict = i wanted glad i was there .
2019-09-17 05:21:37.165203 ... Epoch 10/10, Sample 49368
2019-09-17 05:21:37.165217 ... Source  = 彼 は 仕事 で そこ に 行 っ た 。
2019-09-17 05:21:37.165227 ... Target  = he went there on business .
2019-09-17 05:21:37.165236 ... Predict = he went there by business .
2019-09-17 05:21:37.165392 ... Epoch 10/10, Sample 49369
2019-09-17 05:21:37.165405 ... Source  = 私 が 父 の 代理 を する つもり だ 。
2019-09-17 05:21:37.165415 ... Target  = i 'll take my father 's place .
2019-09-17 05:21:37.165424 ... Predict = i will take my father 's place .
2019-09-17 05:21:37.165580 ... Epoch 10/10, Sample 49370
2019-09-17 05:21:37.165593 ... Source  = この 情報 は 内密 に し て くださ い 。
2019-09-17 05:21:37.165603 ... Target  = keep this information under your hat .
2019-09-17 05:21:37.165613 ... Predict = please this information to yourself own .
2019-09-17 05:21:37.165769 ... Epoch 10/10, Sample 49371
2019-09-17 05:21:37.165782 ... Source  = 時々 彼 ら は 野球 を し ま す 。
2019-09-17 05:21:37.165792 ... Target  = they sometimes play baseball .
2019-09-17 05:21:37.165801 ... Predict = they are play baseball .
2019-09-17 05:21:37.165955 ... Epoch 10/10, Sample 49372
2019-09-17 05:21:37.165968 ... Source  = 彼 は すぐ に 帰 っ て き た 。
2019-09-17 05:21:37.165978 ... Target  = he came back soon .
2019-09-17 05:21:37.165987 ... Predict = he came back soon .
2019-09-17 05:21:37.166143 ... Epoch 10/10, Sample 49373
2019-09-17 05:21:37.166157 ... Source  = 彼女 は 怒 っ て 私 を 見 た 。
2019-09-17 05:21:37.166167 ... Target  = she stared me down with anger .
2019-09-17 05:21:37.166176 ... Predict = she looked me at . the .
2019-09-17 05:21:37.166329 ... Epoch 10/10, Sample 49374
2019-09-17 05:21:37.166343 ... Source  = それ を 私 に ファックス し て くださ い 。
2019-09-17 05:21:37.166353 ... Target  = please send it to me by fax .
2019-09-17 05:21:37.166362 ... Predict = please send it to me in fax .
2019-09-17 05:21:37.166520 ... Epoch 10/10, Sample 49375
2019-09-17 05:21:37.166542 ... Source  = その 両親 は 息子 に 期待 し すぎ だ 。
2019-09-17 05:21:37.166554 ... Target  = the parents expected too much of their son .
2019-09-17 05:21:37.166563 ... Predict = the parents often too much for his son .
2019-09-17 05:21:37.166718 ... Epoch 10/10, Sample 49376
2019-09-17 05:21:37.166732 ... Source  = 彼 は いく つ か 訂正 を し た 。
2019-09-17 05:21:37.166742 ... Target  = he made several corrections .
2019-09-17 05:21:37.166751 ... Predict = he made several corrections .
2019-09-17 05:21:37.166907 ... Epoch 10/10, Sample 49377
2019-09-17 05:21:37.166921 ... Source  = 大雪 の ため 列車 は １０ 分 遅れ た 。
2019-09-17 05:21:37.166931 ... Target  = because of the heavy snow , the train was 10 minutes late .
2019-09-17 05:21:37.166940 ... Predict = the of the heavy snow , the train was ten minutes late .
2019-09-17 05:21:37.167137 ... Epoch 10/10, Sample 49378
2019-09-17 05:21:37.167159 ... Source  = 次 に 踊 っ て もらえ ま す か 。
2019-09-17 05:21:37.167170 ... Target  = may i have your next dance ?
2019-09-17 05:21:37.167179 ... Predict = can i dance a next dance ?
2019-09-17 05:21:37.167335 ... Epoch 10/10, Sample 49379
2019-09-17 05:21:37.167349 ... Source  = この 手紙 を 必ず 投函 し て 下さ い 。
2019-09-17 05:21:37.167359 ... Target  = don 't fail to mail this letter .
2019-09-17 05:21:37.167368 ... Predict = please 't fail to mail this letter .
2019-09-17 05:21:37.167523 ... Epoch 10/10, Sample 49380
2019-09-17 05:21:37.167537 ... Source  = 今朝 から 本 を ３ 冊 読 ん だ 。
2019-09-17 05:21:37.167547 ... Target  = i have read three books since this morning .
2019-09-17 05:21:37.167556 ... Predict = i have read three books since this morning .
2019-09-17 05:21:37.167713 ... Epoch 10/10, Sample 49381
2019-09-17 05:21:37.167726 ... Source  = 彼女 の 母親 は ピアノ が じょうず で す 。
2019-09-17 05:21:37.167736 ... Target  = her mother is a good pianist .
2019-09-17 05:21:37.167746 ... Predict = her mother is a good pianist .
2019-09-17 05:21:37.167900 ... Epoch 10/10, Sample 49382
2019-09-17 05:21:37.167913 ... Source  = 彼 を 知 っ て い る か って ？
2019-09-17 05:21:37.167923 ... Target  = do i know him ?
2019-09-17 05:21:37.167932 ... Predict = do you know him ?
2019-09-17 05:21:37.168089 ... Epoch 10/10, Sample 49383
2019-09-17 05:21:37.168102 ... Source  = 息子 に 家 を 新築 し て あげ た 。
2019-09-17 05:21:37.168112 ... Target  = i built my son a new house .
2019-09-17 05:21:37.168122 ... Predict = i had my son a new house .
2019-09-17 05:21:37.168275 ... Epoch 10/10, Sample 49384
2019-09-17 05:21:37.168288 ... Source  = としお 君 は 誰 が 好き で す か 。
2019-09-17 05:21:37.168298 ... Target  = who does toshio like ?
2019-09-17 05:21:37.168308 ... Predict = who is toshio like ?
2019-09-17 05:21:37.168463 ... Epoch 10/10, Sample 49385
2019-09-17 05:21:37.168477 ... Source  = 彼 は 私 を 見 る と 逃げ た 。
2019-09-17 05:21:37.168487 ... Target  = he ran away at the sight of me .
2019-09-17 05:21:37.168496 ... Predict = he ran away at the sight of me .
2019-09-17 05:21:37.168650 ... Epoch 10/10, Sample 49386
2019-09-17 05:21:37.168663 ... Source  = これ が 私 の 考え た こと で す 。
2019-09-17 05:21:37.168673 ... Target  = this is what i thought .
2019-09-17 05:21:37.168683 ... Predict = this is what i thought .
2019-09-17 05:21:37.168840 ... Epoch 10/10, Sample 49387
2019-09-17 05:21:37.168853 ... Source  = やりくり する ため に 一生 懸命 働 い た 。
2019-09-17 05:21:37.168863 ... Target  = we worked hard to make ends meet .
2019-09-17 05:21:37.168873 ... Predict = we worked hard to leave myself account .
2019-09-17 05:21:37.169028 ... Epoch 10/10, Sample 49388
2019-09-17 05:21:37.169041 ... Source  = 彼 は 一足 の 靴 を 買 っ た 。
2019-09-17 05:21:37.169051 ... Target  = he bought a pair of shoes .
2019-09-17 05:21:37.169060 ... Predict = he bought a pair of shoes .
2019-09-17 05:21:37.169215 ... Epoch 10/10, Sample 49389
2019-09-17 05:21:37.169229 ... Source  = そこ から その 家 の 屋根 が 見え る 。
2019-09-17 05:21:37.169239 ... Target  = you can see the roof of the house from there .
2019-09-17 05:21:37.169248 ... Predict = we can see the roof of the house from there .
2019-09-17 05:21:37.169413 ... Epoch 10/10, Sample 49390
2019-09-17 05:21:37.169426 ... Source  = 新し い 語 を 辞書 で 調べ なさ い 。
2019-09-17 05:21:37.169445 ... Target  = look up the new words in your dictionary .
2019-09-17 05:21:37.169455 ... Predict = look at the new language in your dictionary .
2019-09-17 05:21:37.169610 ... Epoch 10/10, Sample 49391
2019-09-17 05:21:37.169623 ... Source  = 彼 は どう し た の で す か 。
2019-09-17 05:21:37.169633 ... Target  = what is the matter with him ?
2019-09-17 05:21:37.169642 ... Predict = what is it matter with him ?
2019-09-17 05:21:37.169796 ... Epoch 10/10, Sample 49392
2019-09-17 05:21:37.169809 ... Source  = もう 一 度 彼 に 会 う つもり だ 。
2019-09-17 05:21:37.169819 ... Target  = i 'll see him again .
2019-09-17 05:21:37.169828 ... Predict = i 'm see him once .
2019-09-17 05:21:37.169985 ... Epoch 10/10, Sample 49393
2019-09-17 05:21:37.169999 ... Source  = 彼 は 試験 に 一番 に 合格 し た 。
2019-09-17 05:21:37.170009 ... Target  = he passed the examination with the highest score .
2019-09-17 05:21:37.170018 ... Predict = he passed the examination in the best . .
2019-09-17 05:21:37.170174 ... Epoch 10/10, Sample 49394
2019-09-17 05:21:37.170188 ... Source  = 彼 は 来週 ロンドン へ 出張 する 予定 だ 。
2019-09-17 05:21:37.170198 ... Target  = he will make a business trip to london next week .
2019-09-17 05:21:37.170207 ... Predict = he is make a job job to london next week .
2019-09-17 05:21:37.170364 ... Epoch 10/10, Sample 49395
2019-09-17 05:21:37.170377 ... Source  = おなか が す い て 死 に そう だ 。
2019-09-17 05:21:37.170387 ... Target  = i 'm starving !
2019-09-17 05:21:37.170396 ... Predict = i 'm dying .
2019-09-17 05:21:37.170558 ... Epoch 10/10, Sample 49396
2019-09-17 05:21:37.170572 ... Source  = 彼 は とても 感じ の い い 人 だ 。
2019-09-17 05:21:37.170582 ... Target  = he is a very agreeable person .
2019-09-17 05:21:37.170592 ... Predict = he is a very nice person .
2019-09-17 05:21:37.170748 ... Epoch 10/10, Sample 49397
2019-09-17 05:21:37.170761 ... Source  = それ を 買 う お 金 が な い 。
2019-09-17 05:21:37.170771 ... Target  = i have no money to buy it .
2019-09-17 05:21:37.170781 ... Predict = i have no money to buy it .
2019-09-17 05:21:37.170936 ... Epoch 10/10, Sample 49398
2019-09-17 05:21:37.170949 ... Source  = 新し い 髭剃り を 買 い ま し た 。
2019-09-17 05:21:37.170959 ... Target  = i bought a new shaver .
2019-09-17 05:21:37.170969 ... Predict = i bought a new shaver .
2019-09-17 05:21:37.171124 ... Epoch 10/10, Sample 49399
2019-09-17 05:21:37.171137 ... Source  = 私 は 新し い バッグ を 買 っ た 。
2019-09-17 05:21:37.171147 ... Target  = i bought a new bag .
2019-09-17 05:21:37.171156 ... Predict = i bought a new bag .
2019-09-17 05:21:37.171311 ... Epoch 10/10, Sample 49400
2019-09-17 05:21:37.171324 ... Source  = 君 の 誕生 日 は いつ で す か 。
2019-09-17 05:21:37.171334 ... Target  = when is your birthday ?
2019-09-17 05:21:37.171343 ... Predict = what will your birthday ?
2019-09-17 05:21:37.171501 ... Epoch 10/10, Sample 49401
2019-09-17 05:21:37.171514 ... Source  = くだ ら な い 話 は もう たくさん だ 。
2019-09-17 05:21:37.171524 ... Target  = we have had enough of empty words .
2019-09-17 05:21:37.171533 ... Predict = i have had enough of them words .
2019-09-17 05:21:37.171689 ... Epoch 10/10, Sample 49402
2019-09-17 05:21:37.171702 ... Source  = ちょっと 調べ て る こと が あ る から 。
2019-09-17 05:21:37.171712 ... Target  = i just wanna nose around a bit .
2019-09-17 05:21:37.171721 ... Predict = i 've have look it it little .
2019-09-17 05:21:37.171876 ... Epoch 10/10, Sample 49403
2019-09-17 05:21:37.171889 ... Source  = 彼 は 一番 を と る だ ろ う 。
2019-09-17 05:21:37.171899 ... Target  = he will win the first prize .
2019-09-17 05:21:37.171908 ... Predict = he will take the best . .
2019-09-17 05:21:37.172063 ... Epoch 10/10, Sample 49404
2019-09-17 05:21:37.172076 ... Source  = あの 少年 は 彼 の 兄 さん で す 。
2019-09-17 05:21:37.172095 ... Target  = that boy is his brother .
2019-09-17 05:21:37.172105 ... Predict = that boy is his brother .
2019-09-17 05:21:37.172285 ... Epoch 10/10, Sample 49405
2019-09-17 05:21:37.172298 ... Source  = 部屋 は 墓場 の よう に 静か だっ た 。
2019-09-17 05:21:37.172308 ... Target  = the room was as still as the grave .
2019-09-17 05:21:37.172317 ... Predict = the room was quiet quiet as the grave .
2019-09-17 05:21:37.172472 ... Epoch 10/10, Sample 49406
2019-09-17 05:21:37.172485 ... Source  = 我々 は 試合 を 見 て 楽し ん だ 。
2019-09-17 05:21:37.172495 ... Target  = we enjoyed watching the game .
2019-09-17 05:21:37.172505 ... Predict = we enjoyed watching the game .
2019-09-17 05:21:37.172660 ... Epoch 10/10, Sample 49407
2019-09-17 05:21:37.172673 ... Source  = 試着 し て も い い で す か 。
2019-09-17 05:21:37.172683 ... Target  = i 'd like to try this on .
2019-09-17 05:21:37.172692 ... Predict = may 'm like to try on on .
2019-09-17 05:21:37.172846 ... Epoch 10/10, Sample 49408
2019-09-17 05:21:37.172859 ... Source  = トニー 君 は そこ に い ま す か 。
2019-09-17 05:21:37.172869 ... Target  = is tony there ?
2019-09-17 05:21:37.172879 ... Predict = is tony there ?
2019-09-17 05:21:37.173034 ... Epoch 10/10, Sample 49409
2019-09-17 05:21:37.173047 ... Source  = この き の こ は 食べ られ な い 。
2019-09-17 05:21:37.173057 ... Target  = this mushroom is not good to eat .
2019-09-17 05:21:37.173066 ... Predict = i mushroom is not eat for eat .
2019-09-17 05:21:37.173219 ... Epoch 10/10, Sample 49410
2019-09-17 05:21:37.173232 ... Source  = それ を 聞 い て 大変 残念 で す 。
2019-09-17 05:21:37.173242 ... Target  = i 'm very sorry to hear that .
2019-09-17 05:21:37.173251 ... Predict = i 'm very sorry to hear that .
2019-09-17 05:21:37.173405 ... Epoch 10/10, Sample 49411
2019-09-17 05:21:37.173418 ... Source  = あの レコード は この 店 で 買 っ た 。
2019-09-17 05:21:37.173428 ... Target  = i bought that record in this store .
2019-09-17 05:21:37.173437 ... Predict = i bought that record at this store .
2019-09-17 05:21:37.173593 ... Epoch 10/10, Sample 49412
2019-09-17 05:21:37.173606 ... Source  = 彼 ら は ロバート を ボブ と 呼 ぶ 。
2019-09-17 05:21:37.173616 ... Target  = they call robert '' bob '' .
2019-09-17 05:21:37.173626 ... Predict = they call robert from bob . .
2019-09-17 05:21:37.173782 ... Epoch 10/10, Sample 49413
2019-09-17 05:21:37.173795 ... Source  = 彼 は 多く の 苦難 を 経験 し た 。
2019-09-17 05:21:37.173805 ... Target  = he had to go through a lot of hardships .
2019-09-17 05:21:37.173814 ... Predict = he made a experience in a lot of hardships .
2019-09-17 05:21:37.173969 ... Epoch 10/10, Sample 49414
2019-09-17 05:21:37.173982 ... Source  = 私 は 彼女 を 幸福 に し よ う 。
2019-09-17 05:21:37.173992 ... Target  = i made her happy .
2019-09-17 05:21:37.174002 ... Predict = i will her happy .
2019-09-17 05:21:37.174159 ... Epoch 10/10, Sample 49415
2019-09-17 05:21:37.174172 ... Source  = 私 は ライオン を 見 て 驚 い た 。
2019-09-17 05:21:37.174182 ... Target  = i was surprised to see a lion .
2019-09-17 05:21:37.174191 ... Predict = i was surprised at see a lion .
2019-09-17 05:21:37.174345 ... Epoch 10/10, Sample 49416
2019-09-17 05:21:37.174358 ... Source  = 番号 を お 間違い の よう で す が 。
2019-09-17 05:21:37.174368 ... Target  = you seem to have the wrong number .
2019-09-17 05:21:37.174378 ... Predict = may seem to have your wrong number .
2019-09-17 05:21:37.174539 ... Epoch 10/10, Sample 49417
2019-09-17 05:21:37.174553 ... Source  = 私 に と っ て スケート は 難し い 。
2019-09-17 05:21:37.174563 ... Target  = it is difficult for me to skate .
2019-09-17 05:21:37.174573 ... Predict = it is difficult for me to skate .
2019-09-17 05:21:37.174728 ... Epoch 10/10, Sample 49418
2019-09-17 05:21:37.174741 ... Source  = 私 の 部屋 を 覗 か な い で 。
2019-09-17 05:21:37.174751 ... Target  = don 't look into my room .
2019-09-17 05:21:37.174760 ... Predict = don 't look into my room .
2019-09-17 05:21:37.174915 ... Epoch 10/10, Sample 49419
2019-09-17 05:21:37.174928 ... Source  = どこ の 席 が い い で す か 。
2019-09-17 05:21:37.174938 ... Target  = what seats do you want ?
2019-09-17 05:21:37.174947 ... Predict = where seat do you want ?
2019-09-17 05:21:37.175102 ... Epoch 10/10, Sample 49420
2019-09-17 05:21:37.175115 ... Source  = 彼 は 自分 で 時計 を 修理 し た 。
2019-09-17 05:21:37.175125 ... Target  = he repaired his watch by himself .
2019-09-17 05:21:37.175135 ... Predict = he fixed his watch by himself .
2019-09-17 05:21:37.175291 ... Epoch 10/10, Sample 49421
2019-09-17 05:21:37.175304 ... Source  = 犬 は 男 の そば に 座 っ た 。
2019-09-17 05:21:37.175314 ... Target  = the dog sat down by the man .
2019-09-17 05:21:37.175323 ... Predict = the dog sat by by the man .
2019-09-17 05:21:37.175476 ... Epoch 10/10, Sample 49422
2019-09-17 05:21:37.175489 ... Source  = あの 人 は 口 と 腹 が 反対 だ 。
2019-09-17 05:21:37.175500 ... Target  = he says one thing and means another .
2019-09-17 05:21:37.175509 ... Predict = he is the of to a . .
2019-09-17 05:21:37.175663 ... Epoch 10/10, Sample 49423
2019-09-17 05:21:37.175677 ... Source  = 彼女 は 私 を 軽蔑 し て い る 。
2019-09-17 05:21:37.175687 ... Target  = she has scorn for me .
2019-09-17 05:21:37.175696 ... Predict = she looks scorn on me .
2019-09-17 05:21:37.175851 ... Epoch 10/10, Sample 49424
2019-09-17 05:21:37.175864 ... Source  = 式典 は 彼 の 話 から 始ま っ た 。
2019-09-17 05:21:37.175874 ... Target  = the ceremony began with his speech .
2019-09-17 05:21:37.175884 ... Predict = the poll began to his speech .
2019-09-17 05:21:37.176038 ... Epoch 10/10, Sample 49425
2019-09-17 05:21:37.176051 ... Source  = この 部屋 は 日当たり が 良 く な い 。
2019-09-17 05:21:37.176061 ... Target  = this room doesn 't get much sunshine .
2019-09-17 05:21:37.176071 ... Predict = this room doesn 't get a sunshine .
2019-09-17 05:21:37.176225 ... Epoch 10/10, Sample 49426
2019-09-17 05:21:37.176238 ... Source  = その 赤ん坊 は 泣 い て ばかり い た 。
2019-09-17 05:21:37.176248 ... Target  = the baby did nothing but cry .
2019-09-17 05:21:37.176258 ... Predict = the baby did nothing but cry .
2019-09-17 05:21:37.176412 ... Epoch 10/10, Sample 49427
2019-09-17 05:21:37.176425 ... Source  = 私 の 荷物 が 壊れ て い ま す 。
2019-09-17 05:21:37.176435 ... Target  = my suitcase is broken .
2019-09-17 05:21:37.176444 ... Predict = my baggage is broken .
2019-09-17 05:21:37.176599 ... Epoch 10/10, Sample 49428
2019-09-17 05:21:37.176612 ... Source  = 先月 は とても 忙し い 月 で し た 。
2019-09-17 05:21:37.176622 ... Target  = it was a very busy month last month .
2019-09-17 05:21:37.176632 ... Predict = we was very very busy last last month .
2019-09-17 05:21:37.176787 ... Epoch 10/10, Sample 49429
2019-09-17 05:21:37.176801 ... Source  = 今 は 食べ る 気 が し な い 。
2019-09-17 05:21:37.176811 ... Target  = i don 't feel like eating now .
2019-09-17 05:21:37.176820 ... Predict = i don 't feel like eating now .
2019-09-17 05:21:37.176973 ... Epoch 10/10, Sample 49430
2019-09-17 05:21:37.176986 ... Source  = 早 く これ ば よ かっ た の に 。
2019-09-17 05:21:37.176996 ... Target  = you should have come earlier .
2019-09-17 05:21:37.177005 ... Predict = i had have been early .
2019-09-17 05:21:37.177169 ... Epoch 10/10, Sample 49431
2019-09-17 05:21:37.177183 ... Source  = 乗 る 時間 は どの くらい で す か 。
2019-09-17 05:21:37.177194 ... Target  = how long is the ride ?
2019-09-17 05:21:37.177204 ... Predict = how long is the ride ?
2019-09-17 05:21:37.177379 ... Epoch 10/10, Sample 49432
2019-09-17 05:21:37.177393 ... Source  = 彼 ら は 川 に 橋 を かけ た 。
2019-09-17 05:21:37.177403 ... Target  = they built a bridge across the river .
2019-09-17 05:21:37.177412 ... Predict = they gave the bridge in the river .
2019-09-17 05:21:37.177568 ... Epoch 10/10, Sample 49433
2019-09-17 05:21:37.177581 ... Source  = 彼 は ほとんど 死 ん だ も 同然 だ 。
2019-09-17 05:21:37.177591 ... Target  = he is all but dead .
2019-09-17 05:21:37.177601 ... Predict = he is no but dead .
2019-09-17 05:21:37.177755 ... Epoch 10/10, Sample 49434
2019-09-17 05:21:37.177769 ... Source  = この 小説 を 読 ん で 楽し かっ た 。
2019-09-17 05:21:37.177779 ... Target  = i have enjoyed reading this novel .
2019-09-17 05:21:37.177788 ... Predict = i enjoyed enjoyed reading this novel .
2019-09-17 05:21:37.177943 ... Epoch 10/10, Sample 49435
2019-09-17 05:21:37.177956 ... Source  = 少し 待 っ て いただけ ま せ ん か 。
2019-09-17 05:21:37.177966 ... Target  = would you mind waiting a moment ?
2019-09-17 05:21:37.177975 ... Predict = would you please waiting a little ?
2019-09-17 05:21:37.178130 ... Epoch 10/10, Sample 49436
2019-09-17 05:21:37.178143 ... Source  = 私 は 後 で 君 たち に 合流 する 。
2019-09-17 05:21:37.178153 ... Target  = i 'll join you later .
2019-09-17 05:21:37.178163 ... Predict = i 'll join you later .
2019-09-17 05:21:37.178317 ... Epoch 10/10, Sample 49437
2019-09-17 05:21:37.178330 ... Source  = 私 は 古 い ランプ を 買 っ た 。
2019-09-17 05:21:37.178340 ... Target  = i bought an old lamp .
2019-09-17 05:21:37.178350 ... Predict = i bought an old lamp .
2019-09-17 05:21:37.178504 ... Epoch 10/10, Sample 49438
2019-09-17 05:21:37.178517 ... Source  = たった 今 その 事故 を 見 た ばかり だ 。
2019-09-17 05:21:37.178527 ... Target  = i only just saw the accident .
2019-09-17 05:21:37.178542 ... Predict = i just just seen the accident .
2019-09-17 05:21:37.178700 ... Epoch 10/10, Sample 49439
2019-09-17 05:21:37.178713 ... Source  = 彼 の 夢 が つい に 実現 し た 。
2019-09-17 05:21:37.178724 ... Target  = his dream has come true at last .
2019-09-17 05:21:37.178733 ... Predict = his dream has come true at last .
2019-09-17 05:21:37.178888 ... Epoch 10/10, Sample 49440
2019-09-17 05:21:37.178901 ... Source  = ご 署名 を お 願 い し ま す 。
2019-09-17 05:21:37.178911 ... Target  = please sign your name here .
2019-09-17 05:21:37.178921 ... Predict = please help your name . .
2019-09-17 05:21:37.179075 ... Epoch 10/10, Sample 49441
2019-09-17 05:21:37.179088 ... Source  = 子供 たち は 遊び 回 っ て い た 。
2019-09-17 05:21:37.179098 ... Target  = the children were playing about .
2019-09-17 05:21:37.179108 ... Predict = the children were playing after .
2019-09-17 05:21:37.179262 ... Epoch 10/10, Sample 49442
2019-09-17 05:21:37.179275 ... Source  = 私 は その 間 ずっと ここ に い た 。
2019-09-17 05:21:37.179285 ... Target  = i was here all the time .
2019-09-17 05:21:37.179295 ... Predict = i stayed here all the time .
2019-09-17 05:21:37.179450 ... Epoch 10/10, Sample 49443
2019-09-17 05:21:37.179463 ... Source  = 庭 に あなた の ネコ が 見え ま す 。
2019-09-17 05:21:37.179473 ... Target  = i see your cat in the garden .
2019-09-17 05:21:37.179482 ... Predict = we see your cat in the garden .
2019-09-17 05:21:37.179636 ... Epoch 10/10, Sample 49444
2019-09-17 05:21:37.179650 ... Source  = 私 は 行 か ず に 留ま っ た 。
2019-09-17 05:21:37.179660 ... Target  = i didn 't go but stayed .
2019-09-17 05:21:37.179669 ... Predict = i was 't go without in .
2019-09-17 05:21:37.179824 ... Epoch 10/10, Sample 49445
2019-09-17 05:21:37.179837 ... Source  = この 部屋 は 少し も 暖か く な い 。
2019-09-17 05:21:37.179847 ... Target  = this room is anything but warm .
2019-09-17 05:21:37.179856 ... Predict = this room is not but warm .
2019-09-17 05:21:37.180010 ... Epoch 10/10, Sample 49446
2019-09-17 05:21:37.180023 ... Source  = 太郎 は 一生 懸命 勉強 し て い る 。
2019-09-17 05:21:37.180034 ... Target  = taro is studying hard .
2019-09-17 05:21:37.180043 ... Predict = taro is studying hard .
2019-09-17 05:21:37.180199 ... Epoch 10/10, Sample 49447
2019-09-17 05:21:37.180212 ... Source  = 空港 まで は いくら かか り ま す か 。
2019-09-17 05:21:37.180222 ... Target  = how much will it cost to get to the airport ?
2019-09-17 05:21:37.180231 ... Predict = how much will it take to the to the airport ?
2019-09-17 05:21:37.180385 ... Epoch 10/10, Sample 49448
2019-09-17 05:21:37.180398 ... Source  = この 会社 の 社長 は 誰 で す か 。
2019-09-17 05:21:37.180408 ... Target  = who is the boss of this company ?
2019-09-17 05:21:37.180418 ... Predict = who is the president of this company ?
2019-09-17 05:21:37.180574 ... Epoch 10/10, Sample 49449
2019-09-17 05:21:37.180587 ... Source  = 賃上げ を 要求 し たら どう で す か 。
2019-09-17 05:21:37.180597 ... Target  = why don 't you ask for a pay raise ?
2019-09-17 05:21:37.180606 ... Predict = why don 't you make a a message ? ?
2019-09-17 05:21:37.180760 ... Epoch 10/10, Sample 49450
2019-09-17 05:21:37.180773 ... Source  = 彼女 は 僕 の 目 を 引 い た 。
2019-09-17 05:21:37.180783 ... Target  = she caught my eye .
2019-09-17 05:21:37.180793 ... Predict = she took me eyes on
2019-09-17 05:21:37.180947 ... Epoch 10/10, Sample 49451
2019-09-17 05:21:37.180961 ... Source  = 仕事 で 大阪 まで 行 っ て き た 。
2019-09-17 05:21:37.180970 ... Target  = i 've been to osaka on business .
2019-09-17 05:21:37.180980 ... Predict = i came been to osaka by business .
2019-09-17 05:21:37.181134 ... Epoch 10/10, Sample 49452
2019-09-17 05:21:37.181147 ... Source  = 彼女 は 愛 を 切望 し て い た 。
2019-09-17 05:21:37.181157 ... Target  = she was greedy for love .
2019-09-17 05:21:37.181166 ... Predict = she was anxious for her .
2019-09-17 05:21:37.181320 ... Epoch 10/10, Sample 49453
2019-09-17 05:21:37.181333 ... Source  = あなた は 私 より も 背 が 高 い 。
2019-09-17 05:21:37.181343 ... Target  = you are taller than i.
2019-09-17 05:21:37.181353 ... Predict = you are taller than i
2019-09-17 05:21:37.181508 ... Epoch 10/10, Sample 49454
2019-09-17 05:21:37.181521 ... Source  = 東京 は 明日 晴れ る で しょ う か 。
2019-09-17 05:21:37.181531 ... Target  = will it be fair in tokyo tomorrow ?
2019-09-17 05:21:37.181541 ... Predict = will it be fine for tokyo tomorrow ?
2019-09-17 05:21:37.181695 ... Epoch 10/10, Sample 49455
2019-09-17 05:21:37.181708 ... Source  = 十 中八九 彼 は 成功 する だ ろ う 。
2019-09-17 05:21:37.181718 ... Target  = ten to one , he will succeed .
2019-09-17 05:21:37.181728 ... Predict = ten to one he he will succeed .
2019-09-17 05:21:37.181881 ... Epoch 10/10, Sample 49456
2019-09-17 05:21:37.181895 ... Source  = パーティー は 彼 の スピーチ で 始ま っ た 。
2019-09-17 05:21:37.181905 ... Target  = the party opened with his speech .
2019-09-17 05:21:37.181914 ... Predict = the party began his his speech .
2019-09-17 05:21:37.182071 ... Epoch 10/10, Sample 49457
2019-09-17 05:21:37.182084 ... Source  = 君 に は 音楽 の 才能 が あ る 。
2019-09-17 05:21:37.182095 ... Target  = you have a gift for music .
2019-09-17 05:21:37.182104 ... Predict = you have a genius for music .
2019-09-17 05:21:37.182268 ... Epoch 10/10, Sample 49458
2019-09-17 05:21:37.182282 ... Source  = 彼 は 野球 で かなり 見こみ が あ る 。
2019-09-17 05:21:37.182293 ... Target  = he has a big future in baseball .
2019-09-17 05:21:37.182303 ... Predict = he is a big deal in baseball .
2019-09-17 05:21:37.182475 ... Epoch 10/10, Sample 49459
2019-09-17 05:21:37.182489 ... Source  = 彼 は いつ も 勉強 し て い る 。
2019-09-17 05:21:37.182499 ... Target  = he is always studying .
2019-09-17 05:21:37.182508 ... Predict = he is always studying .
2019-09-17 05:21:37.182669 ... Epoch 10/10, Sample 49460
2019-09-17 05:21:37.182683 ... Source  = 私 は この ブレンド の コーヒー が 好き だ 。
2019-09-17 05:21:37.182693 ... Target  = i like this blend of coffee .
2019-09-17 05:21:37.182702 ... Predict = i like this blend of coffee .
2019-09-17 05:21:37.182858 ... Epoch 10/10, Sample 49461
2019-09-17 05:21:37.182871 ... Source  = その 仕事 に 興味 が もて な かっ た 。
2019-09-17 05:21:37.182881 ... Target  = my heart wasn 't in the work .
2019-09-17 05:21:37.182891 ... Predict = the interest was 't in the work .
2019-09-17 05:21:37.183046 ... Epoch 10/10, Sample 49462
2019-09-17 05:21:37.183059 ... Source  = 去年 の 夏 は 雨 が 多 かっ た 。
2019-09-17 05:21:37.183069 ... Target  = we had a good deal of rain last summer .
2019-09-17 05:21:37.183079 ... Predict = we had a lot deal of rain last summer .
2019-09-17 05:21:37.183234 ... Epoch 10/10, Sample 49463
2019-09-17 05:21:37.183247 ... Source  = 私 達 の 計画 は ことごとく 失敗 し た 。
2019-09-17 05:21:37.183258 ... Target  = all our plans went wrong .
2019-09-17 05:21:37.183267 ... Predict = our our plan were out .
2019-09-17 05:21:37.183419 ... Epoch 10/10, Sample 49464
2019-09-17 05:21:37.183433 ... Source  = 彼女 は 昨日 少し 具合 が よ かっ た 。
2019-09-17 05:21:37.183443 ... Target  = she was slightly better yesterday .
2019-09-17 05:21:37.183452 ... Predict = she was better better yesterday .
2019-09-17 05:21:37.183608 ... Epoch 10/10, Sample 49465
2019-09-17 05:21:37.183621 ... Source  = 彼 は その 知らせ に すっかり 失望 し た 。
2019-09-17 05:21:37.183631 ... Target  = he was very much disappointed at the news .
2019-09-17 05:21:37.183640 ... Predict = he was disappointed disappointed disappointed at the news .
2019-09-17 05:21:37.183794 ... Epoch 10/10, Sample 49466
2019-09-17 05:21:37.183807 ... Source  = この 車 は 修理 する 必要 が あ る 。
2019-09-17 05:21:37.183818 ... Target  = this car needs repairing .
2019-09-17 05:21:37.183827 ... Predict = this car needs repairing .
2019-09-17 05:21:37.183982 ... Epoch 10/10, Sample 49467
2019-09-17 05:21:37.183995 ... Source  = ドア を あけ て いただけ ま せ ん か 。
2019-09-17 05:21:37.184005 ... Target  = do you mind opening the door ?
2019-09-17 05:21:37.184014 ... Predict = would you mind opening the door ?
2019-09-17 05:21:37.184167 ... Epoch 10/10, Sample 49468
2019-09-17 05:21:37.184180 ... Source  = お 年寄り に は やさし く し なさ い 。
2019-09-17 05:21:37.184190 ... Target  = be kind to old people .
2019-09-17 05:21:37.184199 ... Predict = do kind to old people .
2019-09-17 05:21:37.184355 ... Epoch 10/10, Sample 49469
2019-09-17 05:21:37.184368 ... Source  = 警察 は 怒 っ た 群衆 を 引き止め た 。
2019-09-17 05:21:37.184378 ... Target  = the police held the angry crowd back .
2019-09-17 05:21:37.184388 ... Predict = the police got the angry crowd to .
2019-09-17 05:21:37.184542 ... Epoch 10/10, Sample 49470
2019-09-17 05:21:37.184555 ... Source  = 東京 駅 を ７ 時 に 出 ま す 。
2019-09-17 05:21:37.184565 ... Target  = the train leaves tokyo station at 7 .
2019-09-17 05:21:37.184575 ... Predict = seven station leaves tokyo at at seven .
2019-09-17 05:21:37.184730 ... Epoch 10/10, Sample 49471
2019-09-17 05:21:37.184743 ... Source  = 遊 ぶ 時間 は 結構 あ り ま す 。
2019-09-17 05:21:37.184754 ... Target  = i still have a lot of time for fun .
2019-09-17 05:21:37.184763 ... Predict = i 'm have time time of time to fun .
2019-09-17 05:21:37.184917 ... Epoch 10/10, Sample 49472
2019-09-17 05:21:37.184930 ... Source  = その 赤ちゃん は 泣 い て ばかり い る 。
2019-09-17 05:21:37.184940 ... Target  = that baby does nothing but cry .
2019-09-17 05:21:37.184949 ... Predict = the baby is nothing but cry .
2019-09-17 05:21:37.185103 ... Epoch 10/10, Sample 49473
2019-09-17 05:21:37.185117 ... Source  = 彼 は 私 の 頭 を 打 っ た 。
2019-09-17 05:21:37.185127 ... Target  = he hit me on the head .
2019-09-17 05:21:37.185136 ... Predict = he hit me on the head .
2019-09-17 05:21:37.185291 ... Epoch 10/10, Sample 49474
2019-09-17 05:21:37.185304 ... Source  = 時間 と 金 を 無駄 に 使 う な 。
2019-09-17 05:21:37.185314 ... Target  = don 't waste your time and money .
2019-09-17 05:21:37.185324 ... Predict = don 't use your time and money .
2019-09-17 05:21:37.185478 ... Epoch 10/10, Sample 49475
2019-09-17 05:21:37.185492 ... Source  = 彼 は 私 たち に 援助 を 求め た 。
2019-09-17 05:21:37.185502 ... Target  = he appealed to us for help .
2019-09-17 05:21:37.185511 ... Predict = he asked to us for help .
2019-09-17 05:21:37.185665 ... Epoch 10/10, Sample 49476
2019-09-17 05:21:37.185678 ... Source  = 警察 が その 殺人 事件 を 調べ 始め た 。
2019-09-17 05:21:37.185688 ... Target  = the police started to look into the murder case .
2019-09-17 05:21:37.185698 ... Predict = the police began to look into the murder case .
2019-09-17 05:21:37.185852 ... Epoch 10/10, Sample 49477
2019-09-17 05:21:37.185865 ... Source  = 彼 ら に 話 す の を やめ た 。
2019-09-17 05:21:37.185875 ... Target  = he stopped talking to them .
2019-09-17 05:21:37.185885 ... Predict = i stopped talking to them .
2019-09-17 05:21:37.186039 ... Epoch 10/10, Sample 49478
2019-09-17 05:21:37.186052 ... Source  = ジェーン は 日本 語 が 大いに 進歩 し た 。
2019-09-17 05:21:37.186062 ... Target  = jane has made great progress in japanese .
2019-09-17 05:21:37.186071 ... Predict = jane has made a progress in japanese .
2019-09-17 05:21:37.186227 ... Epoch 10/10, Sample 49479
2019-09-17 05:21:37.186241 ... Source  = おもしろ い なら どんな 本 で も よ い 。
2019-09-17 05:21:37.186251 ... Target  = any book will do , so long as it is interesting .
2019-09-17 05:21:37.186260 ... Predict = some book is do better interesting well as you is interesting .
2019-09-17 05:21:37.186415 ... Epoch 10/10, Sample 49480
2019-09-17 05:21:37.186429 ... Source  = みんな が 彼 の 案 に 賛成 し た 。
2019-09-17 05:21:37.186439 ... Target  = everybody agreed with his idea .
2019-09-17 05:21:37.186448 ... Predict = everyone agreed to his plan .
2019-09-17 05:21:37.186610 ... Epoch 10/10, Sample 49481
2019-09-17 05:21:37.186624 ... Source  = 一人 で 外出 する の を 許 し た 。
2019-09-17 05:21:37.186634 ... Target  = she allowed him to go alone .
2019-09-17 05:21:37.186643 ... Predict = i allowed to to go out .
2019-09-17 05:21:37.186797 ... Epoch 10/10, Sample 49482
2019-09-17 05:21:37.186810 ... Source  = あなた が 先 に 話 し て 下さ い 。
2019-09-17 05:21:37.186820 ... Target  = you speak first ; i will speak after .
2019-09-17 05:21:37.186829 ... Predict = please may at , i am speak to .
2019-09-17 05:21:37.186984 ... Epoch 10/10, Sample 49483
2019-09-17 05:21:37.186997 ... Source  = どの チーム が 勝 つ だ ろ う か 。
2019-09-17 05:21:37.187007 ... Target  = which team will win ?
2019-09-17 05:21:37.187017 ... Predict = which team will win ?
2019-09-17 05:21:37.187171 ... Epoch 10/10, Sample 49484
2019-09-17 05:21:37.187185 ... Source  = この セーター は 気 に 入 ら な い 。
2019-09-17 05:21:37.187195 ... Target  = i don 't like this sweater .
2019-09-17 05:21:37.187204 ... Predict = i don 't like this sweater .
2019-09-17 05:21:37.187378 ... Epoch 10/10, Sample 49485
2019-09-17 05:21:37.187400 ... Source  = その 小説 の 作者 は 誰 で す か 。
2019-09-17 05:21:37.187410 ... Target  = who is the author of the novel ?
2019-09-17 05:21:37.187420 ... Predict = who is the author of the novel ?
2019-09-17 05:21:37.187574 ... Epoch 10/10, Sample 49486
2019-09-17 05:21:37.187587 ... Source  = 彼 は １ 日 休み を と っ た 。
2019-09-17 05:21:37.187597 ... Target  = he took a day off .
2019-09-17 05:21:37.187607 ... Predict = he took a day off .
2019-09-17 05:21:37.187763 ... Epoch 10/10, Sample 49487
2019-09-17 05:21:37.187777 ... Source  = 彼 の 成功 の 見込み は 十分 あ る 。
2019-09-17 05:21:37.187787 ... Target  = he has a good chance to succeed .
2019-09-17 05:21:37.187796 ... Predict = there is a good hope to succeed .
2019-09-17 05:21:37.187951 ... Epoch 10/10, Sample 49488
2019-09-17 05:21:37.187964 ... Source  = 数 日 は 外出 し な い よう に 。
2019-09-17 05:21:37.187974 ... Target  = you shouldn 't go out for a few days .
2019-09-17 05:21:37.187983 ... Predict = you must 't go out for a few days .
2019-09-17 05:21:40.890331 ... Epoch 10/10, Sample 49489
2019-09-17 05:21:40.890374 ... Source  = 正直 に 言 う と 、 私 は 孤独 だっ た 。
2019-09-17 05:21:40.890388 ... Target  = to tell the truth , i felt lonely .
2019-09-17 05:21:40.890399 ... Predict = to tell the truth , i was lonely .
2019-09-17 05:21:40.890581 ... Epoch 10/10, Sample 49490
2019-09-17 05:21:40.890597 ... Source  = 彼女 の 夫 は ３ 年間 服役 し て い る 。
2019-09-17 05:21:40.890608 ... Target  = her husband has been in prison for three years .
2019-09-17 05:21:40.890618 ... Predict = her husband has been in prison for three years .
2019-09-17 05:21:40.890779 ... Epoch 10/10, Sample 49491
2019-09-17 05:21:40.890792 ... Source  = 我々 は 彼 の 成功 を 期待 し て い る 。
2019-09-17 05:21:40.890802 ... Target  = we expect him to succeed .
2019-09-17 05:21:40.890812 ... Predict = we expect him to succeed .
2019-09-17 05:21:40.890973 ... Epoch 10/10, Sample 49492
2019-09-17 05:21:40.890986 ... Source  = 今夜 は 雪 に な る 可能 性 が あ る 。
2019-09-17 05:21:40.890997 ... Target  = there is a possibility that it will snow tonight .
2019-09-17 05:21:40.891006 ... Predict = we is a possibility of we will snow tonight .
2019-09-17 05:21:40.891166 ... Epoch 10/10, Sample 49493
2019-09-17 05:21:40.891179 ... Source  = 私 は その 質問 は 簡単 だ と 思 っ た 。
2019-09-17 05:21:40.891189 ... Target  = i thought the questions were easy .
2019-09-17 05:21:40.891198 ... Predict = i thought the question was easy .
2019-09-17 05:21:40.891357 ... Epoch 10/10, Sample 49494
2019-09-17 05:21:40.891371 ... Source  = あなた の 本 を 私 に 貸 し て くださ い 。
2019-09-17 05:21:40.891381 ... Target  = please lend me your book .
2019-09-17 05:21:40.891390 ... Predict = please lend me your book .
2019-09-17 05:21:40.891552 ... Epoch 10/10, Sample 49495
2019-09-17 05:21:40.891565 ... Source  = 彼 は その 火事 で 逃げ 遅れ て 死 ん だ 。
2019-09-17 05:21:40.891575 ... Target  = he failed to escape from the fire and burned to death .
2019-09-17 05:21:40.891585 ... Predict = he died to die from the fire away death to death .
2019-09-17 05:21:40.891746 ... Epoch 10/10, Sample 49496
2019-09-17 05:21:40.891759 ... Source  = 両親 は 私 が 留学 する こと に 反対 し た 。
2019-09-17 05:21:40.891769 ... Target  = my parents objected to my studying abroad .
2019-09-17 05:21:40.891779 ... Predict = my parents objected to my abroad abroad .
2019-09-17 05:21:40.891940 ... Epoch 10/10, Sample 49497
2019-09-17 05:21:40.891953 ... Source  = 働き はじめ る と すぐ 雨 が 降 り 始め た 。
2019-09-17 05:21:40.891964 ... Target  = i had hardly started to work when it began to rain .
2019-09-17 05:21:40.891973 ... Predict = i began started started to work without it began to rain .
2019-09-17 05:21:40.892131 ... Epoch 10/10, Sample 49498
2019-09-17 05:21:40.892144 ... Source  = や る から に は 、 ちゃんと や り なさ い 。
2019-09-17 05:21:40.892154 ... Target  = if you do it at all , do it well .
2019-09-17 05:21:40.892163 ... Predict = come you do it , all , go it well .
2019-09-17 05:21:40.892324 ... Epoch 10/10, Sample 49499
2019-09-17 05:21:40.892337 ... Source  = 彼 は 目 を 閉じ て 音楽 を 聞 い た 。
2019-09-17 05:21:40.892347 ... Target  = he listened to the music with his eyes closed .
2019-09-17 05:21:40.892356 ... Predict = he listened to the music with his eyes closed .
2019-09-17 05:21:40.892529 ... Epoch 10/10, Sample 49500
2019-09-17 05:21:40.892543 ... Source  = 彼 の 言 う こと が よく わか ら な い 。
2019-09-17 05:21:40.892563 ... Target  = i don 't quite understand what he says .
2019-09-17 05:21:40.892572 ... Predict = i don 't know understand what he says .
2019-09-17 05:21:40.892742 ... Epoch 10/10, Sample 49501
2019-09-17 05:21:40.892756 ... Source  = 彼 は 銀行 に 多額 の 預金 が あ っ た 。
2019-09-17 05:21:40.892766 ... Target  = he had a lot of money in the bank .
2019-09-17 05:21:40.892775 ... Predict = he had a lot of money in the bank .
2019-09-17 05:21:40.892934 ... Epoch 10/10, Sample 49502
2019-09-17 05:21:40.892947 ... Source  = 彼 は 最後 に 到着 し た 人 で し た 。
2019-09-17 05:21:40.892957 ... Target  = he was the last person to arrive .
2019-09-17 05:21:40.892966 ... Predict = he was the last person to arrive .
2019-09-17 05:21:40.893125 ... Epoch 10/10, Sample 49503
2019-09-17 05:21:40.893138 ... Source  = その 店 は 日曜日 に は しま っ て い る 。
2019-09-17 05:21:40.893148 ... Target  = the shop is closed on sunday .
2019-09-17 05:21:40.893158 ... Predict = the store is closed on sunday .
2019-09-17 05:21:40.893318 ... Epoch 10/10, Sample 49504
2019-09-17 05:21:40.893331 ... Source  = 彼 は 来 れ ば 必ず 私 たち を しか る 。
2019-09-17 05:21:40.893341 ... Target  = whenever he comes , he scolds us .
2019-09-17 05:21:40.893351 ... Predict = without he comes , he will us .
2019-09-17 05:21:40.893510 ... Epoch 10/10, Sample 49505
2019-09-17 05:21:40.893523 ... Source  = 何 時間 も 歩 き 続け て き た の だ 。
2019-09-17 05:21:40.893533 ... Target  = he had been walking for hours .
2019-09-17 05:21:40.893543 ... Predict = i has been walking for hours .
2019-09-17 05:21:40.893702 ... Epoch 10/10, Sample 49506
2019-09-17 05:21:40.893715 ... Source  = コップ に は 水 が 少し 残 っ て い る 。
2019-09-17 05:21:40.893725 ... Target  = there is little water left in the glass .
2019-09-17 05:21:40.893735 ... Predict = there is a water left in the glass .
2019-09-17 05:21:40.893894 ... Epoch 10/10, Sample 49507
2019-09-17 05:21:40.893907 ... Source  = かわり に 君 に そこ へ 行 っ て 欲し い 。
2019-09-17 05:21:40.893917 ... Target  = i wish you to go there instead .
2019-09-17 05:21:40.893927 ... Predict = i want you go go there on .
2019-09-17 05:21:40.894086 ... Epoch 10/10, Sample 49508
2019-09-17 05:21:40.894099 ... Source  = 彼女 は 息子 に カメラ を 買 っ て あげ た 。
2019-09-17 05:21:40.894109 ... Target  = she bought her son a camera .
2019-09-17 05:21:40.894119 ... Predict = she bought her son a camera .
2019-09-17 05:21:40.894278 ... Epoch 10/10, Sample 49509
2019-09-17 05:21:40.894292 ... Source  = 私 達 は この 問題 を 楽 に 解け ま す 。
2019-09-17 05:21:40.894302 ... Target  = we can solve this problem easily .
2019-09-17 05:21:40.894311 ... Predict = we can solve this problem easily .
2019-09-17 05:21:40.894471 ... Epoch 10/10, Sample 49510
2019-09-17 05:21:40.894484 ... Source  = 私 は まだ 夕食 を す ま せ て な い 。
2019-09-17 05:21:40.894494 ... Target  = i have not yet finished my supper .
2019-09-17 05:21:40.894504 ... Predict = i have not to finished dinner dinner .
2019-09-17 05:21:40.894674 ... Epoch 10/10, Sample 49511
2019-09-17 05:21:40.894688 ... Source  = 花 は 切 ら れ る と すぐ に しぼ む 。
2019-09-17 05:21:40.894698 ... Target  = flowers soon fade when they have been cut .
2019-09-17 05:21:40.894708 ... Predict = the soon as as the were been cut .
2019-09-17 05:21:40.894867 ... Epoch 10/10, Sample 49512
2019-09-17 05:21:40.894880 ... Source  = 彼 は いつ 家 に 帰れ る の で す か 。
2019-09-17 05:21:40.894890 ... Target  = when will he go home ?
2019-09-17 05:21:40.894899 ... Predict = when will he go home ?
2019-09-17 05:21:40.895060 ... Epoch 10/10, Sample 49513
2019-09-17 05:21:40.895073 ... Source  = 同時 に 彼 は 走 り 出 し ま し た 。
2019-09-17 05:21:40.895083 ... Target  = at the same time , he began to run .
2019-09-17 05:21:40.895092 ... Predict = at the same time , he began to run .
2019-09-17 05:21:40.895252 ... Epoch 10/10, Sample 49514
2019-09-17 05:21:40.895265 ... Source  = あなた の 事務 所 に 行 き ま しょ う か 。
2019-09-17 05:21:40.895275 ... Target  = shall i come to your office ?
2019-09-17 05:21:40.895284 ... Predict = shall i go to your office ?
2019-09-17 05:21:40.895445 ... Epoch 10/10, Sample 49515
2019-09-17 05:21:40.895458 ... Source  = 彼 の 言 う こと は あ る 意味 正し い 。
2019-09-17 05:21:40.895468 ... Target  = what he says is true in a sense .
2019-09-17 05:21:40.895478 ... Predict = what he says is right in a sense .
2019-09-17 05:21:40.895638 ... Epoch 10/10, Sample 49516
2019-09-17 05:21:40.895651 ... Source  = ラジオ の 音 を 下げ て くれ ま せ ん か 。
2019-09-17 05:21:40.895661 ... Target  = could you turn down the radio ?
2019-09-17 05:21:40.895671 ... Predict = will you show down the radio ?
2019-09-17 05:21:40.895829 ... Epoch 10/10, Sample 49517
2019-09-17 05:21:40.895842 ... Source  = 私 は 帰宅 し て も よ い で す か 。
2019-09-17 05:21:40.895852 ... Target  = may i go home ?
2019-09-17 05:21:40.895862 ... Predict = may i go home ?
2019-09-17 05:21:40.896020 ... Epoch 10/10, Sample 49518
2019-09-17 05:21:40.896034 ... Source  = 私 達 の 出発 時間 が さしせま っ て い る 。
2019-09-17 05:21:40.896044 ... Target  = it 's time to leave .
2019-09-17 05:21:40.896053 ... Predict = it is time to start our
2019-09-17 05:21:40.896214 ... Epoch 10/10, Sample 49519
2019-09-17 05:21:40.896227 ... Source  = その 質問 に 答え られ る の は 君 だけ だ 。
2019-09-17 05:21:40.896237 ... Target  = only you can answer the question .
2019-09-17 05:21:40.896247 ... Predict = it you can answer the question .
2019-09-17 05:21:40.896408 ... Epoch 10/10, Sample 49520
2019-09-17 05:21:40.896421 ... Source  = 飛行 機 の 遅れ る こと は まず 間違い な い 。
2019-09-17 05:21:40.896431 ... Target  = the plane could easily be late .
2019-09-17 05:21:40.896440 ... Predict = it plane is not be late .
2019-09-17 05:21:40.896600 ... Epoch 10/10, Sample 49521
2019-09-17 05:21:40.896613 ... Source  = 彼 は 明日 飛行 機 で 行 っ て しま う 。
2019-09-17 05:21:40.896623 ... Target  = they are going off by plane tomorrow .
2019-09-17 05:21:40.896632 ... Predict = he go going by by plane tomorrow .
2019-09-17 05:21:40.896792 ... Epoch 10/10, Sample 49522
2019-09-17 05:21:40.896805 ... Source  = 彼 の こと は 無視 し て い な さ い 。
2019-09-17 05:21:40.896815 ... Target  = do not take any notice of him .
2019-09-17 05:21:40.896824 ... Predict = you it have sense sense of him .
2019-09-17 05:21:40.896984 ... Epoch 10/10, Sample 49523
2019-09-17 05:21:40.896997 ... Source  = どの 辞書 を あなた は よく 利用 し ま す か 。
2019-09-17 05:21:40.897007 ... Target  = which dictionary do you often use ?
2019-09-17 05:21:40.897017 ... Predict = how dictionary do you take take ?
2019-09-17 05:21:40.897176 ... Epoch 10/10, Sample 49524
2019-09-17 05:21:40.897189 ... Source  = 誰 に も 得意 な 時代 が あ る もの だ 。
2019-09-17 05:21:40.897199 ... Target  = every dog has his day .
2019-09-17 05:21:40.897209 ... Predict = every time has his day .
2019-09-17 05:21:40.897367 ... Epoch 10/10, Sample 49525
2019-09-17 05:21:40.897381 ... Source  = 急 い で 物事 を する と 間違え ま す よ 。
2019-09-17 05:21:40.897391 ... Target  = you make mistakes if you do things in a hurry .
2019-09-17 05:21:40.897400 ... Predict = hurry make a , you have , in a hurry .
2019-09-17 05:21:40.897571 ... Epoch 10/10, Sample 49526
2019-09-17 05:21:40.897594 ... Source  = 以前 は ちょうど ここ に お 寺 が あ っ た 。
2019-09-17 05:21:40.897604 ... Target  = there used to be a temple right here .
2019-09-17 05:21:40.897622 ... Predict = there used to be a temple here here .
2019-09-17 05:21:40.897782 ... Epoch 10/10, Sample 49527
2019-09-17 05:21:40.897795 ... Source  = 目 に は 目 を 、 歯 に は 歯 を 。
2019-09-17 05:21:40.897805 ... Target  = an eye for an eye , a tooth for a tooth .
2019-09-17 05:21:40.897815 ... Predict = my eye and my eye , i tooth . a tooth .
2019-09-17 05:21:40.897975 ... Epoch 10/10, Sample 49528
2019-09-17 05:21:40.897988 ... Source  = 私 は 試験 に 合格 する よう に 努力 し た 。
2019-09-17 05:21:40.897998 ... Target  = i exerted myself to pass the examination .
2019-09-17 05:21:40.898008 ... Predict = i exerted myself to pass the exam .
2019-09-17 05:21:40.898167 ... Epoch 10/10, Sample 49529
2019-09-17 05:21:40.898180 ... Source  = 私 は バス の 前 の 席 に 座 っ た 。
2019-09-17 05:21:40.898190 ... Target  = i sat in the front of the bus .
2019-09-17 05:21:40.898200 ... Predict = i sat on my bus of the bus .
2019-09-17 05:21:40.898359 ... Epoch 10/10, Sample 49530
2019-09-17 05:21:40.898373 ... Source  = 東京 へ 行 く の に ２ 時間 かか っ た 。
2019-09-17 05:21:40.898383 ... Target  = it took us two hours to get to tokyo .
2019-09-17 05:21:40.898392 ... Predict = it took two two hours to go to tokyo .
2019-09-17 05:21:40.898558 ... Epoch 10/10, Sample 49531
2019-09-17 05:21:40.898572 ... Source  = 彼 は よ い 仕事 を さが し て い る 。
2019-09-17 05:21:40.898582 ... Target  = he is seeking a good job .
2019-09-17 05:21:40.898591 ... Predict = he is looking a good job .
2019-09-17 05:21:40.898752 ... Epoch 10/10, Sample 49532
2019-09-17 05:21:40.898765 ... Source  = 彼 等 を からか っ て は いけ ま せ ん 。
2019-09-17 05:21:40.898775 ... Target  = you ought not to make fun of them .
2019-09-17 05:21:40.898784 ... Predict = don must to to make fun of them .
2019-09-17 05:21:40.898944 ... Epoch 10/10, Sample 49533
2019-09-17 05:21:40.898957 ... Source  = 彼 は 日曜日 に は いつ も 家 に い る 。
2019-09-17 05:21:40.898967 ... Target  = he 's always at home on sundays .
2019-09-17 05:21:40.898976 ... Predict = he is always at home on sunday .
2019-09-17 05:21:40.899136 ... Epoch 10/10, Sample 49534
2019-09-17 05:21:40.899149 ... Source  = それ 以後 彼女 に 会 っ て い ま せ ん 。
2019-09-17 05:21:40.899159 ... Target  = i haven 't seen her since then .
2019-09-17 05:21:40.899168 ... Predict = i haven 't seen her since then .
2019-09-17 05:21:40.899328 ... Epoch 10/10, Sample 49535
2019-09-17 05:21:40.899341 ... Source  = 忘れ ず に この 手紙 を 出 し て くださ い 。
2019-09-17 05:21:40.899351 ... Target  = don 't forget to mail this letter .
2019-09-17 05:21:40.899360 ... Predict = don 't forget to mail this letter .
2019-09-17 05:21:40.899520 ... Epoch 10/10, Sample 49536
2019-09-17 05:21:40.899533 ... Source  = 彼女 の 微笑み は 愛 を 語 っ て い た 。
2019-09-17 05:21:40.899543 ... Target  = her smile spoke love .
2019-09-17 05:21:40.899553 ... Predict = her smile spoke love .
2019-09-17 05:21:40.899712 ... Epoch 10/10, Sample 49537
2019-09-17 05:21:40.899725 ... Source  = 彼 が 来る 時 を 知 ら せ て 下さ い 。
2019-09-17 05:21:40.899735 ... Target  = let me know the time when he will come .
2019-09-17 05:21:40.899745 ... Predict = let me know when time when he comes come .
2019-09-17 05:21:40.899905 ... Epoch 10/10, Sample 49538
2019-09-17 05:21:40.899918 ... Source  = 私 は 昨日 、 英語 の 手紙 を 受け取 っ た 。
2019-09-17 05:21:40.899928 ... Target  = i received a letter in english yesterday .
2019-09-17 05:21:40.899937 ... Predict = i received a letter of english yesterday .
2019-09-17 05:21:40.900096 ... Epoch 10/10, Sample 49539
2019-09-17 05:21:40.900109 ... Source  = 俺 は 今 、 都会 に 住 ん で い る 。
2019-09-17 05:21:40.900119 ... Target  = i 'm living in the city .
2019-09-17 05:21:40.900128 ... Predict = i have living in the city now
2019-09-17 05:21:40.900289 ... Epoch 10/10, Sample 49540
2019-09-17 05:21:40.900302 ... Source  = 心 の 傷 を 癒 し て くれ る 愛 を 。
2019-09-17 05:21:40.900312 ... Target  = i 've been waiting for love to come .
2019-09-17 05:21:40.900322 ... Predict = i love been waiting for love of come .
2019-09-17 05:21:40.900481 ... Epoch 10/10, Sample 49541
2019-09-17 05:21:40.900495 ... Source  = 彼 は 私 が 到着 する 前 に 死 ん だ 。
2019-09-17 05:21:40.900505 ... Target  = he died previous to my arrival .
2019-09-17 05:21:40.900514 ... Predict = he died before of me arrival .
2019-09-17 05:21:40.900674 ... Epoch 10/10, Sample 49542
2019-09-17 05:21:40.900687 ... Source  = 彼 は その 車 を ただ で 手 に いれ た 。
2019-09-17 05:21:40.900697 ... Target  = he got the car for nothing .
2019-09-17 05:21:40.900706 ... Predict = he got the car for nothing .
2019-09-17 05:21:40.900867 ... Epoch 10/10, Sample 49543
2019-09-17 05:21:40.900881 ... Source  = 外出 する より 家 に い る ほう が い い 。
2019-09-17 05:21:40.900891 ... Target  = i would rather stay at home than go out .
2019-09-17 05:21:40.900900 ... Predict = i would rather stay at home than go out .
2019-09-17 05:21:40.901058 ... Epoch 10/10, Sample 49544
2019-09-17 05:21:40.901071 ... Source  = 彼 が あなた の 代わり に 行 く だ ろ う 。
2019-09-17 05:21:40.901081 ... Target  = he will go in your place .
2019-09-17 05:21:40.901091 ... Predict = he will go in place place .
2019-09-17 05:21:40.901251 ... Epoch 10/10, Sample 49545
2019-09-17 05:21:40.901265 ... Source  = その 交通 事故 は 、 幹線 道路 で 起こ っ た 。
2019-09-17 05:21:40.901274 ... Target  = the traffic accident took place on the highway .
2019-09-17 05:21:40.901284 ... Predict = the traffic accident happened place on the highway .
2019-09-17 05:21:40.901444 ... Epoch 10/10, Sample 49546
2019-09-17 05:21:40.901457 ... Source  = スミス さん と は いつ から お しりあい で す か 。
2019-09-17 05:21:40.901467 ... Target  = how long have you known miss smith ?
2019-09-17 05:21:40.901476 ... Predict = how long have you been mr smith ?
2019-09-17 05:21:40.901634 ... Epoch 10/10, Sample 49547
2019-09-17 05:21:40.901648 ... Source  = 先生 は 困難 な 問題 を 出 す で しょ う 。
2019-09-17 05:21:40.901658 ... Target  = our teacher will give us difficult problems .
2019-09-17 05:21:40.901667 ... Predict = the teacher will let up a problem .
2019-09-17 05:21:40.901827 ... Epoch 10/10, Sample 49548
2019-09-17 05:21:40.901840 ... Source  = こんな に 迷惑 を かけ て す い ま せ ん 。
2019-09-17 05:21:40.901850 ... Target  = i 'm sorry to cause you all this trouble .
2019-09-17 05:21:40.901859 ... Predict = i 'm sorry to put you this this trouble .
2019-09-17 05:21:40.902019 ... Epoch 10/10, Sample 49549
2019-09-17 05:21:40.902033 ... Source  = あなた は 宿題 を せっせと やり さえ すれ ば よ い 。
2019-09-17 05:21:40.902043 ... Target  = you have only to work away at your homework .
2019-09-17 05:21:40.902052 ... Predict = you have only to do off with your homework .
2019-09-17 05:21:40.902211 ... Epoch 10/10, Sample 49550
2019-09-17 05:21:40.902225 ... Source  = すべて が 旨 く 言 っ て い る よう だ 。
2019-09-17 05:21:40.902235 ... Target  = everything appears to be going well .
2019-09-17 05:21:40.902244 ... Predict = everything seems well be good well .
2019-09-17 05:21:40.902403 ... Epoch 10/10, Sample 49551
2019-09-17 05:21:40.902416 ... Source  = ボブ が いつ 来日 し た か 分か ら な い 。
2019-09-17 05:21:40.902426 ... Target  = i don 't know when bob came to japan .
2019-09-17 05:21:40.902436 ... Predict = i don 't know when bob is to japan .
2019-09-17 05:21:40.902601 ... Epoch 10/10, Sample 49552
2019-09-17 05:21:40.902624 ... Source  = 彼 は 学校 の 近く に 住 ん で い る 。
2019-09-17 05:21:40.902635 ... Target  = he lives in the neighborhood of the school .
2019-09-17 05:21:40.902645 ... Predict = he lives near the neighborhood of the school .
2019-09-17 05:21:40.902823 ... Epoch 10/10, Sample 49553
2019-09-17 05:21:40.902836 ... Source  = 彼 は 手 を 出 す もの すべて に 熱中 する 。
2019-09-17 05:21:40.902846 ... Target  = he really gets into anything he tries .
2019-09-17 05:21:40.902856 ... Predict = he is gets in everything he can .
2019-09-17 05:21:40.903015 ... Epoch 10/10, Sample 49554
2019-09-17 05:21:40.903029 ... Source  = 娘 が ときどき 私 の 見舞い に き て くれ た 。
2019-09-17 05:21:40.903039 ... Target  = my daughter came to see me from time to time .
2019-09-17 05:21:40.903048 ... Predict = my daughter came to see me in time to time .
2019-09-17 05:21:40.903208 ... Epoch 10/10, Sample 49555
2019-09-17 05:21:40.903221 ... Source  = 彼 は もう ここ に は 勤め て い な い 。
2019-09-17 05:21:40.903231 ... Target  = he no longer works here .
2019-09-17 05:21:40.903240 ... Predict = he has longer worked here .
2019-09-17 05:21:40.903400 ... Epoch 10/10, Sample 49556
2019-09-17 05:21:40.903413 ... Source  = 私 の 意見 は 君 の と 似 て い る 。
2019-09-17 05:21:40.903423 ... Target  = my opinion is similar to yours .
2019-09-17 05:21:40.903432 ... Predict = my opinion is similar to yours .
2019-09-17 05:21:40.903593 ... Epoch 10/10, Sample 49557
2019-09-17 05:21:40.903606 ... Source  = 試験 に 受か る か どう か は 君 次第 だ 。
2019-09-17 05:21:40.903616 ... Target  = it is up to you whether you pass the examination or not .
2019-09-17 05:21:40.903625 ... Predict = it is up to you whether you will the exam or not .
2019-09-17 05:21:40.903785 ... Epoch 10/10, Sample 49558
2019-09-17 05:21:40.903798 ... Source  = あなた の おかげ で 旨 く 行 き ま し た 。
2019-09-17 05:21:40.903808 ... Target  = i owe my success to you .
2019-09-17 05:21:40.903818 ... Predict = i am your own . you .
2019-09-17 05:21:40.903976 ... Epoch 10/10, Sample 49559
2019-09-17 05:21:40.903989 ... Source  = 彼女 は 君 を 知 っ て い ま す か 。
2019-09-17 05:21:40.903999 ... Target  = does she know you ?
2019-09-17 05:21:40.904009 ... Predict = do she know you ?
2019-09-17 05:21:40.904169 ... Epoch 10/10, Sample 49560
2019-09-17 05:21:40.904182 ... Source  = 私 は 、 一人 で その 服 を 縫 っ た 。
2019-09-17 05:21:40.904192 ... Target  = i made that dress by myself .
2019-09-17 05:21:40.904202 ... Predict = i made the dress by myself .
2019-09-17 05:21:40.904363 ... Epoch 10/10, Sample 49561
2019-09-17 05:21:40.904376 ... Source  = 休暇 を でき る だけ 上手 く 利用 し なさ い 。
2019-09-17 05:21:40.904387 ... Target  = make the most of your vacation .
2019-09-17 05:21:40.904396 ... Predict = make your best of your vacation .
2019-09-17 05:21:40.904556 ... Epoch 10/10, Sample 49562
2019-09-17 05:21:40.904570 ... Source  = 彼女 は 若 い ころ は 美人 だっ た よう だ 。
2019-09-17 05:21:40.904580 ... Target  = she seems to have been a beauty in her day .
2019-09-17 05:21:40.904589 ... Predict = she seems to have been beautiful beautiful when her day .
2019-09-17 05:21:40.904749 ... Epoch 10/10, Sample 49563
2019-09-17 05:21:40.904762 ... Source  = 両親 に 知れ たら 面倒 な こと に な る よ 。
2019-09-17 05:21:40.904772 ... Target  = you 'll get into trouble if your parents find out .
2019-09-17 05:21:40.904782 ... Predict = you 'll be to your if your parents find out .
2019-09-17 05:21:40.904941 ... Epoch 10/10, Sample 49564
2019-09-17 05:21:40.904954 ... Source  = あなた は そんな 大声 で 話 す 必要 は な い 。
2019-09-17 05:21:40.904964 ... Target  = you don 't have to talk so loud .
2019-09-17 05:21:40.904974 ... Predict = you don 't have such speak so loud .
2019-09-17 05:21:40.905134 ... Epoch 10/10, Sample 49565
2019-09-17 05:21:40.905147 ... Source  = 家 で 休 む くらい なら むしろ 外出 し た い 。
2019-09-17 05:21:40.905157 ... Target  = i would rather go out than stay at home .
2019-09-17 05:21:40.905166 ... Predict = i would rather go out as a at home .
2019-09-17 05:21:40.905326 ... Epoch 10/10, Sample 49566
2019-09-17 05:21:40.905340 ... Source  = 彼 の 兄 さん は 彼 より も 我慢 強 い 。
2019-09-17 05:21:40.905350 ... Target  = his brother is more patient than he is .
2019-09-17 05:21:40.905359 ... Predict = his brother is more patient than he is .
2019-09-17 05:21:40.905519 ... Epoch 10/10, Sample 49567
2019-09-17 05:21:40.905532 ... Source  = その 時 あなた は 学校 に い ま し た か 。
2019-09-17 05:21:40.905542 ... Target  = were you at school at that time ?
2019-09-17 05:21:40.905551 ... Predict = did you school school then that time ?
2019-09-17 05:21:40.905711 ... Epoch 10/10, Sample 49568
2019-09-17 05:21:40.905724 ... Source  = あなた は 私 を 助け て くれ る と 思 う 。
2019-09-17 05:21:40.905734 ... Target  = i hope that you will help me .
2019-09-17 05:21:40.905743 ... Predict = i think you you will help me .
2019-09-17 05:21:40.905902 ... Epoch 10/10, Sample 49569
2019-09-17 05:21:40.905915 ... Source  = 彼 は それ が でき る 唯一 の 人 で す 。
2019-09-17 05:21:40.905925 ... Target  = he is the only person that can do it .
2019-09-17 05:21:40.905935 ... Predict = he is the only man to can do it .
2019-09-17 05:21:40.906096 ... Epoch 10/10, Sample 49570
2019-09-17 05:21:40.906109 ... Source  = 彼 は 楽しみ に ピアノ を 弾 い て い る 。
2019-09-17 05:21:40.906119 ... Target  = he plays the piano for his own amusement .
2019-09-17 05:21:40.906128 ... Predict = he is the piano with his mind amusement .
2019-09-17 05:21:40.906288 ... Epoch 10/10, Sample 49571
2019-09-17 05:21:40.906301 ... Source  = 本題 から それ な い よう に し ま しょ う 。
2019-09-17 05:21:40.906311 ... Target  = let 's not deviate from the subject .
2019-09-17 05:21:40.906321 ... Predict = let 's not say from the menu .
2019-09-17 05:21:40.906481 ... Epoch 10/10, Sample 49572
2019-09-17 05:21:40.906494 ... Source  = 彼 ら は その 場所 に 近づ か な かっ た 。
2019-09-17 05:21:40.906504 ... Target  = they stayed away from the place .
2019-09-17 05:21:40.906513 ... Predict = they didn away from the place .
2019-09-17 05:21:40.906679 ... Epoch 10/10, Sample 49573
2019-09-17 05:21:40.906693 ... Source  = どの くらい 洋行 し て い た の で す か 。
2019-09-17 05:21:40.906703 ... Target  = how long have you been abroad ?
2019-09-17 05:21:40.906712 ... Predict = how long have you been abroad ?
2019-09-17 05:21:40.906872 ... Epoch 10/10, Sample 49574
2019-09-17 05:21:40.906885 ... Source  = 私 は 父 を 誇り に 思 っ て い る 。
2019-09-17 05:21:40.906895 ... Target  = i 'm proud of my father .
2019-09-17 05:21:40.906904 ... Predict = i am proud of my father .
2019-09-17 05:21:40.907083 ... Epoch 10/10, Sample 49575
2019-09-17 05:21:40.907096 ... Source  = うれし い こと に 彼女 は 試験 に パス し た 。
2019-09-17 05:21:40.907106 ... Target  = to her delight , she got through the examination .
2019-09-17 05:21:40.907115 ... Predict = to her happy , she passed the the examination .
2019-09-17 05:21:40.907274 ... Epoch 10/10, Sample 49576
2019-09-17 05:21:40.907287 ... Source  = お 願い を 聞 い て もらえ な い か な 。
2019-09-17 05:21:40.907298 ... Target  = i wonder if you could do me a favor .
2019-09-17 05:21:40.907307 ... Predict = i can if you can help a a favor .
2019-09-17 05:21:40.907466 ... Epoch 10/10, Sample 49577
2019-09-17 05:21:40.907479 ... Source  = 君 は 一生 懸命 勉強 し さえ すれ ば よ い 。
2019-09-17 05:21:40.907489 ... Target  = you have only to study hard .
2019-09-17 05:21:40.907498 ... Predict = you have only to study hard .
2019-09-17 05:21:40.907657 ... Epoch 10/10, Sample 49578
2019-09-17 05:21:40.907671 ... Source  = いつ か 君 の 夢 も かな う だ ろ う 。
2019-09-17 05:21:40.907689 ... Target  = someday your dream will come true .
2019-09-17 05:21:40.907699 ... Predict = when is dream will come true .
2019-09-17 05:21:40.907878 ... Epoch 10/10, Sample 49579
2019-09-17 05:21:40.907891 ... Source  = 青空 以外 に は 何 も 見 られ な かっ た 。
2019-09-17 05:21:40.907901 ... Target  = nothing was to be seen but the blue sky .
2019-09-17 05:21:40.907910 ... Predict = i was to see seen but a blue sky .
2019-09-17 05:21:40.908070 ... Epoch 10/10, Sample 49580
2019-09-17 05:21:40.908083 ... Source  = この 古 い 家 は 木材 で でき て い る 。
2019-09-17 05:21:40.908093 ... Target  = this old house is made of wood .
2019-09-17 05:21:40.908102 ... Predict = this old house is made of wood .
2019-09-17 05:21:40.908262 ... Epoch 10/10, Sample 49581
2019-09-17 05:21:40.908275 ... Source  = 彼 の 小説 の 中 で これ が 一番 好き だ 。
2019-09-17 05:21:40.908285 ... Target  = among his novels , i like this best .
2019-09-17 05:21:40.908294 ... Predict = this his novels like i like this best .
2019-09-17 05:21:40.908454 ... Epoch 10/10, Sample 49582
2019-09-17 05:21:40.908467 ... Source  = ニューヨーク は 行 っ て み る 価値 が あ る 。
2019-09-17 05:21:40.908477 ... Target  = new york is worth visiting .
2019-09-17 05:21:40.908486 ... Predict = new york is worth visiting .
2019-09-17 05:21:40.908645 ... Epoch 10/10, Sample 49583
2019-09-17 05:21:40.908658 ... Source  = 私 達 と 一緒 に いらっしゃ い ま せ ん か 。
2019-09-17 05:21:40.908669 ... Target  = wouldn 't you like to come with us , too ?
2019-09-17 05:21:40.908678 ... Predict = would 't you come to come with us ? aren ?
2019-09-17 05:21:40.908838 ... Epoch 10/10, Sample 49584
2019-09-17 05:21:40.908852 ... Source  = これ ら の 絵葉書 は 全部 私 の もの で す 。
2019-09-17 05:21:40.908862 ... Target  = all of these picture postcards are mine .
2019-09-17 05:21:40.908871 ... Predict = these these these troubles postcards are mine .
2019-09-17 05:21:40.909030 ... Epoch 10/10, Sample 49585
2019-09-17 05:21:40.909043 ... Source  = 彼 は ゆっくり と 通り を 歩 い て い た 。
2019-09-17 05:21:40.909053 ... Target  = he was walking slowly down the street .
2019-09-17 05:21:40.909063 ... Predict = he was walking slowly walking the street .
2019-09-17 05:21:40.909221 ... Epoch 10/10, Sample 49586
2019-09-17 05:21:40.909234 ... Source  = 私 は この 機械 を よく 知 っ て い る 。
2019-09-17 05:21:40.909244 ... Target  = this machine is familiar to me .
2019-09-17 05:21:40.909254 ... Predict = i machine is familiar to me .
2019-09-17 05:21:40.909412 ... Epoch 10/10, Sample 49587
2019-09-17 05:21:40.909426 ... Source  = 私 は 息子 の した こと に 責任 が あ る 。
2019-09-17 05:21:40.909435 ... Target  = i 'm responsible for what my son has done .
2019-09-17 05:21:40.909445 ... Predict = i am responsible for my my son is done .
2019-09-17 05:21:40.909604 ... Epoch 10/10, Sample 49588
2019-09-17 05:21:40.909617 ... Source  = 約束 し て から 彼 を 訪問 す べ き だ 。
2019-09-17 05:21:40.909627 ... Target  = you should visit him by appointment .
2019-09-17 05:21:40.909637 ... Predict = you should call him after your .
2019-09-17 05:21:40.909795 ... Epoch 10/10, Sample 49589
2019-09-17 05:21:40.909808 ... Source  = 手紙 を 書 く の を 忘れ な い で ね 。
2019-09-17 05:21:40.909818 ... Target  = don 't forget to write to me .
2019-09-17 05:21:40.909827 ... Predict = don 't forget to write to me .
2019-09-17 05:21:40.909986 ... Epoch 10/10, Sample 49590
2019-09-17 05:21:40.909999 ... Source  = 彼女 は すばらし い 健康 な 赤ちゃん を 生 ん だ 。
2019-09-17 05:21:40.910009 ... Target  = she gave birth to a fine healthy baby .
2019-09-17 05:21:40.910019 ... Predict = she was a a a wonderful health baby .
2019-09-17 05:21:40.910178 ... Epoch 10/10, Sample 49591
2019-09-17 05:21:40.910191 ... Source  = 我々 は 我々 の 計画 に 固執 する べ き だ 。
2019-09-17 05:21:40.910201 ... Target  = we should stick to our plan .
2019-09-17 05:21:40.910211 ... Predict = we should make to our plan .
2019-09-17 05:21:40.910370 ... Epoch 10/10, Sample 49592
2019-09-17 05:21:40.910384 ... Source  = 彼 は １ 週間 前 に ニューヨーク へ 発 っ た 。
2019-09-17 05:21:40.910394 ... Target  = he left for new york a week ago .
2019-09-17 05:21:40.910403 ... Predict = he left new new york a week ago .
2019-09-17 05:21:40.910568 ... Epoch 10/10, Sample 49593
2019-09-17 05:21:40.910582 ... Source  = 昨日 ここ に 来 た と 彼 は 言 っ た 。
2019-09-17 05:21:40.910592 ... Target  = he told me that he had gone there .
2019-09-17 05:21:40.910601 ... Predict = he said me that he was been to yesterday
2019-09-17 05:21:40.910760 ... Epoch 10/10, Sample 49594
2019-09-17 05:21:40.910773 ... Source  = 君 は なぜ 昨日 授業 を さぼ っ た の か 。
2019-09-17 05:21:40.910783 ... Target  = why did you absent yourself from class yesterday ?
2019-09-17 05:21:40.910793 ... Predict = why did you play played yesterday class yesterday ?
2019-09-17 05:21:40.910951 ... Epoch 10/10, Sample 49595
2019-09-17 05:21:40.910964 ... Source  = 私 に は 書 く ため の 鉛筆 が な い 。
2019-09-17 05:21:40.910974 ... Target  = i have no pencil to write with .
2019-09-17 05:21:40.910984 ... Predict = i have no pencil to write with .
2019-09-17 05:21:40.911144 ... Epoch 10/10, Sample 49596
2019-09-17 05:21:40.911158 ... Source  = 日本 で は コンピューター 用 品 が 非常 に 高 い 。
2019-09-17 05:21:40.911168 ... Target  = computer supplies are very expensive in japan .
2019-09-17 05:21:40.911177 ... Predict = some supplies are very high in japan .
2019-09-17 05:21:40.911337 ... Epoch 10/10, Sample 49597
2019-09-17 05:21:40.911350 ... Source  = ネズミ が テーブル の 上 を 散歩 に 行 っ た 。
2019-09-17 05:21:40.911360 ... Target  = a mouse went for a walk on the table .
2019-09-17 05:21:40.911369 ... Predict = a mouse went for a walk on the table .
2019-09-17 05:21:40.911527 ... Epoch 10/10, Sample 49598
2019-09-17 05:21:40.911540 ... Source  = 彼 は 彼女 の 冗談 が 分か ら な かっ た 。
2019-09-17 05:21:40.911550 ... Target  = he didn 't get her joke .
2019-09-17 05:21:40.911560 ... Predict = he didn 't like her joke .
2019-09-17 05:21:40.911718 ... Epoch 10/10, Sample 49599
2019-09-17 05:21:40.911731 ... Source  = あなた は 、 計画 を 実行 し ま し た か 。
2019-09-17 05:21:40.911741 ... Target  = did you carry out your plan ?
2019-09-17 05:21:40.911751 ... Predict = did you carry out your plan ?
2019-09-17 05:21:40.911909 ... Epoch 10/10, Sample 49600
2019-09-17 05:21:40.911922 ... Source  = あの 映画 を 見 る べ き だっ た の に 。
2019-09-17 05:21:40.911932 ... Target  = you should have seen the movie .
2019-09-17 05:21:40.911942 ... Predict = you should have seen that film .
2019-09-17 05:21:40.912099 ... Epoch 10/10, Sample 49601
2019-09-17 05:21:40.912113 ... Source  = その 会合 は 要 する に 時間 の 無駄 だっ た 。
2019-09-17 05:21:40.912123 ... Target  = the meeting , in short , was a waste of time .
2019-09-17 05:21:40.912132 ... Predict = the meeting was the the , lost a waste of time .
2019-09-17 05:21:40.912290 ... Epoch 10/10, Sample 49602
2019-09-17 05:21:40.912303 ... Source  = 彼女 は 、 その 本 を ロンドン に 注文 し た 。
2019-09-17 05:21:40.912313 ... Target  = she ordered the book from london .
2019-09-17 05:21:40.912322 ... Predict = she ordered the book to london .
2019-09-17 05:21:40.912480 ... Epoch 10/10, Sample 49603
2019-09-17 05:21:40.912493 ... Source  = たくさん の 友人 が 見送り に や っ て き た 。
2019-09-17 05:21:40.912503 ... Target  = a host of friends came to see me off .
2019-09-17 05:21:40.912513 ... Predict = a lot of friends came off see me off .
2019-09-17 05:21:40.912670 ... Epoch 10/10, Sample 49604
2019-09-17 05:21:40.912683 ... Source  = そこ の は 何 時 も だれ か が い る 。
2019-09-17 05:21:40.912693 ... Target  = there 's always somebody there .
2019-09-17 05:21:40.912703 ... Predict = there is no somebody there .
2019-09-17 05:21:40.912890 ... Epoch 10/10, Sample 49605
2019-09-17 05:21:40.912903 ... Source  = その 少年 は 手 に 負え な く な っ た 。
2019-09-17 05:21:40.912913 ... Target  = the boy has got out of hand .
2019-09-17 05:21:40.912922 ... Predict = the boy became become out of hand .
2019-09-17 05:21:40.913080 ... Epoch 10/10, Sample 49606
2019-09-17 05:21:40.913093 ... Source  = 痛み が ひど く な っ て き ま し た 。
2019-09-17 05:21:40.913103 ... Target  = the pain is getting worse .
2019-09-17 05:21:40.913113 ... Predict = the pain has running running .
2019-09-17 05:21:40.913270 ... Epoch 10/10, Sample 49607
2019-09-17 05:21:40.913283 ... Source  = 彼女 は 少し お 金 を 持 っ て い た 。
2019-09-17 05:21:40.913293 ... Target  = she had a little money .
2019-09-17 05:21:40.913302 ... Predict = she had a little money .
2019-09-17 05:21:40.913462 ... Epoch 10/10, Sample 49608
2019-09-17 05:21:40.913476 ... Source  = 彼 の 父 は 弁護 士 で あ る よう だ 。
2019-09-17 05:21:40.913485 ... Target  = it seems that his father is a lawyer .
2019-09-17 05:21:40.913495 ... Predict = his seems that his father is a lawyer .
2019-09-17 05:21:40.913654 ... Epoch 10/10, Sample 49609
2019-09-17 05:21:40.913668 ... Source  = もう 少し 、 大きな 声 で 話 し て くださ い 。
2019-09-17 05:21:40.913678 ... Target  = could you speak a little louder please ?
2019-09-17 05:21:40.913687 ... Predict = please you speak a little louder ? ?
2019-09-17 05:21:40.913847 ... Epoch 10/10, Sample 49610
2019-09-17 05:21:40.913860 ... Source  = 彼 は 誰 より も 良 く 出き る 生徒 だ 。
2019-09-17 05:21:40.913870 ... Target  = he is by far the best student .
2019-09-17 05:21:40.913879 ... Predict = he is as far the best student .
2019-09-17 05:21:40.914037 ... Epoch 10/10, Sample 49611
2019-09-17 05:21:40.914050 ... Source  = あなた は 車 を 持 っ て い ま す か 。
2019-09-17 05:21:40.914060 ... Target  = do you have a car ?
2019-09-17 05:21:40.914069 ... Predict = do you have a car ?
2019-09-17 05:21:40.914228 ... Epoch 10/10, Sample 49612
2019-09-17 05:21:40.914241 ... Source  = 彼 は 偶然 に なく し た カメラ を 見つけ た 。
2019-09-17 05:21:40.914251 ... Target  = he found his lost camera by chance .
2019-09-17 05:21:40.914260 ... Predict = he found his lost camera by chance .
2019-09-17 05:21:40.914420 ... Epoch 10/10, Sample 49613
2019-09-17 05:21:40.914434 ... Source  = 鳥 が 空 を 高 く 飛 ん で い る 。
2019-09-17 05:21:40.914444 ... Target  = some birds are flying high in the sky .
2019-09-17 05:21:40.914453 ... Predict = the birds are flying high in the sky .
2019-09-17 05:21:40.914618 ... Epoch 10/10, Sample 49614
2019-09-17 05:21:40.914632 ... Source  = 彼 は すぐ に トム に 追いつ く で しょ う 。
2019-09-17 05:21:40.914642 ... Target  = he 'll soon catch up with tom .
2019-09-17 05:21:40.914651 ... Predict = he will soon catch up with tom .
2019-09-17 05:21:40.914810 ... Epoch 10/10, Sample 49615
2019-09-17 05:21:40.914823 ... Source  = 彼 は 空腹 そう な 顔 を し て い る 。
2019-09-17 05:21:40.914833 ... Target  = he has a hungry look .
2019-09-17 05:21:40.914842 ... Predict = he has a hungry look .
2019-09-17 05:21:40.915002 ... Epoch 10/10, Sample 49616
2019-09-17 05:21:40.915015 ... Source  = 前 、 偶然 彼 に 会 い ま し た ね ？
2019-09-17 05:21:40.915025 ... Target  = you ran into him earlier , didn 't you ?
2019-09-17 05:21:40.915034 ... Predict = you met on him , . didn 't you ?
2019-09-17 05:21:43.970551 ... Epoch 10/10, Sample 49617
2019-09-17 05:21:43.970591 ... Source  = 雪 に な る で しょ う 。
2019-09-17 05:21:43.970605 ... Target  = it is going to snow .
2019-09-17 05:21:43.970616 ... Predict = it 's going to snow .
2019-09-17 05:21:43.970770 ... Epoch 10/10, Sample 49618
2019-09-17 05:21:43.970785 ... Source  = 寒 さ が 身 に しみ る 。
2019-09-17 05:21:43.970796 ... Target  = i do feel the cold .
2019-09-17 05:21:43.970806 ... Predict = i feel love cold cold .
2019-09-17 05:21:43.970953 ... Epoch 10/10, Sample 49619
2019-09-17 05:21:43.970967 ... Source  = 私 は 何 か 食べ た い 。
2019-09-17 05:21:43.970978 ... Target  = i want to eat something .
2019-09-17 05:21:43.970988 ... Predict = i want to eat something .
2019-09-17 05:21:43.971135 ... Epoch 10/10, Sample 49620
2019-09-17 05:21:43.971149 ... Source  = この 場合 は 君 が 悪 い 。
2019-09-17 05:21:43.971160 ... Target  = you 're wrong in this case .
2019-09-17 05:21:43.971170 ... Predict = you are in in this case .
2019-09-17 05:21:43.971317 ... Epoch 10/10, Sample 49621
2019-09-17 05:21:43.971331 ... Source  = それ は どの 劇場 で す か 。
2019-09-17 05:21:43.971342 ... Target  = which theater is that ?
2019-09-17 05:21:43.971351 ... Predict = what road is that ?
2019-09-17 05:21:43.971498 ... Epoch 10/10, Sample 49622
2019-09-17 05:21:43.971512 ... Source  = すぐ に ここ へ 来 なさ い 。
2019-09-17 05:21:43.971522 ... Target  = come here at once .
2019-09-17 05:21:43.971532 ... Predict = come here at once .
2019-09-17 05:21:43.971678 ... Epoch 10/10, Sample 49623
2019-09-17 05:21:43.971692 ... Source  = スティーブ は 帰宅 し な かっ た 。
2019-09-17 05:21:43.971702 ... Target  = steve did not come home .
2019-09-17 05:21:43.971712 ... Predict = steve didn not come home .
2019-09-17 05:21:43.971858 ... Epoch 10/10, Sample 49624
2019-09-17 05:21:43.971871 ... Source  = 彼 は 仰向け に な っ た 。
2019-09-17 05:21:43.971882 ... Target  = he laid on his back .
2019-09-17 05:21:43.971891 ... Predict = he became on his back .
2019-09-17 05:21:43.972039 ... Epoch 10/10, Sample 49625
2019-09-17 05:21:43.972053 ... Source  = あの ハンサム な 男の子 を 見 て 。
2019-09-17 05:21:43.972063 ... Target  = look at that good-looking boy .
2019-09-17 05:21:43.972073 ... Predict = look at that good-looking boy .
2019-09-17 05:21:43.972218 ... Epoch 10/10, Sample 49626
2019-09-17 05:21:43.972232 ... Source  = 周り を 見 て ごらん なさ い 。
2019-09-17 05:21:43.972242 ... Target  = look about you .
2019-09-17 05:21:43.972252 ... Predict = look up you .
2019-09-17 05:21:43.972428 ... Epoch 10/10, Sample 49627
2019-09-17 05:21:43.972441 ... Source  = もう 雨 に は あきあき し た 。
2019-09-17 05:21:43.972451 ... Target  = we have had enough of rain .
2019-09-17 05:21:43.972460 ... Predict = i had had enough of rain .
2019-09-17 05:21:43.972621 ... Epoch 10/10, Sample 49628
2019-09-17 05:21:43.972634 ... Source  = この 実 は たべ られ ま す 。
2019-09-17 05:21:43.972644 ... Target  = this berry is good to eat .
2019-09-17 05:21:43.972653 ... Predict = this fact is good to eat .
2019-09-17 05:21:43.972794 ... Epoch 10/10, Sample 49629
2019-09-17 05:21:43.972807 ... Source  = 彼 は お 人好 し で す 。
2019-09-17 05:21:43.972817 ... Target  = he 's very soft-hearted .
2019-09-17 05:21:43.972827 ... Predict = he is very soft-hearted .
2019-09-17 05:21:43.972966 ... Epoch 10/10, Sample 49630
2019-09-17 05:21:43.972979 ... Source  = 彼 は その ホテル に い た 。
2019-09-17 05:21:43.972989 ... Target  = he stayed in the hotel .
2019-09-17 05:21:43.972999 ... Predict = he was at the hotel .
2019-09-17 05:21:43.973140 ... Epoch 10/10, Sample 49631
2019-09-17 05:21:43.973153 ... Source  = もう 宿題 は 終わ っ た の ？
2019-09-17 05:21:43.973163 ... Target  = have you finished your homework ?
2019-09-17 05:21:43.973173 ... Predict = have you finished your homework ?
2019-09-17 05:21:43.973313 ... Epoch 10/10, Sample 49632
2019-09-17 05:21:43.973327 ... Source  = 彼女 は 老人 に 親切 で す 。
2019-09-17 05:21:43.973337 ... Target  = she is kind to old people .
2019-09-17 05:21:43.973346 ... Predict = she is kind to the people .
2019-09-17 05:21:43.973486 ... Epoch 10/10, Sample 49633
2019-09-17 05:21:43.973500 ... Source  = 彼女 は 今 外出 中 で す 。
2019-09-17 05:21:43.973510 ... Target  = she is out now .
2019-09-17 05:21:43.973519 ... Predict = she is out now .
2019-09-17 05:21:43.973661 ... Epoch 10/10, Sample 49634
2019-09-17 05:21:43.973674 ... Source  = この 問題 は 、 難し そう だ 。
2019-09-17 05:21:43.973684 ... Target  = this problem seems difficult .
2019-09-17 05:21:43.973693 ... Predict = this problem seems to to
2019-09-17 05:21:43.973835 ... Epoch 10/10, Sample 49635
2019-09-17 05:21:43.973848 ... Source  = 電話 が かか っ て る よ 。
2019-09-17 05:21:43.973858 ... Target  = you are wanted on the phone .
2019-09-17 05:21:43.973867 ... Predict = the are on on the phone .
2019-09-17 05:21:43.974007 ... Epoch 10/10, Sample 49636
2019-09-17 05:21:43.974020 ... Source  = 私 は マイク と 友達 で す 。
2019-09-17 05:21:43.974030 ... Target  = i am friends with mike .
2019-09-17 05:21:43.974040 ... Predict = i am friends with mike .
2019-09-17 05:21:43.974180 ... Epoch 10/10, Sample 49637
2019-09-17 05:21:43.974194 ... Source  = 鼻 が で て い る よ 。
2019-09-17 05:21:43.974204 ... Target  = your nose is running .
2019-09-17 05:21:43.974213 ... Predict = the nose is right .
2019-09-17 05:21:43.974353 ... Epoch 10/10, Sample 49638
2019-09-17 05:21:43.974367 ... Source  = 少年 たち は みな 見下ろ し た 。
2019-09-17 05:21:43.974377 ... Target  = all the boys looked down .
2019-09-17 05:21:43.974386 ... Predict = all the boys looked down .
2019-09-17 05:21:43.974528 ... Epoch 10/10, Sample 49639
2019-09-17 05:21:43.974547 ... Source  = 彼 は 英語 を すらすら 話 す 。
2019-09-17 05:21:43.974558 ... Target  = he speaks english fluently .
2019-09-17 05:21:43.974567 ... Predict = he speaks english fluently .
2019-09-17 05:21:43.974709 ... Epoch 10/10, Sample 49640
2019-09-17 05:21:43.974723 ... Source  = 我々 は 途中 で 会 っ た 。
2019-09-17 05:21:43.974733 ... Target  = we met along the way .
2019-09-17 05:21:43.974742 ... Predict = we met on the way .
2019-09-17 05:21:43.974885 ... Epoch 10/10, Sample 49641
2019-09-17 05:21:43.974898 ... Source  = 自転 車 を 片付け な さ い 。
2019-09-17 05:21:43.974908 ... Target  = put away your bicycle .
2019-09-17 05:21:43.974917 ... Predict = put your your bicycle .
2019-09-17 05:21:43.975059 ... Epoch 10/10, Sample 49642
2019-09-17 05:21:43.975072 ... Source  = この 靴 は 同じ 大き さ だ 。
2019-09-17 05:21:43.975082 ... Target  = the shoes are the same size .
2019-09-17 05:21:43.975091 ... Predict = this shoes are the same size .
2019-09-17 05:21:43.975232 ... Epoch 10/10, Sample 49643
2019-09-17 05:21:43.975245 ... Source  = 朝食 は お すみ で す か 。
2019-09-17 05:21:43.975255 ... Target  = are you through with your breakfast ?
2019-09-17 05:21:43.975265 ... Predict = are you through with the breakfast ?
2019-09-17 05:21:43.975407 ... Epoch 10/10, Sample 49644
2019-09-17 05:21:43.975420 ... Source  = 学校 は 駅 より も 遠 い 。
2019-09-17 05:21:43.975430 ... Target  = the school is farther than the station .
2019-09-17 05:21:43.975440 ... Predict = the school is far than the station .
2019-09-17 05:21:43.975582 ... Epoch 10/10, Sample 49645
2019-09-17 05:21:43.975595 ... Source  = 突然 計画 が 変更 さ れ た 。
2019-09-17 05:21:43.975605 ... Target  = there was a sudden change of plan .
2019-09-17 05:21:43.975614 ... Predict = suddenly was a sudden change the plan .
2019-09-17 05:21:43.975755 ... Epoch 10/10, Sample 49646
2019-09-17 05:21:43.975768 ... Source  = 彼 は 叫び声 を 聞 い た 。
2019-09-17 05:21:43.975779 ... Target  = he heard a shout .
2019-09-17 05:21:43.975788 ... Predict = he heard a cry .
2019-09-17 05:21:43.975930 ... Epoch 10/10, Sample 49647
2019-09-17 05:21:43.975943 ... Source  = 人 ごと だ と 思 っ て 。
2019-09-17 05:21:43.975953 ... Target  = that 's only because you don 't have to do it .
2019-09-17 05:21:43.975962 ... Predict = i 's up for you are 't have to do it .
2019-09-17 05:21:43.976103 ... Epoch 10/10, Sample 49648
2019-09-17 05:21:43.976116 ... Source  = きみ は 僕 を 幸せ に する 。
2019-09-17 05:21:43.976126 ... Target  = you make me happy .
2019-09-17 05:21:43.976136 ... Predict = you do me happy .
2019-09-17 05:21:43.976278 ... Epoch 10/10, Sample 49649
2019-09-17 05:21:43.976291 ... Source  = 私 は 彼 に 失望 し た 。
2019-09-17 05:21:43.976301 ... Target  = i was disappointed in him .
2019-09-17 05:21:43.976310 ... Predict = i was disappointed by him .
2019-09-17 05:21:43.976451 ... Epoch 10/10, Sample 49650
2019-09-17 05:21:43.976465 ... Source  = きみ ！ こっち に 来 なさ い 。
2019-09-17 05:21:43.976475 ... Target  = you , come here .
2019-09-17 05:21:43.976484 ... Predict = come may come here !
2019-09-17 05:21:43.976626 ... Epoch 10/10, Sample 49651
2019-09-17 05:21:43.976639 ... Source  = 彼 は 会合 を 欠席 し た 。
2019-09-17 05:21:43.976650 ... Target  = he absented himself from the meeting .
2019-09-17 05:21:43.976659 ... Predict = he absented himself from the meeting .
2019-09-17 05:21:43.976799 ... Epoch 10/10, Sample 49652
2019-09-17 05:21:43.976813 ... Source  = 彼女 は ピクニック に 行 っ た 。
2019-09-17 05:21:43.976823 ... Target  = she went on a picnic .
2019-09-17 05:21:43.976832 ... Predict = she went on a picnic .
2019-09-17 05:21:43.976974 ... Epoch 10/10, Sample 49653
2019-09-17 05:21:43.976987 ... Source  = ７ 時 に 電話 し ま す 。
2019-09-17 05:21:43.976997 ... Target  = i 'll call you at seven .
2019-09-17 05:21:43.977007 ... Predict = i 'll call you at seven .
2019-09-17 05:21:43.977148 ... Epoch 10/10, Sample 49654
2019-09-17 05:21:43.977161 ... Source  = 時々 私 は 空 を 見上げ た 。
2019-09-17 05:21:43.977172 ... Target  = now and then , i looked up at the sky .
2019-09-17 05:21:43.977181 ... Predict = at at i i i looked up at the sky .
2019-09-17 05:21:43.977322 ... Epoch 10/10, Sample 49655
2019-09-17 05:21:43.977335 ... Source  = この コーヒー は にが すぎ ま す 。
2019-09-17 05:21:43.977345 ... Target  = this coffee is too bitter .
2019-09-17 05:21:43.977354 ... Predict = this coffee is too bitter .
2019-09-17 05:21:43.977523 ... Epoch 10/10, Sample 49656
2019-09-17 05:21:43.977536 ... Source  = それ は たいへん な 話 だ 。
2019-09-17 05:21:43.977546 ... Target  = that 's quite a story .
2019-09-17 05:21:43.977556 ... Predict = that 's a a story .
2019-09-17 05:21:43.977696 ... Epoch 10/10, Sample 49657
2019-09-17 05:21:43.977710 ... Source  = 健康 が 彼 の 資本 だ 。
2019-09-17 05:21:43.977720 ... Target  = good health is his capital .
2019-09-17 05:21:43.977729 ... Predict = health health is his capital .
2019-09-17 05:21:43.977870 ... Epoch 10/10, Sample 49658
2019-09-17 05:21:43.977882 ... Source  = 空 は すっかり 曇 っ た 。
2019-09-17 05:21:43.977892 ... Target  = the sky clouded over .
2019-09-17 05:21:43.977902 ... Predict = the sky clouded all .
2019-09-17 05:21:43.978043 ... Epoch 10/10, Sample 49659
2019-09-17 05:21:43.978056 ... Source  = この 本 が 一番 面白 い 。
2019-09-17 05:21:43.978066 ... Target  = this is the most interesting .
2019-09-17 05:21:43.978075 ... Predict = this book the most interesting .
2019-09-17 05:21:43.978216 ... Epoch 10/10, Sample 49660
2019-09-17 05:21:43.978229 ... Source  = 心 より 感謝 し ま す 。
2019-09-17 05:21:43.978239 ... Target  = i thank you from the bottom of my heart .
2019-09-17 05:21:43.978249 ... Predict = i thank you more my heart of my heart .
2019-09-17 05:21:43.978389 ... Epoch 10/10, Sample 49661
2019-09-17 05:21:43.978403 ... Source  = どう する つもり な の か 。
2019-09-17 05:21:43.978413 ... Target  = what are you driving at ?
2019-09-17 05:21:43.978422 ... Predict = what do you going ? ?
2019-09-17 05:21:43.978572 ... Epoch 10/10, Sample 49662
2019-09-17 05:21:43.978586 ... Source  = 概し て 彼女 は 早起き だ 。
2019-09-17 05:21:43.978596 ... Target  = as a rule , she is an early riser .
2019-09-17 05:21:43.978606 ... Predict = in a rule , she is early early riser .
2019-09-17 05:21:43.978747 ... Epoch 10/10, Sample 49663
2019-09-17 05:21:43.978760 ... Source  = その 犬 は 病気 ら しい 。
2019-09-17 05:21:43.978770 ... Target  = the dog seems to be sick .
2019-09-17 05:21:43.978779 ... Predict = the dog seems to be sick .
2019-09-17 05:21:43.978920 ... Epoch 10/10, Sample 49664
2019-09-17 05:21:43.978933 ... Source  = 今 、 有給 休暇 中 だ 。
2019-09-17 05:21:43.978943 ... Target  = i 'm on a paid vacation .
2019-09-17 05:21:43.978952 ... Predict = now 'm on a vacation vacation .
2019-09-17 05:21:43.979094 ... Epoch 10/10, Sample 49665
2019-09-17 05:21:43.979107 ... Source  = それ は 素晴らし い 考え だ 。
2019-09-17 05:21:43.979117 ... Target  = that 's a splendid idea .
2019-09-17 05:21:43.979127 ... Predict = that 's a great idea .
2019-09-17 05:21:43.979266 ... Epoch 10/10, Sample 49666
2019-09-17 05:21:43.979279 ... Source  = 私 は 、 生徒 で す 。
2019-09-17 05:21:43.979289 ... Target  = i am a student .
2019-09-17 05:21:43.979299 ... Predict = i am a student .
2019-09-17 05:21:43.979439 ... Epoch 10/10, Sample 49667
2019-09-17 05:21:43.979452 ... Source  = お 手伝い し ま す よ 。
2019-09-17 05:21:43.979462 ... Target  = i am willing to help you .
2019-09-17 05:21:43.979472 ... Predict = i 'll willing to help you .
2019-09-17 05:21:43.979612 ... Epoch 10/10, Sample 49668
2019-09-17 05:21:43.979625 ... Source  = 足 が だる い で す 。
2019-09-17 05:21:43.979635 ... Target  = my legs feel heavy .
2019-09-17 05:21:43.979644 ... Predict = my feet feel heavy .
2019-09-17 05:21:43.979784 ... Epoch 10/10, Sample 49669
2019-09-17 05:21:43.979797 ... Source  = お 手伝い し ま しょ う 。
2019-09-17 05:21:43.979807 ... Target  = let me help you .
2019-09-17 05:21:43.979817 ... Predict = let me help you .
2019-09-17 05:21:43.979957 ... Epoch 10/10, Sample 49670
2019-09-17 05:21:43.979970 ... Source  = 私 は ほとんど 泳げ な い 。
2019-09-17 05:21:43.979981 ... Target  = i can hardly swim .
2019-09-17 05:21:43.979990 ... Predict = i can hardly swim .
2019-09-17 05:21:43.980129 ... Epoch 10/10, Sample 49671
2019-09-17 05:21:43.980143 ... Source  = それ は 新し く な い 。
2019-09-17 05:21:43.980153 ... Target  = it isn 't new .
2019-09-17 05:21:43.980162 ... Predict = that is 't new .
2019-09-17 05:21:43.980304 ... Epoch 10/10, Sample 49672
2019-09-17 05:21:43.980317 ... Source  = その 秘密 は ついに ばれ た 。
2019-09-17 05:21:43.980327 ... Target  = the secret came to light at last .
2019-09-17 05:21:43.980336 ... Predict = the secret came to light at last .
2019-09-17 05:21:43.980477 ... Epoch 10/10, Sample 49673
2019-09-17 05:21:43.980490 ... Source  = テレックス で ご 返事 くださ い 。
2019-09-17 05:21:43.980500 ... Target  = please answer by telex .
2019-09-17 05:21:43.980510 ... Predict = please answer by telex .
2019-09-17 05:21:43.980650 ... Epoch 10/10, Sample 49674
2019-09-17 05:21:43.980663 ... Source  = 覚え る 価値 さえ な い 。
2019-09-17 05:21:43.980673 ... Target  = not worth the remembered value .
2019-09-17 05:21:43.980683 ... Predict = it worth your value value .
2019-09-17 05:21:43.980823 ... Epoch 10/10, Sample 49675
2019-09-17 05:21:43.980837 ... Source  = 彼 は いつ も 明る い 。
2019-09-17 05:21:43.980847 ... Target  = he is always cheerful .
2019-09-17 05:21:43.980856 ... Predict = he always always cheerful .
2019-09-17 05:21:43.980995 ... Epoch 10/10, Sample 49676
2019-09-17 05:21:43.981008 ... Source  = 彼女 は 昨日 自殺 し た 。
2019-09-17 05:21:43.981018 ... Target  = she killed herself yesterday .
2019-09-17 05:21:43.981028 ... Predict = she killed herself yesterday .
2019-09-17 05:21:43.981168 ... Epoch 10/10, Sample 49677
2019-09-17 05:21:43.981181 ... Source  = 彼 は 夜 の 帝王 だ 。
2019-09-17 05:21:43.981191 ... Target  = he is the king of the night .
2019-09-17 05:21:43.981200 ... Predict = he is the king of the night .
2019-09-17 05:21:43.981341 ... Epoch 10/10, Sample 49678
2019-09-17 05:21:43.981354 ... Source  = 昨夜 大火 が あ っ た 。
2019-09-17 05:21:43.981364 ... Target  = there was a big fire last night .
2019-09-17 05:21:43.981373 ... Predict = there was a big fire last night .
2019-09-17 05:21:43.981513 ... Epoch 10/10, Sample 49679
2019-09-17 05:21:43.981526 ... Source  = 睡眠 は 健康 に 必要 だ 。
2019-09-17 05:21:43.981536 ... Target  = sleep is necessary to good health .
2019-09-17 05:21:43.981546 ... Predict = sleep is necessary for be health .
2019-09-17 05:21:43.981686 ... Epoch 10/10, Sample 49680
2019-09-17 05:21:43.981699 ... Source  = 賞 は 彼 の 手 に 。
2019-09-17 05:21:43.981709 ... Target  = the prize went to him .
2019-09-17 05:21:43.981719 ... Predict = the prize leaves to him .
2019-09-17 05:21:43.981859 ... Epoch 10/10, Sample 49681
2019-09-17 05:21:43.981873 ... Source  = 我々 は 投票 で 決め た 。
2019-09-17 05:21:43.981883 ... Target  = we decided by vote .
2019-09-17 05:21:43.981892 ... Predict = we made on vote .
2019-09-17 05:21:43.982031 ... Epoch 10/10, Sample 49682
2019-09-17 05:21:43.982044 ... Source  = もう 行 か な い と 。
2019-09-17 05:21:43.982054 ... Target  = got to go now .
2019-09-17 05:21:43.982064 ... Predict = i it going now .
2019-09-17 05:21:43.982204 ... Epoch 10/10, Sample 49683
2019-09-17 05:21:43.982217 ... Source  = とても おもしろ い 試合 だっ た 。
2019-09-17 05:21:43.982227 ... Target  = it was an exciting game .
2019-09-17 05:21:43.982237 ... Predict = it was a interesting game .
2019-09-17 05:21:43.982377 ... Epoch 10/10, Sample 49684
2019-09-17 05:21:43.982390 ... Source  = 彼女 は スポーツ が 苦手 だ 。
2019-09-17 05:21:43.982400 ... Target  = she is bad at sports .
2019-09-17 05:21:43.982410 ... Predict = she is poor at sports .
2019-09-17 05:21:43.982585 ... Epoch 10/10, Sample 49685
2019-09-17 05:21:43.982598 ... Source  = これ を もら い ま す 。
2019-09-17 05:21:43.982608 ... Target  = i 'll take this one .
2019-09-17 05:21:43.982618 ... Predict = i 'll get this . .
2019-09-17 05:21:43.982759 ... Epoch 10/10, Sample 49686
2019-09-17 05:21:43.982772 ... Source  = この 本 は 笑え た よ 。
2019-09-17 05:21:43.982782 ... Target  = this book gave me a good laugh .
2019-09-17 05:21:43.982792 ... Predict = this book gave me a good time .
2019-09-17 05:21:43.982931 ... Epoch 10/10, Sample 49687
2019-09-17 05:21:43.982944 ... Source  = この 教室 は とても 大き い 。
2019-09-17 05:21:43.982954 ... Target  = this classroom is very large .
2019-09-17 05:21:43.982964 ... Predict = this classroom is very big .
2019-09-17 05:21:43.983103 ... Epoch 10/10, Sample 49688
2019-09-17 05:21:43.983116 ... Source  = ここ が 痛 い で す 。
2019-09-17 05:21:43.983126 ... Target  = i have a pain here .
2019-09-17 05:21:43.983136 ... Predict = i have a pain here .
2019-09-17 05:21:43.983275 ... Epoch 10/10, Sample 49689
2019-09-17 05:21:43.983288 ... Source  = 私 は 植木 が 好き だ 。
2019-09-17 05:21:43.983298 ... Target  = i like a garden tree .
2019-09-17 05:21:43.983308 ... Predict = i like a tree . .
2019-09-17 05:21:43.983448 ... Epoch 10/10, Sample 49690
2019-09-17 05:21:43.983461 ... Source  = ぐっすり 眠れ ま し た か 。
2019-09-17 05:21:43.983471 ... Target  = did you have a good sleep ?
2019-09-17 05:21:43.983481 ... Predict = did you sleep a great sleep ?
2019-09-17 05:21:43.983620 ... Epoch 10/10, Sample 49691
2019-09-17 05:21:43.983633 ... Source  = 納得 い か な い なあ 。
2019-09-17 05:21:43.983643 ... Target  = what a pain .
2019-09-17 05:21:43.983653 ... Predict = i a pity ,
2019-09-17 05:21:43.983838 ... Epoch 10/10, Sample 49692
2019-09-17 05:21:43.983860 ... Source  = 外気 は 少し 冷た かっ た 。
2019-09-17 05:21:43.983870 ... Target  = the air felt a little cold .
2019-09-17 05:21:43.983880 ... Predict = the air went a little cold .
2019-09-17 05:21:43.984020 ... Epoch 10/10, Sample 49693
2019-09-17 05:21:43.984033 ... Source  = 夜 の 八 時 で す 。
2019-09-17 05:21:43.984043 ... Target  = it 's eight o 'clock at night .
2019-09-17 05:21:43.984053 ... Predict = it 's eight o 'clock at night .
2019-09-17 05:21:43.984212 ... Epoch 10/10, Sample 49694
2019-09-17 05:21:43.984225 ... Source  = 彼 は 射撃 が うま い 。
2019-09-17 05:21:43.984235 ... Target  = he is a good shot .
2019-09-17 05:21:43.984244 ... Predict = he is a good shot .
2019-09-17 05:21:43.984385 ... Epoch 10/10, Sample 49695
2019-09-17 05:21:43.984398 ... Source  = １ 万 円 で たり る ？
2019-09-17 05:21:43.984408 ... Target  = will ten thousand yen do ?
2019-09-17 05:21:43.984418 ... Predict = can you yen yen ? ?
2019-09-17 05:21:43.984557 ... Epoch 10/10, Sample 49696
2019-09-17 05:21:43.984570 ... Source  = 偶然 その レストラン を 見つけ た 。
2019-09-17 05:21:43.984581 ... Target  = i found that restaurant by accident .
2019-09-17 05:21:43.984590 ... Predict = i found the restaurant by accident .
2019-09-17 05:21:43.984739 ... Epoch 10/10, Sample 49697
2019-09-17 05:21:43.984753 ... Source  = その 鍵 を いじ る な ！
2019-09-17 05:21:43.984764 ... Target  = don 't play with that key !
2019-09-17 05:21:43.984796 ... Predict = don 't play with the key !
2019-09-17 05:21:43.984977 ... Epoch 10/10, Sample 49698
2019-09-17 05:21:43.984991 ... Source  = 掘り出 し もの で す よ 。
2019-09-17 05:21:43.985001 ... Target  = it 's a catch .
2019-09-17 05:21:43.985011 ... Predict = i 's a break .
2019-09-17 05:21:43.985156 ... Epoch 10/10, Sample 49699
2019-09-17 05:21:43.985170 ... Source  = 音 を 大き く し て 。
2019-09-17 05:21:43.985181 ... Target  = turn the volume up .
2019-09-17 05:21:43.985191 ... Predict = turn the noise . .
2019-09-17 05:21:43.985334 ... Epoch 10/10, Sample 49700
2019-09-17 05:21:43.985348 ... Source  = わたし は 疲れ て い る 。
2019-09-17 05:21:43.985358 ... Target  = i 'm tired now .
2019-09-17 05:21:43.985368 ... Predict = i 'm tired . .
2019-09-17 05:21:43.985514 ... Epoch 10/10, Sample 49701
2019-09-17 05:21:43.985527 ... Source  = 天気 は 刻々 と 変わ る 。
2019-09-17 05:21:43.985537 ... Target  = the weather varies from hour to hour .
2019-09-17 05:21:43.985547 ... Predict = the weather varies from quickly to time .
2019-09-17 05:21:43.985692 ... Epoch 10/10, Sample 49702
2019-09-17 05:21:43.985706 ... Source  = すぐ に はじめ ま しょ う 。
2019-09-17 05:21:43.985716 ... Target  = let 's start right away .
2019-09-17 05:21:43.985726 ... Predict = let 's begin at away .
2019-09-17 05:21:43.985871 ... Epoch 10/10, Sample 49703
2019-09-17 05:21:43.985885 ... Source  = その 少女 は 金髪 で す 。
2019-09-17 05:21:43.985895 ... Target  = the girl has golden hair .
2019-09-17 05:21:43.985905 ... Predict = the girl is golden hair .
2019-09-17 05:21:43.986049 ... Epoch 10/10, Sample 49704
2019-09-17 05:21:43.986063 ... Source  = 息 を 止め て くださ い 。
2019-09-17 05:21:43.986073 ... Target  = hold your breath , please .
2019-09-17 05:21:43.986083 ... Predict = please it breath , please .
2019-09-17 05:21:43.986227 ... Epoch 10/10, Sample 49705
2019-09-17 05:21:43.986241 ... Source  = この 部屋 は 禁煙 で す 。
2019-09-17 05:21:43.986251 ... Target  = smoking is not allowed in this room .
2019-09-17 05:21:43.986261 ... Predict = smoking is ready ready in this room .
2019-09-17 05:21:43.986406 ... Epoch 10/10, Sample 49706
2019-09-17 05:21:43.986420 ... Source  = 肉 は 欲し く な い 。
2019-09-17 05:21:43.986430 ... Target  = i don 't want meat .
2019-09-17 05:21:43.986440 ... Predict = i don 't want meat .
2019-09-17 05:21:43.986591 ... Epoch 10/10, Sample 49707
2019-09-17 05:21:43.986606 ... Source  = いつ で も 合図 し て 。
2019-09-17 05:21:43.986616 ... Target  = say the word any time .
2019-09-17 05:21:43.986626 ... Predict = always on word at time .
2019-09-17 05:21:43.986771 ... Epoch 10/10, Sample 49708
2019-09-17 05:21:43.986784 ... Source  = それ は すばらし い 考え だ 。
2019-09-17 05:21:43.986794 ... Target  = that 's a great idea .
2019-09-17 05:21:43.986804 ... Predict = that 's a wonderful idea .
2019-09-17 05:21:43.986949 ... Epoch 10/10, Sample 49709
2019-09-17 05:21:43.986962 ... Source  = 私 は 歌 い た い 。
2019-09-17 05:21:43.986972 ... Target  = i like to sing .
2019-09-17 05:21:43.986982 ... Predict = i want to sing .
2019-09-17 05:21:43.987127 ... Epoch 10/10, Sample 49710
2019-09-17 05:21:43.987141 ... Source  = 彼女 の 鼻 は 高 い 。
2019-09-17 05:21:43.987151 ... Target  = she has a long nose .
2019-09-17 05:21:43.987161 ... Predict = she has a high nose .
2019-09-17 05:21:43.987305 ... Epoch 10/10, Sample 49711
2019-09-17 05:21:43.987319 ... Source  = 騒がし く て いけ な い 。
2019-09-17 05:21:43.987329 ... Target  = you must not be noisy .
2019-09-17 05:21:43.987339 ... Predict = you must not be noisy .
2019-09-17 05:21:43.987483 ... Epoch 10/10, Sample 49712
2019-09-17 05:21:43.987497 ... Source  = ちょっと 待 っ て 下さ い 。
2019-09-17 05:21:43.987507 ... Target  = just a moment , please .
2019-09-17 05:21:43.987517 ... Predict = please a little , please .
2019-09-17 05:21:43.987681 ... Epoch 10/10, Sample 49713
2019-09-17 05:21:43.987694 ... Source  = ブラウン 氏 は 医者 で す 。
2019-09-17 05:21:43.987705 ... Target  = mr brown is a doctor .
2019-09-17 05:21:43.987715 ... Predict = mr brown is a doctor .
2019-09-17 05:21:43.987858 ... Epoch 10/10, Sample 49714
2019-09-17 05:21:43.987872 ... Source  = あまり に 忙し かっ た から 。
2019-09-17 05:21:43.987882 ... Target  = because i was too busy .
2019-09-17 05:21:43.987892 ... Predict = i i was too busy .
2019-09-17 05:21:43.988036 ... Epoch 10/10, Sample 49715
2019-09-17 05:21:43.988050 ... Source  = その グラス は から だっ た 。
2019-09-17 05:21:43.988060 ... Target  = i found the glass empty .
2019-09-17 05:21:43.988070 ... Predict = the was the glass pain .
2019-09-17 05:21:43.988214 ... Epoch 10/10, Sample 49716
2019-09-17 05:21:43.988228 ... Source  = 私 たち は 耳 で ささやく 。
2019-09-17 05:21:43.988238 ... Target  = we hear with our ears .
2019-09-17 05:21:43.988248 ... Predict = we hear with our ears .
2019-09-17 05:21:43.988401 ... Epoch 10/10, Sample 49717
2019-09-17 05:21:43.988415 ... Source  = 秋 は 月 が 美し い 。
2019-09-17 05:21:43.988425 ... Target  = the moon is beautiful in fall .
2019-09-17 05:21:43.988434 ... Predict = the moon is beautiful in fall .
2019-09-17 05:21:43.988574 ... Epoch 10/10, Sample 49718
2019-09-17 05:21:43.988587 ... Source  = 父 は 海軍 に い た 。
2019-09-17 05:21:43.988597 ... Target  = my father was in the navy .
2019-09-17 05:21:43.988607 ... Predict = my father was in the navy .
2019-09-17 05:21:43.988747 ... Epoch 10/10, Sample 49719
2019-09-17 05:21:43.988761 ... Source  = 海 は 荒れ て い た 。
2019-09-17 05:21:43.988771 ... Target  = the sea was running high .
2019-09-17 05:21:43.988780 ... Predict = the sea was running . .
2019-09-17 05:21:43.988920 ... Epoch 10/10, Sample 49720
2019-09-17 05:21:43.988933 ... Source  = 彼 は 口 が 重 い 。
2019-09-17 05:21:43.988943 ... Target  = he is a man of few words .
2019-09-17 05:21:43.988953 ... Predict = he is heavy heavy of ability words .
2019-09-17 05:21:43.989092 ... Epoch 10/10, Sample 49721
2019-09-17 05:21:43.989105 ... Source  = 何 人 か 遅刻 し た 。
2019-09-17 05:21:43.989116 ... Target  = some were late .
2019-09-17 05:21:43.989125 ... Predict = a being late .
2019-09-17 05:21:43.989264 ... Epoch 10/10, Sample 49722
2019-09-17 05:21:43.989277 ... Source  = 彼女 と 結婚 し た い 。
2019-09-17 05:21:43.989287 ... Target  = i hope to marry her .
2019-09-17 05:21:43.989296 ... Predict = i want to marry her .
2019-09-17 05:21:43.989437 ... Epoch 10/10, Sample 49723
2019-09-17 05:21:43.989450 ... Source  = すり に 注意 し なさ い 。
2019-09-17 05:21:43.989460 ... Target  = look out for pickpockets .
2019-09-17 05:21:43.989470 ... Predict = be on of your .
2019-09-17 05:21:43.989609 ... Epoch 10/10, Sample 49724
2019-09-17 05:21:43.989622 ... Source  = 時 に は 歩 き 疲れ 。
2019-09-17 05:21:43.989632 ... Target  = sometimes i feel tired of walking .
2019-09-17 05:21:43.989642 ... Predict = walk time 'm tired to walking .
2019-09-17 05:21:43.989783 ... Epoch 10/10, Sample 49725
2019-09-17 05:21:43.989796 ... Source  = もう 仕事 を 済ま せ た 。
2019-09-17 05:21:43.989806 ... Target  = i 've already finished my work .
2019-09-17 05:21:43.989816 ... Predict = i have finished finished my work .
2019-09-17 05:21:43.989973 ... Epoch 10/10, Sample 49726
2019-09-17 05:21:43.989986 ... Source  = 彼 は 意志 が 弱 い 。
2019-09-17 05:21:43.989996 ... Target  = he doesn 't have a mind of his own .
2019-09-17 05:21:43.990005 ... Predict = he is 't have a good for his own .
2019-09-17 05:21:43.990145 ... Epoch 10/10, Sample 49727
2019-09-17 05:21:43.990158 ... Source  = ご 苦労 様 で し た 。
2019-09-17 05:21:43.990168 ... Target  = thank you for your trouble .
2019-09-17 05:21:43.990178 ... Predict = it you for your help .
2019-09-17 05:21:43.990317 ... Epoch 10/10, Sample 49728
2019-09-17 05:21:43.990330 ... Source  = 次 は 君 の 番 だ 。
2019-09-17 05:21:43.990340 ... Target  = it 's your turn next .
2019-09-17 05:21:43.990350 ... Predict = next 's your turn . .
2019-09-17 05:21:43.990490 ... Epoch 10/10, Sample 49729
2019-09-17 05:21:43.990503 ... Source  = 突然 、 明かり が 消え た 。
2019-09-17 05:21:43.990513 ... Target  = suddenly the light went out .
2019-09-17 05:21:43.990522 ... Predict = suddenly , light went out .
2019-09-17 05:21:43.990667 ... Epoch 10/10, Sample 49730
2019-09-17 05:21:43.990681 ... Source  = 彼 は テニス を し た 。
2019-09-17 05:21:43.990691 ... Target  = he played tennis .
2019-09-17 05:21:43.990701 ... Predict = he played tennis .
2019-09-17 05:21:43.990841 ... Epoch 10/10, Sample 49731
2019-09-17 05:21:43.990854 ... Source  = 私 たち は 兄弟 で す 。
2019-09-17 05:21:43.990864 ... Target  = we are brother and sister .
2019-09-17 05:21:43.990873 ... Predict = we are brothers . brothers .
2019-09-17 05:21:43.991014 ... Epoch 10/10, Sample 49732
2019-09-17 05:21:43.991027 ... Source  = 口 を 大き く 開け て 。
2019-09-17 05:21:43.991037 ... Target  = open your mouth wide .
2019-09-17 05:21:43.991047 ... Predict = open your mouth . .
2019-09-17 05:21:43.991189 ... Epoch 10/10, Sample 49733
2019-09-17 05:21:43.991202 ... Source  = この ドレス は 高 そう だ 。
2019-09-17 05:21:43.991213 ... Target  = that dress seems to be very expensive .
2019-09-17 05:21:43.991222 ... Predict = this dress looks to be a high .
2019-09-17 05:21:43.991361 ... Epoch 10/10, Sample 49734
2019-09-17 05:21:43.991374 ... Source  = 気軽 に し て 下さ い 。
2019-09-17 05:21:43.991384 ... Target  = please make yourself at home .
2019-09-17 05:21:43.991394 ... Predict = please make yourself at home .
2019-09-17 05:21:43.991534 ... Epoch 10/10, Sample 49735
2019-09-17 05:21:43.991546 ... Source  = 彼女 は 服 を 片付け た 。
2019-09-17 05:21:43.991557 ... Target  = she put away her clothes .
2019-09-17 05:21:43.991566 ... Predict = she put up the clothes .
2019-09-17 05:21:43.991706 ... Epoch 10/10, Sample 49736
2019-09-17 05:21:43.991719 ... Source  = 冬 は 火事 が 多 い 。
2019-09-17 05:21:43.991729 ... Target  = we often have fires in winter .
2019-09-17 05:21:43.991739 ... Predict = there have have a in winter .
2019-09-17 05:21:43.991878 ... Epoch 10/10, Sample 49737
2019-09-17 05:21:43.991891 ... Source  = 家 に は い な い 。
2019-09-17 05:21:43.991901 ... Target  = he is away from home .
2019-09-17 05:21:43.991911 ... Predict = i is not from home .
2019-09-17 05:21:43.992052 ... Epoch 10/10, Sample 49738
2019-09-17 05:21:43.992066 ... Source  = 私 は 今 風邪 で す 。
2019-09-17 05:21:43.992075 ... Target  = i have caught a cold .
2019-09-17 05:21:43.992085 ... Predict = i 'm a a cold .
2019-09-17 05:21:43.992226 ... Epoch 10/10, Sample 49739
2019-09-17 05:21:43.992239 ... Source  = 泳 ぐ こと は やさし い 。
2019-09-17 05:21:43.992249 ... Target  = it is easy to swim .
2019-09-17 05:21:43.992259 ... Predict = it is easy to swim .
2019-09-17 05:21:43.992398 ... Epoch 10/10, Sample 49740
2019-09-17 05:21:43.992411 ... Source  = その 法案 に 賛成 で す 。
2019-09-17 05:21:43.992421 ... Target  = i 'm for the bill .
2019-09-17 05:21:43.992431 ... Predict = i agree in the bill .
2019-09-17 05:21:43.992571 ... Epoch 10/10, Sample 49741
2019-09-17 05:21:43.992585 ... Source  = 私 は アメリカ で 生まれ た 。
2019-09-17 05:21:43.992595 ... Target  = i was born in america .
2019-09-17 05:21:43.992613 ... Predict = i was born in america .
2019-09-17 05:21:43.992776 ... Epoch 10/10, Sample 49742
2019-09-17 05:21:43.992789 ... Source  = ケイコ は 親切 で す ね 。
2019-09-17 05:21:43.992799 ... Target  = keiko is kind , isn 't she ?
2019-09-17 05:21:43.992809 ... Predict = keiko is kind , isn 't she ?
2019-09-17 05:21:43.992949 ... Epoch 10/10, Sample 49743
2019-09-17 05:21:43.992962 ... Source  = 山田 さん お 電話 で す 。
2019-09-17 05:21:43.992973 ... Target  = mr yamada , you are wanted on the phone .
2019-09-17 05:21:43.992982 ... Predict = mr yamada , you are on on the phone .
2019-09-17 05:21:43.993122 ... Epoch 10/10, Sample 49744
2019-09-17 05:21:43.993135 ... Source  = また 会 い ま しょ う 。
2019-09-17 05:21:43.993145 ... Target  = see you again .
2019-09-17 05:21:43.993155 ... Predict = i you again .
2019-09-17 05:21:48.194101 ... Epoch 10/10, Sample 49745
2019-09-17 05:21:48.194144 ... Source  = その 家族 の 者 は お 互い に 愛し 合 っ て い る 。
2019-09-17 05:21:48.194157 ... Target  = the family love one another .
2019-09-17 05:21:48.194168 ... Predict = the family love each one .
2019-09-17 05:21:48.194351 ... Epoch 10/10, Sample 49746
2019-09-17 05:21:48.194366 ... Source  = 父 は 昼食 を 食べ る ため に 読書 を やめ ま し た 。
2019-09-17 05:21:48.194377 ... Target  = my father stopped reading to have lunch .
2019-09-17 05:21:48.194387 ... Predict = my father stopped reading for eat lunch .
2019-09-17 05:21:48.194580 ... Epoch 10/10, Sample 49747
2019-09-17 05:21:48.194595 ... Source  = ペン を と る 前 に 考え を まとめ る 必要 が あ る 。
2019-09-17 05:21:48.194605 ... Target  = i must put my ideas together before i take up a pen .
2019-09-17 05:21:48.194615 ... Predict = i need bring my ideas before before i get back a pen .
2019-09-17 05:21:48.194792 ... Epoch 10/10, Sample 49748
2019-09-17 05:21:48.194806 ... Source  = す み ま せ ん 、 少し もど っ て もらえ ま す か 。
2019-09-17 05:21:48.194816 ... Target  = i 'm sorry . would you mind turning around ?
2019-09-17 05:21:48.194825 ... Predict = excuse 'm sorry , i you get something ? ?
2019-09-17 05:21:48.195002 ... Epoch 10/10, Sample 49749
2019-09-17 05:21:48.195015 ... Source  = ジェーン が 私 たち の 町 に 来 て ３ 年 に な る 。
2019-09-17 05:21:48.195025 ... Target  = jane came to our town three years ago .
2019-09-17 05:21:48.195035 ... Predict = jane has to our town for years . .
2019-09-17 05:21:48.195211 ... Epoch 10/10, Sample 49750
2019-09-17 05:21:48.195224 ... Source  = タバコ を 吸 わ な けれ ば 、 長生き でき る だ ろ う 。
2019-09-17 05:21:48.195234 ... Target  = you will live longer if you don 't smoke .
2019-09-17 05:21:48.195244 ... Predict = i will be long if you don 't smoke .
2019-09-17 05:21:48.195432 ... Epoch 10/10, Sample 49751
2019-09-17 05:21:48.195446 ... Source  = ここ に ５ 個 と もう ５ 個 持 っ て い ま す 。
2019-09-17 05:21:48.195465 ... Target  = i have five here and as many again .
2019-09-17 05:21:48.195475 ... Predict = i have five five and have many as .
2019-09-17 05:21:48.195661 ... Epoch 10/10, Sample 49752
2019-09-17 05:21:48.195674 ... Source  = 私 は 、 そう い っ た 音 は すべて 我慢 でき な い 。
2019-09-17 05:21:48.195684 ... Target  = i cannot put up with all that noise .
2019-09-17 05:21:48.195693 ... Predict = i cannot put up with a the noise .
2019-09-17 05:21:48.195868 ... Epoch 10/10, Sample 49753
2019-09-17 05:21:48.195881 ... Source  = 彼女 は 周り を 見回 し た が 何 も 見え な かっ た 。
2019-09-17 05:21:48.195900 ... Target  = she looked around , but she couldn 't see anything .
2019-09-17 05:21:48.195909 ... Predict = she looked around , but she could 't see anything .
2019-09-17 05:21:48.196094 ... Epoch 10/10, Sample 49754
2019-09-17 05:21:48.196107 ... Source  = でき る だけ 多く の 本 を 読 む べ き で あ る 。
2019-09-17 05:21:48.196117 ... Target  = you should read as many books as you can .
2019-09-17 05:21:48.196126 ... Predict = you should read as many books as possible can .
2019-09-17 05:21:48.196301 ... Epoch 10/10, Sample 49755
2019-09-17 05:21:48.196315 ... Source  = わたし は 、 その 前日 に 買 っ た カメラ を なく し た 。
2019-09-17 05:21:48.196325 ... Target  = i lost the camera i had bought the day before .
2019-09-17 05:21:48.196334 ... Predict = i lost the camera i lost bought the day before .
2019-09-17 05:21:48.196509 ... Epoch 10/10, Sample 49756
2019-09-17 05:21:48.196522 ... Source  = なぜ 彼 は 不 機嫌 な 顔 を し て る ん だ い 。
2019-09-17 05:21:48.196532 ... Target  = why does he look grumpy ?
2019-09-17 05:21:48.196542 ... Predict = why he he face in ?
2019-09-17 05:21:48.196716 ... Epoch 10/10, Sample 49757
2019-09-17 05:21:48.196729 ... Source  = 彼 なら 何 を し て も 誉め られ る で あ ろ う 。
2019-09-17 05:21:48.196739 ... Target  = no matter what he may do , he will be praised .
2019-09-17 05:21:48.196748 ... Predict = everything matter what he did do , he will be praised .
2019-09-17 05:21:48.196923 ... Epoch 10/10, Sample 49758
2019-09-17 05:21:48.196936 ... Source  = スタッフ の 人 たち と も うま く い っ て い ま す 。
2019-09-17 05:21:48.196946 ... Target  = i get along well with all the staff .
2019-09-17 05:21:48.196955 ... Predict = i am along with with the the staff .
2019-09-17 05:21:48.197130 ... Epoch 10/10, Sample 49759
2019-09-17 05:21:48.197143 ... Source  = あなた が 手伝 っ て くれ たら すぐ に 終わ る で しょ う 。
2019-09-17 05:21:48.197153 ... Target  = if you will help me , we will soon finish .
2019-09-17 05:21:48.197163 ... Predict = i you will help me , i will finish finish .
2019-09-17 05:21:48.197337 ... Epoch 10/10, Sample 49760
2019-09-17 05:21:48.197350 ... Source  = 私 は ウサギ の 世話 を し な けれ ば な ら な い 。
2019-09-17 05:21:48.197360 ... Target  = i must look after the rabbits .
2019-09-17 05:21:48.197369 ... Predict = i must take after the rabbits .
2019-09-17 05:21:48.197547 ... Epoch 10/10, Sample 49761
2019-09-17 05:21:48.197560 ... Source  = こんな すばらし い 映画 を 見 た こと が あ り ま す か 。
2019-09-17 05:21:48.197571 ... Target  = did you ever see such a film ?
2019-09-17 05:21:48.197580 ... Predict = have you ever watch such a wonderful movie
2019-09-17 05:21:48.197755 ... Epoch 10/10, Sample 49762
2019-09-17 05:21:48.197769 ... Source  = そんな わけ で 、 彼 は ほとんど 言 う こと が な かっ た 。
2019-09-17 05:21:48.197779 ... Target  = that being the case , he had little to say .
2019-09-17 05:21:48.197788 ... Predict = that is the reason , he had no saying say .
2019-09-17 05:21:48.197965 ... Epoch 10/10, Sample 49763
2019-09-17 05:21:48.197978 ... Source  = 私 たち は 英語 で 少な から ず 誤り を 犯 し ま す 。
2019-09-17 05:21:48.197988 ... Target  = we make not a few mistakes when speaking in english .
2019-09-17 05:21:48.197998 ... Predict = we make little a little mistakes in you in english .
2019-09-17 05:21:48.198200 ... Epoch 10/10, Sample 49764
2019-09-17 05:21:48.198214 ... Source  = 我々 を 生か そ う と し て 彼 は 死 ん だ 。
2019-09-17 05:21:48.198233 ... Target  = he died , so we might live .
2019-09-17 05:21:48.198243 ... Predict = he died , but we should live .
2019-09-17 05:21:48.198455 ... Epoch 10/10, Sample 49765
2019-09-17 05:21:48.198468 ... Source  = 彼 の 人生 の 目的 は 名医 に な る こと だっ た 。
2019-09-17 05:21:48.198478 ... Target  = his aim in life was to become a great doctor .
2019-09-17 05:21:48.198487 ... Predict = his life in life was to be a good doctor .
2019-09-17 05:21:48.198669 ... Epoch 10/10, Sample 49766
2019-09-17 05:21:48.198683 ... Source  = 何 か これ を 切 る もの を 貸 し て くださ い 。
2019-09-17 05:21:48.198693 ... Target  = lend me something with which to cut this .
2019-09-17 05:21:48.198702 ... Predict = lend me something to you to change this .
2019-09-17 05:21:48.198877 ... Epoch 10/10, Sample 49767
2019-09-17 05:21:48.198890 ... Source  = 私 は 引退 し よ う と 決心 し て い ま す 。
2019-09-17 05:21:48.198900 ... Target  = i have decided to retire .
2019-09-17 05:21:48.198910 ... Predict = i 'm my to retire .
2019-09-17 05:21:48.199084 ... Epoch 10/10, Sample 49768
2019-09-17 05:21:48.199097 ... Source  = 納屋 に は 自動 車 を 置 い て は いけ な い 。
2019-09-17 05:21:48.199107 ... Target  = don 't keep the car in the barn .
2019-09-17 05:21:48.199116 ... Predict = don 't keep the car in the barn .
2019-09-17 05:21:48.199292 ... Epoch 10/10, Sample 49769
2019-09-17 05:21:48.199305 ... Source  = 君 が 仕事 を 終え て しま う まで 待 ち ま す 。
2019-09-17 05:21:48.199315 ... Target  = i will wait until you have finished your homework .
2019-09-17 05:21:48.199325 ... Predict = i 'll wait until you have finished your work .
2019-09-17 05:21:48.199499 ... Epoch 10/10, Sample 49770
2019-09-17 05:21:48.199512 ... Source  = あの 橋 は この 橋 の 半分 の 長 さ で あ る 。
2019-09-17 05:21:48.199522 ... Target  = that bridge is half as long as this one .
2019-09-17 05:21:48.199531 ... Predict = that bridge is half the long as this . .
2019-09-17 05:21:48.199705 ... Epoch 10/10, Sample 49771
2019-09-17 05:21:48.199718 ... Source  = 彼 は ま も な く 帰 っ て くる だ ろ う 。
2019-09-17 05:21:48.199728 ... Target  = he will be back soon .
2019-09-17 05:21:48.199738 ... Predict = he will come back before .
2019-09-17 05:21:48.199913 ... Epoch 10/10, Sample 49772
2019-09-17 05:21:48.199926 ... Source  = その 町 は 大き く な っ て 市 と な っ た 。
2019-09-17 05:21:48.199937 ... Target  = the town grew into a city .
2019-09-17 05:21:48.199946 ... Predict = the town became to a city .
2019-09-17 05:21:48.200122 ... Epoch 10/10, Sample 49773
2019-09-17 05:21:48.200135 ... Source  = 私 は あなた を 親友 だ と 思 っ て い ま す 。
2019-09-17 05:21:48.200145 ... Target  = i look on you as my best friend .
2019-09-17 05:21:48.200154 ... Predict = i think at you as a friend friend .
2019-09-17 05:21:48.200328 ... Epoch 10/10, Sample 49774
2019-09-17 05:21:48.200341 ... Source  = 私 に でき る の は 全力 を 尽く す だけ で す 。
2019-09-17 05:21:48.200351 ... Target  = all i can do is to do my best .
2019-09-17 05:21:48.200361 ... Predict = it i can do my to do my best .
2019-09-17 05:21:48.200566 ... Epoch 10/10, Sample 49775
2019-09-17 05:21:48.200579 ... Source  = これ は 若 い 人 たち が 解決 す べ き 問題 だ 。
2019-09-17 05:21:48.200589 ... Target  = this is a problem for young people to solve .
2019-09-17 05:21:48.200598 ... Predict = this is the problem that young people to solve .
2019-09-17 05:21:48.200773 ... Epoch 10/10, Sample 49776
2019-09-17 05:21:48.200785 ... Source  = 私 たち は もう じき 幸福 に な る の で す ね 。
2019-09-17 05:21:48.200796 ... Target  = we 'll soon be happy , won 't we ?
2019-09-17 05:21:48.200805 ... Predict = we are be be happy now won 't we ?
2019-09-17 05:21:48.200979 ... Epoch 10/10, Sample 49777
2019-09-17 05:21:48.200992 ... Source  = 私 は お 金 が な く て 困 っ て い る 。
2019-09-17 05:21:48.201002 ... Target  = i 'm a bit pushed for money .
2019-09-17 05:21:48.201012 ... Predict = i am afraid little trouble for money .
2019-09-17 05:21:48.201185 ... Epoch 10/10, Sample 49778
2019-09-17 05:21:48.201198 ... Source  = あれ ほど 金 を 持 ち ながら 彼 は 幸福 で な い 。
2019-09-17 05:21:48.201209 ... Target  = for all his wealth , he is not happy .
2019-09-17 05:21:48.201218 ... Predict = i the , life , he is not happy .
2019-09-17 05:21:48.201393 ... Epoch 10/10, Sample 49779
2019-09-17 05:21:48.201406 ... Source  = 空 から 見 る と 、 その 島 は とても 美し かっ た 。
2019-09-17 05:21:48.201416 ... Target  = seen from the sky , the island was very beautiful .
2019-09-17 05:21:48.201426 ... Predict = seeing from the sky , the island was very beautiful .
2019-09-17 05:21:48.201600 ... Epoch 10/10, Sample 49780
2019-09-17 05:21:48.201614 ... Source  = おとなし く し て い たら 、 この 時計 を あげ よ う 。
2019-09-17 05:21:48.201624 ... Target  = if you are a good boy , i will give you this watch .
2019-09-17 05:21:48.201633 ... Predict = if you keep quiet quiet rest , i will give you this watch .
2019-09-17 05:21:48.201809 ... Epoch 10/10, Sample 49781
2019-09-17 05:21:48.201822 ... Source  = 私 の 車 は 古 い が 、 まだ 良 く 走 る 。
2019-09-17 05:21:48.201832 ... Target  = although my car is very old , it still runs very well .
2019-09-17 05:21:48.201842 ... Predict = my my car is still old , i runs runs very well .
2019-09-17 05:21:48.202016 ... Epoch 10/10, Sample 49782
2019-09-17 05:21:48.202029 ... Source  = 全体 的 に み る と 、 君 の 考え は よろし い 。
2019-09-17 05:21:48.202039 ... Target  = by and large , your idea is a good one .
2019-09-17 05:21:48.202048 ... Predict = you the the , you idea is good good one .
2019-09-17 05:21:48.202223 ... Epoch 10/10, Sample 49783
2019-09-17 05:21:48.202236 ... Source  = 僕 は もっと い い 目 を みせ る こと が でき る 。
2019-09-17 05:21:48.202246 ... Target  = i can show you a better time .
2019-09-17 05:21:48.202255 ... Predict = i can look you a more . .
2019-09-17 05:21:48.202432 ... Epoch 10/10, Sample 49784
2019-09-17 05:21:48.202446 ... Source  = 私 は 彼女 に 事実 を 伝え る 勇気 が 無 かっ た 。
2019-09-17 05:21:48.202456 ... Target  = i didn 't have the courage to tell her the fact .
2019-09-17 05:21:48.202465 ... Predict = i didn 't have the reason to tell her the fact .
2019-09-17 05:21:48.202648 ... Epoch 10/10, Sample 49785
2019-09-17 05:21:48.202662 ... Source  = 彼 は とても おもしろ い 本 を 持 っ て い ま す 。
2019-09-17 05:21:48.202672 ... Target  = he has an interesting book .
2019-09-17 05:21:48.202681 ... Predict = he has a interesting book .
2019-09-17 05:21:48.202855 ... Epoch 10/10, Sample 49786
2019-09-17 05:21:48.202868 ... Source  = 彼 は 私 に その 本 は おもしろ い と 言 っ た 。
2019-09-17 05:21:48.202878 ... Target  = he told me that the book was interesting .
2019-09-17 05:21:48.202888 ... Predict = he said me that the book was interesting .
2019-09-17 05:21:48.203064 ... Epoch 10/10, Sample 49787
2019-09-17 05:21:48.203077 ... Source  = 私 は その 問 が たいへん やさし い こと を 発見 し た 。
2019-09-17 05:21:48.203087 ... Target  = i found the question very easy .
2019-09-17 05:21:48.203097 ... Predict = i found the meaning very easy .
2019-09-17 05:21:48.203271 ... Epoch 10/10, Sample 49788
2019-09-17 05:21:48.203284 ... Source  = 私 たち は 朝食 前 に 散歩 する の が 習慣 だっ た 。
2019-09-17 05:21:48.203294 ... Target  = we used to take a walk before breakfast .
2019-09-17 05:21:48.203304 ... Predict = we used to take a walk before breakfast .
2019-09-17 05:21:48.203479 ... Epoch 10/10, Sample 49789
2019-09-17 05:21:48.203493 ... Source  = 「 ユリ 」 を 英語 で 何 と い い ま す か 。
2019-09-17 05:21:48.203503 ... Target  = what do you call '' yuri '' in english ?
2019-09-17 05:21:48.203513 ... Predict = what do you say '' anything '' in english ?
2019-09-17 05:21:48.203688 ... Epoch 10/10, Sample 49790
2019-09-17 05:21:48.203701 ... Source  = あなた は 去年 から 篠山 に 住 ん で い ま す か 。
2019-09-17 05:21:48.203711 ... Target  = have you lived in sasayama since last year ?
2019-09-17 05:21:48.203721 ... Predict = have you lived in sasayama since last year ?
2019-09-17 05:21:48.203895 ... Epoch 10/10, Sample 49791
2019-09-17 05:21:48.203908 ... Source  = 私 たち は 彼 が すぐ 戻 っ て くる と 思 う 。
2019-09-17 05:21:48.203918 ... Target  = we expect him to come back soon .
2019-09-17 05:21:48.203928 ... Predict = we think him to come back soon .
2019-09-17 05:21:48.204102 ... Epoch 10/10, Sample 49792
2019-09-17 05:21:48.204115 ... Source  = 私 は 両親 に は 必ず 毎月 手紙 を 書 き ま す 。
2019-09-17 05:21:48.204125 ... Target  = i never fail to write to my parents every month .
2019-09-17 05:21:48.204135 ... Predict = i often write to write to my parents every month .
2019-09-17 05:21:48.204309 ... Epoch 10/10, Sample 49793
2019-09-17 05:21:48.204322 ... Source  = 今日 は 、 この 辺 で やめ て お き ま しょ う 。
2019-09-17 05:21:48.204332 ... Target  = let 's call it a day today .
2019-09-17 05:21:48.204342 ... Predict = let 's get this around day today .
2019-09-17 05:21:48.204516 ... Epoch 10/10, Sample 49794
2019-09-17 05:21:48.204529 ... Source  = 私 は 疲れ て い た の で 行 か な かっ た 。
2019-09-17 05:21:48.204539 ... Target  = i was tired , so i didn 't go .
2019-09-17 05:21:48.204549 ... Predict = i was tired , but i didn 't go .
2019-09-17 05:21:48.204723 ... Epoch 10/10, Sample 49795
2019-09-17 05:21:48.204737 ... Source  = 私 は 今度 の 日曜 日 に テニス を する つもり で す 。
2019-09-17 05:21:48.204747 ... Target  = i am going to play tennis next sunday .
2019-09-17 05:21:48.204756 ... Predict = i am going to play tennis next sunday .
2019-09-17 05:21:48.204930 ... Epoch 10/10, Sample 49796
2019-09-17 05:21:48.204943 ... Source  = 誕生 日 に は 、 新し い 時計 を プレゼント し よ う 。
2019-09-17 05:21:48.204953 ... Target  = you shall have a new watch for your birthday .
2019-09-17 05:21:48.204962 ... Predict = i 'll have a new watch for your birthday .
2019-09-17 05:21:48.205137 ... Epoch 10/10, Sample 49797
2019-09-17 05:21:48.205150 ... Source  = 学生 の 中 に は 車 で 通学 する 者 も い る 。
2019-09-17 05:21:48.205160 ... Target  = some of the students come to school by car .
2019-09-17 05:21:48.205170 ... Predict = some other the students go to school by car .
2019-09-17 05:21:48.205343 ... Epoch 10/10, Sample 49798
2019-09-17 05:21:48.205357 ... Source  = おと と い 私 の 友人 達 が 遊 び に 来 た 。
2019-09-17 05:21:48.205367 ... Target  = my friends came to see me the day before yesterday .
2019-09-17 05:21:48.205376 ... Predict = my friends came to see me yesterday day before yesterday .
2019-09-17 05:21:48.205560 ... Epoch 10/10, Sample 49799
2019-09-17 05:21:48.205574 ... Source  = 何 が 起こ っ て も 僕 は 気 に し な い 。
2019-09-17 05:21:48.205593 ... Target  = i don 't care what happens .
2019-09-17 05:21:48.205603 ... Predict = i don 't care what to .
2019-09-17 05:21:48.205785 ... Epoch 10/10, Sample 49800
2019-09-17 05:21:48.205798 ... Source  = 早 く 回復 さ れ る よう に お 祈り し ま す 。
2019-09-17 05:21:48.205808 ... Target  = i hope you 'll get well soon .
2019-09-17 05:21:48.205818 ... Predict = i hope you will get well soon .
2019-09-17 05:21:48.205993 ... Epoch 10/10, Sample 49801
2019-09-17 05:21:48.206007 ... Source  = その うわさ を 彼 は 何 と も 感 じ な かっ た 。
2019-09-17 05:21:48.206017 ... Target  = the rumor was nothing to him .
2019-09-17 05:21:48.206026 ... Predict = the rumor was not of him .
2019-09-17 05:21:48.206202 ... Epoch 10/10, Sample 49802
2019-09-17 05:21:48.206216 ... Source  = あなた は 私 を ばか 者 だ と 思 い ま す か 。
2019-09-17 05:21:48.206226 ... Target  = do you take me for a fool ?
2019-09-17 05:21:48.206235 ... Predict = do you think me for a fool ?
2019-09-17 05:21:48.206412 ... Epoch 10/10, Sample 49803
2019-09-17 05:21:48.206425 ... Source  = 私 たち は 昨夜 映画 を 見 に 行 き ま し た 。
2019-09-17 05:21:48.206435 ... Target  = we went to the movies last night .
2019-09-17 05:21:48.206444 ... Predict = we went to the movies last night .
2019-09-17 05:21:48.206626 ... Epoch 10/10, Sample 49804
2019-09-17 05:21:48.206640 ... Source  = 私 たち は ２ 月 に 短 い 休暇 を と っ た 。
2019-09-17 05:21:48.206650 ... Target  = we had a short vacation in february .
2019-09-17 05:21:48.206659 ... Predict = we had a short vacation in two .
2019-09-17 05:21:48.206832 ... Epoch 10/10, Sample 49805
2019-09-17 05:21:48.206845 ... Source  = 彼 の 犬 は 庭 の 中 を 走 っ て い た 。
2019-09-17 05:21:48.206855 ... Target  = his dog was running in the yard .
2019-09-17 05:21:48.206864 ... Predict = his dog was running in the garden .
2019-09-17 05:21:48.207039 ... Epoch 10/10, Sample 49806
2019-09-17 05:21:48.207052 ... Source  = 彼女 は 彼 が 遅刻 し た と い っ て 責め た 。
2019-09-17 05:21:48.207062 ... Target  = she accused him of being late .
2019-09-17 05:21:48.207071 ... Predict = she was him that being late .
2019-09-17 05:21:48.207247 ... Epoch 10/10, Sample 49807
2019-09-17 05:21:48.207260 ... Source  = もし 急 げ ば あなた は 彼 に 追いつ く だ ろ う 。
2019-09-17 05:21:48.207270 ... Target  = if you hurry , you will catch up with him .
2019-09-17 05:21:48.207280 ... Predict = if you hurry , you will catch up with him .
2019-09-17 05:21:48.207454 ... Epoch 10/10, Sample 49808
2019-09-17 05:21:48.207467 ... Source  = 私 は 宿題 が 終わ っ て から テレビ を 見 ま す 。
2019-09-17 05:21:48.207477 ... Target  = i watch tv after i finish my homework .
2019-09-17 05:21:48.207486 ... Predict = i often television from i have my homework .
2019-09-17 05:21:48.207661 ... Epoch 10/10, Sample 49809
2019-09-17 05:21:48.207674 ... Source  = なぜ 彼 の 話 し 方 が 嫌い な の で す か 。
2019-09-17 05:21:48.207684 ... Target  = why do you dislike his way of speaking ?
2019-09-17 05:21:48.207694 ... Predict = why do you like him way with his ?
2019-09-17 05:21:48.207869 ... Epoch 10/10, Sample 49810
2019-09-17 05:21:48.207883 ... Source  = トム が 試験 に 合格 する の を 期待 し て い る 。
2019-09-17 05:21:48.207893 ... Target  = i expect that tom will pass the exam .
2019-09-17 05:21:48.207902 ... Predict = i expect that tom will pass the examination .
2019-09-17 05:21:48.208077 ... Epoch 10/10, Sample 49811
2019-09-17 05:21:48.208090 ... Source  = 私 は 自分 が 正し い と 信 じ て い ま す 。
2019-09-17 05:21:48.208100 ... Target  = i believe i am in the right .
2019-09-17 05:21:48.208110 ... Predict = i believe myself am right the right .
2019-09-17 05:21:48.208284 ... Epoch 10/10, Sample 49812
2019-09-17 05:21:48.208297 ... Source  = そこ に 行 く の に 一 時間 かか る で しょ う 。
2019-09-17 05:21:48.208307 ... Target  = it will take an hour to get there .
2019-09-17 05:21:48.208316 ... Predict = it will be an hour to get there .
2019-09-17 05:21:48.208491 ... Epoch 10/10, Sample 49813
2019-09-17 05:21:48.208505 ... Source  = 良 い 返事 を お 待 ち し て お り ま す 。
2019-09-17 05:21:48.208515 ... Target  = i am looking forward to receiving your favorable answer .
2019-09-17 05:21:48.208524 ... Predict = i 'll forward forward to receiving your reply answer .
2019-09-17 05:21:48.208697 ... Epoch 10/10, Sample 49814
2019-09-17 05:21:48.208710 ... Source  = 彼 と 知り合 っ て も う 長 い の で す か 。
2019-09-17 05:21:48.208720 ... Target  = have you known him for a long time ?
2019-09-17 05:21:48.208730 ... Predict = do you known him for a long time ?
2019-09-17 05:21:48.208904 ... Epoch 10/10, Sample 49815
2019-09-17 05:21:48.208917 ... Source  = 彼 ら は お 昼 を す ま せ て から 出かけ た 。
2019-09-17 05:21:48.208927 ... Target  = they went out after they finished lunch .
2019-09-17 05:21:48.208937 ... Predict = they went out after you went their .
2019-09-17 05:21:48.209110 ... Epoch 10/10, Sample 49816
2019-09-17 05:21:48.209123 ... Source  = 彼 ら は 何 時間 も の 間 彼 を 待 っ た 。
2019-09-17 05:21:48.209133 ... Target  = they waited for him for hours .
2019-09-17 05:21:48.209143 ... Predict = they waited for him for hours .
2019-09-17 05:21:48.209318 ... Epoch 10/10, Sample 49817
2019-09-17 05:21:48.209332 ... Source  = 火事 の とき に は この 窓 を 破 っ て くださ い 。
2019-09-17 05:21:48.209342 ... Target  = in case of fire , break this window .
2019-09-17 05:21:48.209351 ... Predict = please the , the , please this window .
2019-09-17 05:21:48.209524 ... Epoch 10/10, Sample 49818
2019-09-17 05:21:48.209537 ... Source  = 医者 に み て もら わ な けれ ば な ら な い 。
2019-09-17 05:21:48.209547 ... Target  = you need to see a doctor .
2019-09-17 05:21:48.209556 ... Predict = you must to be the doctor .
2019-09-17 05:21:48.209731 ... Epoch 10/10, Sample 49819
2019-09-17 05:21:48.209744 ... Source  = 誰 が 君 に その ニュース を 教え た の で す か 。
2019-09-17 05:21:48.209754 ... Target  = who told you the news ?
2019-09-17 05:21:48.209763 ... Predict = who told you the news ?
2019-09-17 05:21:48.209938 ... Epoch 10/10, Sample 49820
2019-09-17 05:21:48.209951 ... Source  = 彼 は 私 たち に 多く の 教訓 を 与え て くれ た 。
2019-09-17 05:21:48.209962 ... Target  = he told us many lessons .
2019-09-17 05:21:48.209971 ... Predict = he gave us many lessons .
2019-09-17 05:21:48.210146 ... Epoch 10/10, Sample 49821
2019-09-17 05:21:48.210160 ... Source  = この 本 は やさし い 英語 で 書 か れ て い る 。
2019-09-17 05:21:48.210170 ... Target  = this book is written in simple english .
2019-09-17 05:21:48.210179 ... Predict = this book is written in easy english .
2019-09-17 05:21:48.210354 ... Epoch 10/10, Sample 49822
2019-09-17 05:21:48.210367 ... Source  = なん と か そこ で 支え よ 。 入市 を 許 す な 。
2019-09-17 05:21:48.210377 ... Target  = hold them there ! don 't let them into the city !
2019-09-17 05:21:48.210386 ... Predict = i there there , i 't let me know the swim !
2019-09-17 05:21:48.210567 ... Epoch 10/10, Sample 49823
2019-09-17 05:21:48.210581 ... Source  = 君 は コアラ を 見 た こと が あ り ま す か 。
2019-09-17 05:21:48.210591 ... Target  = have you ever seen a koala ?
2019-09-17 05:21:48.210600 ... Predict = have you ever seen a koala ?
2019-09-17 05:21:48.210804 ... Epoch 10/10, Sample 49824
2019-09-17 05:21:48.210817 ... Source  = 彼 ら に は ほとんど 時間 が 残 っ て い な い 。
2019-09-17 05:21:48.210827 ... Target  = they hardly have any time left .
2019-09-17 05:21:48.210836 ... Predict = they have have had time left .
2019-09-17 05:21:48.211010 ... Epoch 10/10, Sample 49825
2019-09-17 05:21:48.211023 ... Source  = 一 時間 半 も ずっと 待 っ て い た の で す 。
2019-09-17 05:21:48.211033 ... Target  = i 've been waiting for one and a half-hours .
2019-09-17 05:21:48.211043 ... Predict = i have been waiting for an and once couple .
2019-09-17 05:21:48.211218 ... Epoch 10/10, Sample 49826
2019-09-17 05:21:48.211231 ... Source  = その ルール を 私 に 説明 し て くれ ま せ ん か 。
2019-09-17 05:21:48.211241 ... Target  = will you explain the rule to me ?
2019-09-17 05:21:48.211251 ... Predict = will you explain the rule to me ?
2019-09-17 05:21:48.211425 ... Epoch 10/10, Sample 49827
2019-09-17 05:21:48.211438 ... Source  = 彼女 は 大学 で 勉強 でき る よう に アルバイト を 見つけ た 。
2019-09-17 05:21:48.211449 ... Target  = she got a part-time job so that she could study at college .
2019-09-17 05:21:48.211458 ... Predict = she found a part-time job to that she could study at college .
2019-09-17 05:21:48.211633 ... Epoch 10/10, Sample 49828
2019-09-17 05:21:48.211646 ... Source  = あなた は どの くらい 長 く 釧路 に い ま し た か 。
2019-09-17 05:21:48.211656 ... Target  = how long have you been in kushiro ?
2019-09-17 05:21:48.211666 ... Predict = how long have you been in kushiro ?
2019-09-17 05:21:48.211840 ... Epoch 10/10, Sample 49829
2019-09-17 05:21:48.211853 ... Source  = 何 人 ぐらい 音楽 が わか る と 思 い ま す か 。
2019-09-17 05:21:48.211863 ... Target  = how many people do you think have an ear for music ?
2019-09-17 05:21:48.211872 ... Predict = do many people do you think we an ear for music ?
2019-09-17 05:21:48.212045 ... Epoch 10/10, Sample 49830
2019-09-17 05:21:48.212058 ... Source  = 彼 ら は 列 を 作 っ て バス を 待 っ た 。
2019-09-17 05:21:48.212068 ... Target  = they waited in line for the bus .
2019-09-17 05:21:48.212078 ... Predict = they waited the for for the bus .
2019-09-17 05:21:48.212252 ... Epoch 10/10, Sample 49831
2019-09-17 05:21:48.212265 ... Source  = 彼女 は 目 を 輝 か せ て 入 っ て 来 た 。
2019-09-17 05:21:48.212275 ... Target  = she came in with her eyes shining .
2019-09-17 05:21:48.212284 ... Predict = she came into her her eyes shining .
2019-09-17 05:21:48.212458 ... Epoch 10/10, Sample 49832
2019-09-17 05:21:48.212471 ... Source  = 木 の 下 に あ る 自転 車 は 私 の で す 。
2019-09-17 05:21:48.212481 ... Target  = the bicycle under the tree is mine .
2019-09-17 05:21:48.212491 ... Predict = a bicycle under the tree is mine .
2019-09-17 05:21:48.212664 ... Epoch 10/10, Sample 49833
2019-09-17 05:21:48.212677 ... Source  = 彼女 は 彼 の い い 相手 に な る だ ろ う 。
2019-09-17 05:21:48.212687 ... Target  = she will be a good companion for him .
2019-09-17 05:21:48.212696 ... Predict = she will be a good companion to him .
2019-09-17 05:21:48.212871 ... Epoch 10/10, Sample 49834
2019-09-17 05:21:48.212884 ... Source  = その 子 は 眠 り ながら 何 か を ぶつぶつ 言 っ た 。
2019-09-17 05:21:48.212894 ... Target  = that child murmured something in his sleep .
2019-09-17 05:21:48.212903 ... Predict = the child murmured something to the sleep .
2019-09-17 05:21:48.213078 ... Epoch 10/10, Sample 49835
2019-09-17 05:21:48.213091 ... Source  = やっと 雪 が や み 、 暖か く な り ま し た 。
2019-09-17 05:21:48.213101 ... Target  = it has finally stopped snowing and has warmed up .
2019-09-17 05:21:48.213111 ... Predict = at has been been hot and we done up .
2019-09-17 05:21:48.213283 ... Epoch 10/10, Sample 49836
2019-09-17 05:21:48.213296 ... Source  = あなた の 父 さん は 何 を し て い ま す か 。
2019-09-17 05:21:48.213306 ... Target  = what does your father do ?
2019-09-17 05:21:48.213315 ... Predict = what do your father do ?
2019-09-17 05:21:48.213491 ... Epoch 10/10, Sample 49837
2019-09-17 05:21:48.213504 ... Source  = 彼女 は 私 を 見 て にこり と さえ し な かっ た 。
2019-09-17 05:21:48.213514 ... Target  = she did not so much as smile at me .
2019-09-17 05:21:48.213523 ... Predict = she did not so much as smile at me .
2019-09-17 05:21:48.213697 ... Epoch 10/10, Sample 49838
2019-09-17 05:21:48.213710 ... Source  = お 金 の こと は ジェーン に あた っ て み る よ 。
2019-09-17 05:21:48.213720 ... Target  = i 'll try jane for some money .
2019-09-17 05:21:48.213729 ... Predict = you 'll be jane for the money .
2019-09-17 05:21:48.213904 ... Epoch 10/10, Sample 49839
2019-09-17 05:21:48.213917 ... Source  = 彼 の 部屋 に は 机 の ほか 何 も 無 かっ た 。
2019-09-17 05:21:48.213927 ... Target  = there was nothing but a desk in his room .
2019-09-17 05:21:48.213936 ... Predict = there was nothing but the desk in his room .
2019-09-17 05:21:48.214110 ... Epoch 10/10, Sample 49840
2019-09-17 05:21:48.214123 ... Source  = 彼 は 非常 に 貧乏 だ と 言 わ れ て い る 。
2019-09-17 05:21:48.214133 ... Target  = he is said to be very poor .
2019-09-17 05:21:48.214143 ... Predict = he is said to be very poor .
2019-09-17 05:21:48.214317 ... Epoch 10/10, Sample 49841
2019-09-17 05:21:48.214330 ... Source  = 今日 は どちら か と い え ば 、 気分 が 良 い 。
2019-09-17 05:21:48.214340 ... Target  = today he is better , if anything .
2019-09-17 05:21:48.214349 ... Predict = either , is better , but anything .
2019-09-17 05:21:48.214524 ... Epoch 10/10, Sample 49842
2019-09-17 05:21:48.214543 ... Source  = 彼 の 言葉 は 私 の 人生 に 大きな 影響 を 与え た 。
2019-09-17 05:21:48.214554 ... Target  = his words had a great effect on my life .
2019-09-17 05:21:48.214563 ... Predict = his words gave a large impact on my life .
2019-09-17 05:21:48.214738 ... Epoch 10/10, Sample 49843
2019-09-17 05:21:48.214751 ... Source  = 彼女 は 息子 の 成功 を 切 に 望 ん で い る 。
2019-09-17 05:21:48.214761 ... Target  = she is very anxious for her son to succeed .
2019-09-17 05:21:48.214771 ... Predict = she is anxious anxious for her son 's succeed .
2019-09-17 05:21:48.214947 ... Epoch 10/10, Sample 49844
2019-09-17 05:21:48.214960 ... Source  = 母 は 私 に 部屋 を 掃除 する よう に 言 っ た 。
2019-09-17 05:21:48.214970 ... Target  = mother told me to clean the room .
2019-09-17 05:21:48.214980 ... Predict = mother told me to clean the room .
2019-09-17 05:21:48.215154 ... Epoch 10/10, Sample 49845
2019-09-17 05:21:48.215167 ... Source  = 私 の 計画 に 賛成 で す か 、 反対 で す か 。
2019-09-17 05:21:48.215177 ... Target  = are you for or against my plan ?
2019-09-17 05:21:48.215187 ... Predict = are you for or against my plan ?
2019-09-17 05:21:48.215360 ... Epoch 10/10, Sample 49846
2019-09-17 05:21:48.215373 ... Source  = 彼 は 必要 以上 も お 金 を 持 っ て い る 。
2019-09-17 05:21:48.215383 ... Target  = he has more money than is needed .
2019-09-17 05:21:48.215393 ... Predict = he has more money than he needed .
2019-09-17 05:21:48.215567 ... Epoch 10/10, Sample 49847
2019-09-17 05:21:48.215580 ... Source  = 夏 に 関係 あ る こと なら 何 で も 好き で す 。
2019-09-17 05:21:48.215590 ... Target  = i like everything that comes with summer .
2019-09-17 05:21:48.215600 ... Predict = i like anything to is in summer .
2019-09-17 05:21:48.215805 ... Epoch 10/10, Sample 49848
2019-09-17 05:21:48.215818 ... Source  = 夫 は 失業 中 で 職 を 探 し て い ま す 。
2019-09-17 05:21:48.215828 ... Target  = my husband is out of work and looking for a job .
2019-09-17 05:21:48.215837 ... Predict = my husband is looking of work for looking for a job .
2019-09-17 05:21:48.216013 ... Epoch 10/10, Sample 49849
2019-09-17 05:21:48.216026 ... Source  = すぐ に 新し い 洋服 が 必要 に な り ま し た 。
2019-09-17 05:21:48.216036 ... Target  = soon he needed new clothes .
2019-09-17 05:21:48.216045 ... Predict = at as need a clothes .
2019-09-17 05:21:48.216220 ... Epoch 10/10, Sample 49850
2019-09-17 05:21:48.216234 ... Source  = あなた を 驚 か せ た く な かっ た の で す 。
2019-09-17 05:21:48.216244 ... Target  = i did not want to alarm you .
2019-09-17 05:21:48.216253 ... Predict = i didn not want to see you .
2019-09-17 05:21:48.216429 ... Epoch 10/10, Sample 49851
2019-09-17 05:21:48.216443 ... Source  = 私 は 学校 に 行 く 途中 で 叔父 に あ っ た 。
2019-09-17 05:21:48.216453 ... Target  = i saw my uncle on the way to school .
2019-09-17 05:21:48.216462 ... Predict = i met my uncle on my way to school .
2019-09-17 05:21:48.216637 ... Epoch 10/10, Sample 49852
2019-09-17 05:21:48.216650 ... Source  = 私 は 言葉 で 気持ち を 伝え る こと が でき ま す 。
2019-09-17 05:21:48.216660 ... Target  = i can convey my feelings in words .
2019-09-17 05:21:48.216669 ... Predict = i can convey myself feelings in words .
2019-09-17 05:21:48.216844 ... Epoch 10/10, Sample 49853
2019-09-17 05:21:48.216857 ... Source  = うち の 電話 は どこ か 故障 し て い る ら しい 。
2019-09-17 05:21:48.216868 ... Target  = there seems to be something wrong with our telephone .
2019-09-17 05:21:48.216877 ... Predict = i 's to be wrong wrong with my telephone .
2019-09-17 05:21:48.217052 ... Epoch 10/10, Sample 49854
2019-09-17 05:21:48.217065 ... Source  = 彼 は 彼女 に 愛 し て る よ と 言 っ た 。
2019-09-17 05:21:48.217075 ... Target  = he told her that he loved her .
2019-09-17 05:21:48.217084 ... Predict = he said her that he loved her .
2019-09-17 05:21:48.217261 ... Epoch 10/10, Sample 49855
2019-09-17 05:21:48.217274 ... Source  = この 単語 の 意味 が わか ら な い の で す が 。
2019-09-17 05:21:48.217284 ... Target  = i don 't understand this word .
2019-09-17 05:21:48.217293 ... Predict = i don 't understand this word .
2019-09-17 05:21:48.217468 ... Epoch 10/10, Sample 49856
2019-09-17 05:21:48.217481 ... Source  = 彼 ら は この 下 の 階 に 住 ん で い る 。
2019-09-17 05:21:48.217491 ... Target  = they live downstairs .
2019-09-17 05:21:48.217501 ... Predict = they live on .
2019-09-17 05:21:48.217676 ... Epoch 10/10, Sample 49857
2019-09-17 05:21:48.217689 ... Source  = 医者 を 呼 ぶ べ き だ と 思 い ま す か 。
2019-09-17 05:21:48.217699 ... Target  = do you think we should send for the doctor ?
2019-09-17 05:21:48.217709 ... Predict = do you think we should call for the doctor ?
2019-09-17 05:21:48.217883 ... Epoch 10/10, Sample 49858
2019-09-17 05:21:48.217896 ... Source  = 彼女 は 多く の 高価 な 本 を 持 っ て い る 。
2019-09-17 05:21:48.217906 ... Target  = she has many valuable books .
2019-09-17 05:21:48.217915 ... Predict = she has many valuable books .
2019-09-17 05:21:48.218091 ... Epoch 10/10, Sample 49859
2019-09-17 05:21:48.218104 ... Source  = 私 は 人前 で 演説 する こと に 慣れ て い な い 。
2019-09-17 05:21:48.218114 ... Target  = i 'm not used to making speeches in public .
2019-09-17 05:21:48.218123 ... Predict = i am not used to making a in public .
2019-09-17 05:21:48.218298 ... Epoch 10/10, Sample 49860
2019-09-17 05:21:48.218311 ... Source  = 私 の 父 は 英語 と 同様 に フランス 語 も 話せ る 。
2019-09-17 05:21:48.218321 ... Target  = my father can speak french as well as english .
2019-09-17 05:21:48.218331 ... Predict = my father can speak french as well as english .
2019-09-17 05:21:48.218506 ... Epoch 10/10, Sample 49861
2019-09-17 05:21:48.218519 ... Source  = 私 たち は いつ 病気 に な る か も 知れ な い 。
2019-09-17 05:21:48.218529 ... Target  = we may fall ill at any moment .
2019-09-17 05:21:48.218544 ... Predict = we may have ill when any moment .
2019-09-17 05:21:48.218718 ... Epoch 10/10, Sample 49862
2019-09-17 05:21:48.218731 ... Source  = 父 は 、 私 に 自転 車 を 買 っ て くれ た 。
2019-09-17 05:21:48.218741 ... Target  = father bought me a bicycle .
2019-09-17 05:21:48.218750 ... Predict = my bought me a bicycle .
2019-09-17 05:21:48.218926 ... Epoch 10/10, Sample 49863
2019-09-17 05:21:48.218939 ... Source  = 人生 で 一番 い い 時 は 、 若 い 時 で す 。
2019-09-17 05:21:48.218949 ... Target  = the best time of life is when you are young .
2019-09-17 05:21:48.218959 ... Predict = the best time is life is when the are young .
2019-09-17 05:21:48.219133 ... Epoch 10/10, Sample 49864
2019-09-17 05:21:48.219146 ... Source  = あなた が 学校 に 遅れ た 理由 を 言 い な さ い 。
2019-09-17 05:21:48.219156 ... Target  = tell me the reason why you were late for school .
2019-09-17 05:21:48.219165 ... Predict = tell me the reason why you were late for school .
2019-09-17 05:21:48.219340 ... Epoch 10/10, Sample 49865
2019-09-17 05:21:48.219353 ... Source  = 我々 の チーム は ３ 対 １ で 試合 に 勝 っ た 。
2019-09-17 05:21:48.219364 ... Target  = we won the game by three goals to one .
2019-09-17 05:21:48.219373 ... Predict = our won the game in three goals to one .
2019-09-17 05:21:48.219549 ... Epoch 10/10, Sample 49866
2019-09-17 05:21:48.219562 ... Source  = 忘れ ず に 明日 ６ 時 に 迎え に 来 て くださ い 。
2019-09-17 05:21:48.219572 ... Target  = don 't forget to pick me up at 6 o 'clock tomorrow .
2019-09-17 05:21:48.219581 ... Predict = don 't forget to come me up at six o 'clock tomorrow .
2019-09-17 05:21:48.219757 ... Epoch 10/10, Sample 49867
2019-09-17 05:21:48.219770 ... Source  = それ は 触れ な い で お く の が 一番 い い 。
2019-09-17 05:21:48.219780 ... Target  = it is best left untouched .
2019-09-17 05:21:48.219789 ... Predict = it 's the but untouched .
2019-09-17 05:21:48.219965 ... Epoch 10/10, Sample 49868
2019-09-17 05:21:48.219978 ... Source  = 誰 一人 彼 の よう な 男 を 尊敬 し な かっ た 。
2019-09-17 05:21:48.219988 ... Target  = no one respected men like him .
2019-09-17 05:21:48.219998 ... Predict = no one respected man . him .
2019-09-17 05:21:48.220173 ... Epoch 10/10, Sample 49869
2019-09-17 05:21:48.220186 ... Source  = 彼 が どう な っ た か 誰 も 想像 でき な い 。
2019-09-17 05:21:48.220196 ... Target  = no one can imagine what has become of him .
2019-09-17 05:21:48.220205 ... Predict = no one can imagine him has become of him .
2019-09-17 05:21:48.220381 ... Epoch 10/10, Sample 49870
2019-09-17 05:21:48.220394 ... Source  = 僕 が 訪ね て 行 っ た 時 彼 は 不在 だっ た 。
2019-09-17 05:21:48.220404 ... Target  = he wasn 't at home when i called on him .
2019-09-17 05:21:48.220414 ... Predict = he was 't at the when i called out him .
2019-09-17 05:21:48.220587 ... Epoch 10/10, Sample 49871
2019-09-17 05:21:48.220600 ... Source  = 机 の 上 に ペン が 何 本 あ り ま す か 。
2019-09-17 05:21:48.220610 ... Target  = are there any pens on the desk ?
2019-09-17 05:21:48.220620 ... Predict = what there any pens on the desk ?
2019-09-17 05:21:48.220805 ... Epoch 10/10, Sample 49872
2019-09-17 05:21:48.220829 ... Source  = 誰 が 彼 の 母 の 代わり を する だ ろ う か 。
2019-09-17 05:21:48.220848 ... Target  = who will take the place of his mother ?
2019-09-17 05:21:48.220857 ... Predict = who will take place place of his mother ?
2019-09-17 05:21:52.093081 ... Epoch 10/10, Sample 49873
2019-09-17 05:21:52.093122 ... Source  = 我々 は 誰 も その 問題 に は 関係 な い 。
2019-09-17 05:21:52.093136 ... Target  = none of us are related to the problem .
2019-09-17 05:21:52.093147 ... Predict = none of us have concerned to the problem .
2019-09-17 05:21:52.093318 ... Epoch 10/10, Sample 49874
2019-09-17 05:21:52.093333 ... Source  = 彼 は 私 の 忠告 を 相手 に し な い 。
2019-09-17 05:21:52.093343 ... Target  = he won 't look at my advice .
2019-09-17 05:21:52.093353 ... Predict = he never 't make on my advice .
2019-09-17 05:21:52.093521 ... Epoch 10/10, Sample 49875
2019-09-17 05:21:52.093535 ... Source  = この 川 は 泳 ぐ の に 危険 で あ る 。
2019-09-17 05:21:52.093545 ... Target  = this river is dangerous to swim in .
2019-09-17 05:21:52.093555 ... Predict = this river is dangerous to swim in .
2019-09-17 05:21:52.093721 ... Epoch 10/10, Sample 49876
2019-09-17 05:21:52.093734 ... Source  = 金 で 幸せ を 買 う こと は でき な い 。
2019-09-17 05:21:52.093745 ... Target  = money can 't buy happiness .
2019-09-17 05:21:52.093754 ... Predict = money cannot 't buy happiness .
2019-09-17 05:21:52.093920 ... Epoch 10/10, Sample 49877
2019-09-17 05:21:52.093933 ... Source  = 彼 は もっと 注意 す べ き だっ た の に 。
2019-09-17 05:21:52.093943 ... Target  = he should have been more careful .
2019-09-17 05:21:52.093953 ... Predict = he should have been more careful .
2019-09-17 05:21:52.094118 ... Epoch 10/10, Sample 49878
2019-09-17 05:21:52.094131 ... Source  = 私 は この 部屋 を 居心地 よ く し よ う 。
2019-09-17 05:21:52.094141 ... Target  = i 'll make the room comfortable .
2019-09-17 05:21:52.094150 ... Predict = i will make this room in .
2019-09-17 05:21:52.094314 ... Epoch 10/10, Sample 49879
2019-09-17 05:21:52.094327 ... Source  = 彼 は すぐ に 返事 を 書 く だ ろ う 。
2019-09-17 05:21:52.094337 ... Target  = he will write an answer very soon .
2019-09-17 05:21:52.094346 ... Predict = he will write to answer soon soon .
2019-09-17 05:21:52.094512 ... Epoch 10/10, Sample 49880
2019-09-17 05:21:52.094525 ... Source  = 多く の 人 が 同じ 間違い を し て き た 。
2019-09-17 05:21:52.094544 ... Target  = many a man has made the same mistake .
2019-09-17 05:21:52.094554 ... Predict = many people man has made the same mistake .
2019-09-17 05:21:52.094722 ... Epoch 10/10, Sample 49881
2019-09-17 05:21:52.094735 ... Source  = あなた は 昨日 公園 へ 行 き ま し た ね 。
2019-09-17 05:21:52.094745 ... Target  = you went to the park yesterday , didn 't you ?
2019-09-17 05:21:52.094754 ... Predict = you went to the park yesterday , did 't you ?
2019-09-17 05:21:52.094920 ... Epoch 10/10, Sample 49882
2019-09-17 05:21:52.094934 ... Source  = ここ から 駅 まで どの くらい の 距離 で す か 。
2019-09-17 05:21:52.094944 ... Target  = how far is it from here to the station ?
2019-09-17 05:21:52.094953 ... Predict = how far is it from here to the station ?
2019-09-17 05:21:52.095119 ... Epoch 10/10, Sample 49883
2019-09-17 05:21:52.095132 ... Source  = 今日 の 私 は 、 ひとえに 両親 の おかげ で す 。
2019-09-17 05:21:52.095142 ... Target  = i owe what i am today to my parents .
2019-09-17 05:21:52.095152 ... Predict = i owe what i am today . my parents .
2019-09-17 05:21:52.095317 ... Epoch 10/10, Sample 49884
2019-09-17 05:21:52.095330 ... Source  = 君 から 手紙 が 届 く の が 楽しみ で す 。
2019-09-17 05:21:52.095340 ... Target  = i 'm looking forward to hearing from you .
2019-09-17 05:21:52.095349 ... Predict = i 'm looking forward to hearing from you .
2019-09-17 05:21:52.095516 ... Epoch 10/10, Sample 49885
2019-09-17 05:21:52.095529 ... Source  = これ は 彼 の 小説 の 中 で 断然 おもしろ い 。
2019-09-17 05:21:52.095539 ... Target  = this is by far the most interesting of his novels .
2019-09-17 05:21:52.095548 ... Predict = this is by far the most interesting of his novels .
2019-09-17 05:21:52.095713 ... Epoch 10/10, Sample 49886
2019-09-17 05:21:52.095727 ... Source  = 彼 は 私 を 知 っ て い ま す か 。
2019-09-17 05:21:52.095737 ... Target  = does he know me ?
2019-09-17 05:21:52.095746 ... Predict = do he know me ?
2019-09-17 05:21:52.095912 ... Epoch 10/10, Sample 49887
2019-09-17 05:21:52.095925 ... Source  = 彼 は 新し い 仕事 に わくわく し て い る 。
2019-09-17 05:21:52.095935 ... Target  = he 's thrilled with his new job .
2019-09-17 05:21:52.095945 ... Predict = he is thrilled with his new job .
2019-09-17 05:21:52.096111 ... Epoch 10/10, Sample 49888
2019-09-17 05:21:52.096124 ... Source  = 彼 は 読書 の し すぎ で 目 を 痛め た 。
2019-09-17 05:21:52.096134 ... Target  = he strained his eyes by reading too much .
2019-09-17 05:21:52.096143 ... Predict = he gave his eyes too reading too much .
2019-09-17 05:21:52.096308 ... Epoch 10/10, Sample 49889
2019-09-17 05:21:52.096321 ... Source  = 彼 は 信頼 す べ き 人 で は な い 。
2019-09-17 05:21:52.096331 ... Target  = he is not a man to rely on .
2019-09-17 05:21:52.096340 ... Predict = he is not a man to rely on .
2019-09-17 05:21:52.096506 ... Epoch 10/10, Sample 49890
2019-09-17 05:21:52.096520 ... Source  = 私 は 町 の ほうへ 歩 い て 行 っ た 。
2019-09-17 05:21:52.096530 ... Target  = i was walking in the direction of the town .
2019-09-17 05:21:52.096539 ... Predict = i walked walking in the place of town town .
2019-09-17 05:21:52.096705 ... Epoch 10/10, Sample 49891
2019-09-17 05:21:52.096719 ... Source  = 彼女 は 長 時間 かけ て 帽子 を 選 ん だ 。
2019-09-17 05:21:52.096729 ... Target  = it took her a long time to choose a hat .
2019-09-17 05:21:52.096738 ... Predict = it took her a long time . choose a hat .
2019-09-17 05:21:52.096902 ... Epoch 10/10, Sample 49892
2019-09-17 05:21:52.096916 ... Source  = 僕 の そば に 立 た な い で くれ よ 。
2019-09-17 05:21:52.096926 ... Target  = don 't stand by me .
2019-09-17 05:21:52.096935 ... Predict = don 't sit by me .
2019-09-17 05:21:52.097101 ... Epoch 10/10, Sample 49893
2019-09-17 05:21:52.097114 ... Source  = 彼 は ひとり で そこ に いけ る ほど 勇敢 だ 。
2019-09-17 05:21:52.097124 ... Target  = he is brave enough to go there by himself .
2019-09-17 05:21:52.097133 ... Predict = he is brave enough to go there by himself .
2019-09-17 05:21:52.097298 ... Epoch 10/10, Sample 49894
2019-09-17 05:21:52.097311 ... Source  = 何 を 言 お う と し て い る の 。
2019-09-17 05:21:52.097321 ... Target  = what are you getting at ?
2019-09-17 05:21:52.097330 ... Predict = what are you doing at ?
2019-09-17 05:21:52.097497 ... Epoch 10/10, Sample 49895
2019-09-17 05:21:52.097510 ... Source  = 今朝 に な っ て やっと 雨 が や ん だ 。
2019-09-17 05:21:52.097520 ... Target  = it was not till this morning that it stopped raining .
2019-09-17 05:21:52.097530 ... Predict = it was up until this morning that it stopped raining .
2019-09-17 05:21:52.097693 ... Epoch 10/10, Sample 49896
2019-09-17 05:21:52.097707 ... Source  = 彼女 は ピアノ を 持 っ て い ま す か 。
2019-09-17 05:21:52.097717 ... Target  = has she a piano ?
2019-09-17 05:21:52.097726 ... Predict = does she a piano ?
2019-09-17 05:21:52.097892 ... Epoch 10/10, Sample 49897
2019-09-17 05:21:52.097905 ... Source  = よく も 彼女 に そんな こと が いえ る もの だ 。
2019-09-17 05:21:52.097915 ... Target  = how dare you say such a thing to her .
2019-09-17 05:21:52.097924 ... Predict = well dare you like that a thing to her .
2019-09-17 05:21:52.098121 ... Epoch 10/10, Sample 49898
2019-09-17 05:21:52.098134 ... Source  = 息子 の こと を 考え る と 頭 が 痛 い 。
2019-09-17 05:21:52.098144 ... Target  = my son is my biggest headache .
2019-09-17 05:21:52.098153 ... Predict = my son has my mind headache .
2019-09-17 05:21:52.098319 ... Epoch 10/10, Sample 49899
2019-09-17 05:21:52.098332 ... Source  = 私 は 疲れ て い た が 眠れ な かっ た 。
2019-09-17 05:21:52.098342 ... Target  = i was tired , but i couldn 't sleep .
2019-09-17 05:21:52.098351 ... Predict = i was tired , but i couldn 't sleep .
2019-09-17 05:21:52.098516 ... Epoch 10/10, Sample 49900
2019-09-17 05:21:52.098529 ... Source  = 会 い に 来 て いただ い て どう も ありがとう 。
2019-09-17 05:21:52.098545 ... Target  = it is nice of you to come and see me .
2019-09-17 05:21:52.098555 ... Predict = thank is kind of you to come and see me .
2019-09-17 05:21:52.098722 ... Epoch 10/10, Sample 49901
2019-09-17 05:21:52.098736 ... Source  = 彼女 は ３０ 分 で 反対 側 に つ い た 。
2019-09-17 05:21:52.098746 ... Target  = she was across in thirty minutes .
2019-09-17 05:21:52.098755 ... Predict = she was in in thirty minutes .
2019-09-17 05:21:52.098920 ... Epoch 10/10, Sample 49902
2019-09-17 05:21:52.098933 ... Source  = 私 は ２ 時間 テレビ を 見 て 楽し ん だ 。
2019-09-17 05:21:52.098943 ... Target  = i enjoyed watching tv for two hours .
2019-09-17 05:21:52.098953 ... Predict = i enjoyed watching tv for two hours .
2019-09-17 05:21:52.099119 ... Epoch 10/10, Sample 49903
2019-09-17 05:21:52.099132 ... Source  = 彼 の 考え は 僕 の と 似 て い る 。
2019-09-17 05:21:52.099142 ... Target  = his way of thinking is very similar to mine .
2019-09-17 05:21:52.099151 ... Predict = his idea of thinking is similar similar to mine .
2019-09-17 05:21:52.099316 ... Epoch 10/10, Sample 49904
2019-09-17 05:21:52.099329 ... Source  = 私 は 自分 の 仕事 に 興味 を 失 っ た 。
2019-09-17 05:21:52.099339 ... Target  = i lost interest in my work .
2019-09-17 05:21:52.099348 ... Predict = i lost interest in my work .
2019-09-17 05:21:52.099514 ... Epoch 10/10, Sample 49905
2019-09-17 05:21:52.099527 ... Source  = 日本 語 を 話せ る アメリカ 人 が 大勢 い る 。
2019-09-17 05:21:52.099537 ... Target  = there are many americans who can speak japanese .
2019-09-17 05:21:52.099547 ... Predict = there are many americans who can speak japanese .
2019-09-17 05:21:52.099712 ... Epoch 10/10, Sample 49906
2019-09-17 05:21:52.099725 ... Source  = 誰 が この 手紙 を 書 き ま し た か 。
2019-09-17 05:21:52.099735 ... Target  = who wrote this letter ?
2019-09-17 05:21:52.099744 ... Predict = who wrote this letter ?
2019-09-17 05:21:52.099909 ... Epoch 10/10, Sample 49907
2019-09-17 05:21:52.099922 ... Source  = 私 は その コンピューター を ティム と 呼 び ま す 。
2019-09-17 05:21:52.099932 ... Target  = i call the computer tim .
2019-09-17 05:21:52.099942 ... Predict = i call the computer tim .
2019-09-17 05:21:52.100108 ... Epoch 10/10, Sample 49908
2019-09-17 05:21:52.100121 ... Source  = 彼 は 身 の 危険 に 気づ い て い る 。
2019-09-17 05:21:52.100131 ... Target  = he is aware of his danger .
2019-09-17 05:21:52.100141 ... Predict = he is aware of his danger .
2019-09-17 05:21:52.100307 ... Epoch 10/10, Sample 49909
2019-09-17 05:21:52.100320 ... Source  = 彼女 は ずばぬけ て 一番 背 の 高 い 少女 だ 。
2019-09-17 05:21:52.100330 ... Target  = she is much the tallest girl .
2019-09-17 05:21:52.100340 ... Predict = she is far as tallest girl .
2019-09-17 05:21:52.100506 ... Epoch 10/10, Sample 49910
2019-09-17 05:21:52.100519 ... Source  = 彼 は 奨学 金 を もら っ て 勉強 し た 。
2019-09-17 05:21:52.100529 ... Target  = he studied on a scholarship .
2019-09-17 05:21:52.100538 ... Predict = he studied on a scholarship .
2019-09-17 05:21:52.100704 ... Epoch 10/10, Sample 49911
2019-09-17 05:21:52.100718 ... Source  = 父 は 外出 する とき は たいてい 帽子 を かぶ る 。
2019-09-17 05:21:52.100728 ... Target  = my father usually wears a hat when he goes out .
2019-09-17 05:21:52.100737 ... Predict = my father leaves takes a hat when he goes out .
2019-09-17 05:21:52.100901 ... Epoch 10/10, Sample 49912
2019-09-17 05:21:52.100914 ... Source  = ある 意味 で あなた の 言 う 事 は 正し い 。
2019-09-17 05:21:52.100924 ... Target  = in a sense , you are right .
2019-09-17 05:21:52.100934 ... Predict = in a sense , you are right .
2019-09-17 05:21:52.101101 ... Epoch 10/10, Sample 49913
2019-09-17 05:21:52.101114 ... Source  = 彼 ら は 冬 の 間 は 店 を 閉め る 。
2019-09-17 05:21:52.101124 ... Target  = they shut up their store for the winter .
2019-09-17 05:21:52.101133 ... Predict = they closed the the shop for the winter .
2019-09-17 05:21:52.101301 ... Epoch 10/10, Sample 49914
2019-09-17 05:21:52.101314 ... Source  = その 警官 は 犯人 が 逃げ る の を 見つけ た 。
2019-09-17 05:21:52.101324 ... Target  = the police caught sight of the criminal running away .
2019-09-17 05:21:52.101333 ... Predict = the policeman found running of the criminal running away .
2019-09-17 05:21:52.101498 ... Epoch 10/10, Sample 49915
2019-09-17 05:21:52.101511 ... Source  = 私 の 学校 に は バンド が あ り ま す 。
2019-09-17 05:21:52.101521 ... Target  = my school has a band .
2019-09-17 05:21:52.101530 ... Predict = my school has a band .
2019-09-17 05:21:52.101695 ... Epoch 10/10, Sample 49916
2019-09-17 05:21:52.101708 ... Source  = 飛行 機 事故 で 多く の 人 が 死亡 し た 。
2019-09-17 05:21:52.101718 ... Target  = many people were killed in the plane accident .
2019-09-17 05:21:52.101728 ... Predict = many people were killed in the plane accident .
2019-09-17 05:21:52.101893 ... Epoch 10/10, Sample 49917
2019-09-17 05:21:52.101906 ... Source  = どうぞ 、 塩 を 私 に 回 し て 下さ い 。
2019-09-17 05:21:52.101916 ... Target  = pass me the salt , please .
2019-09-17 05:21:52.101925 ... Predict = please me the salt , please .
2019-09-17 05:21:52.102091 ... Epoch 10/10, Sample 49918
2019-09-17 05:21:52.102104 ... Source  = 彼 は 身 を 入れ て 勉強 を し 始め た 。
2019-09-17 05:21:52.102114 ... Target  = he began to study with a will .
2019-09-17 05:21:52.102123 ... Predict = he began to study in his order .
2019-09-17 05:21:52.102289 ... Epoch 10/10, Sample 49919
2019-09-17 05:21:52.102302 ... Source  = 二 つ の 家 の 間 に 庭 が あ る 。
2019-09-17 05:21:52.102312 ... Target  = there is a yard between the two houses .
2019-09-17 05:21:52.102322 ... Predict = there is a garden for the two house .
2019-09-17 05:21:52.102487 ... Epoch 10/10, Sample 49920
2019-09-17 05:21:52.102500 ... Source  = 生徒 たち は 皆 彼 を 尊敬 し て い る 。
2019-09-17 05:21:52.102510 ... Target  = all the students look up to him .
2019-09-17 05:21:52.102520 ... Predict = all the students are up to him .
2019-09-17 05:21:52.102692 ... Epoch 10/10, Sample 49921
2019-09-17 05:21:52.102706 ... Source  = この 戸 は どう し て も 開 か な い 。
2019-09-17 05:21:52.102716 ... Target  = the door will not open .
2019-09-17 05:21:52.102725 ... Predict = this door won not open .
2019-09-17 05:21:52.102893 ... Epoch 10/10, Sample 49922
2019-09-17 05:21:52.102906 ... Source  = トム は 今夜 早 く 帰 る と 約束 し た 。
2019-09-17 05:21:52.102916 ... Target  = tom made a promise to come home early tonight .
2019-09-17 05:21:52.102925 ... Predict = tom promised a promise to come home early tonight .
2019-09-17 05:21:52.103121 ... Epoch 10/10, Sample 49923
2019-09-17 05:21:52.103134 ... Source  = 私 たち は 昨日 、 野球 の 試合 を し た 。
2019-09-17 05:21:52.103144 ... Target  = we played a baseball game yesterday .
2019-09-17 05:21:52.103153 ... Predict = we had a baseball game yesterday .
2019-09-17 05:21:52.103349 ... Epoch 10/10, Sample 49924
2019-09-17 05:21:52.103362 ... Source  = 地球 は 太陽 の 惑星 の １ つ で あ る 。
2019-09-17 05:21:52.103372 ... Target  = the earth is one of the sun 's planets .
2019-09-17 05:21:52.103382 ... Predict = the earth is one of the sun 's planets .
2019-09-17 05:21:52.103548 ... Epoch 10/10, Sample 49925
2019-09-17 05:21:52.103561 ... Source  = 君 の 努力 は もう じき 実 る だ ろ う 。
2019-09-17 05:21:52.103571 ... Target  = your efforts will soon pay off .
2019-09-17 05:21:52.103580 ... Predict = i efforts will soon soon . .
2019-09-17 05:21:52.103746 ... Epoch 10/10, Sample 49926
2019-09-17 05:21:52.103760 ... Source  = お腹 が 空 い て い る の で す か 。
2019-09-17 05:21:52.103770 ... Target  = are you hungry ?
2019-09-17 05:21:52.103779 ... Predict = are you hungry ?
2019-09-17 05:21:52.103945 ... Epoch 10/10, Sample 49927
2019-09-17 05:21:52.103958 ... Source  = 熱 い 風呂 に 入 っ て から 寝 よ う 。
2019-09-17 05:21:52.103968 ... Target  = a hot bath and then bed .
2019-09-17 05:21:52.103977 ... Predict = let hot hot is go you .
2019-09-17 05:21:52.104144 ... Epoch 10/10, Sample 49928
2019-09-17 05:21:52.104157 ... Source  = 午後 に ２ 科目 の 授業 が あ り ま す 。
2019-09-17 05:21:52.104167 ... Target  = we have two lessons in the afternoon .
2019-09-17 05:21:52.104177 ... Predict = we have two classes in the afternoon .
2019-09-17 05:21:52.104343 ... Epoch 10/10, Sample 49929
2019-09-17 05:21:52.104356 ... Source  = 今日 授業 を 欠席 し た 学生 は 多 かっ た 。
2019-09-17 05:21:52.104367 ... Target  = there were quite a few students absent from class today .
2019-09-17 05:21:52.104376 ... Predict = there were many a lot students absent from class today .
2019-09-17 05:21:52.104541 ... Epoch 10/10, Sample 49930
2019-09-17 05:21:52.104554 ... Source  = 君 は 計画 を 中止 し た ほう が よ い 。
2019-09-17 05:21:52.104564 ... Target  = you had better call off your plan .
2019-09-17 05:21:52.104574 ... Predict = you had better call off your plan .
2019-09-17 05:21:52.104741 ... Epoch 10/10, Sample 49931
2019-09-17 05:21:52.104754 ... Source  = 試験 に 受か る よう に 真剣 に 勉強 し た 。
2019-09-17 05:21:52.104764 ... Target  = i studied really hard so as to pass the exam .
2019-09-17 05:21:52.104774 ... Predict = i studied so hard that that to pass the exam .
2019-09-17 05:21:52.104940 ... Epoch 10/10, Sample 49932
2019-09-17 05:21:52.104953 ... Source  = 彼 は 僕 に 礼状 を 送 っ て くれ た 。
2019-09-17 05:21:52.104963 ... Target  = he sent me a letter of appreciation .
2019-09-17 05:21:52.104972 ... Predict = he sent me a piece . appreciation .
2019-09-17 05:21:52.105137 ... Epoch 10/10, Sample 49933
2019-09-17 05:21:52.105150 ... Source  = 彼 に は その 仕事 を する 能力 が あ る 。
2019-09-17 05:21:52.105160 ... Target  = he has the ability to do the job .
2019-09-17 05:21:52.105169 ... Predict = he has the ability to do the job .
2019-09-17 05:21:52.105334 ... Epoch 10/10, Sample 49934
2019-09-17 05:21:52.105347 ... Source  = 私 の 意見 は あなた の と 違 い ま す 。
2019-09-17 05:21:52.105357 ... Target  = my opinion is different from yours .
2019-09-17 05:21:52.105366 ... Predict = my opinion is different from yours .
2019-09-17 05:21:52.105532 ... Epoch 10/10, Sample 49935
2019-09-17 05:21:52.105545 ... Source  = 彼 は 新し い 靴 を 履 い て み た 。
2019-09-17 05:21:52.105555 ... Target  = he tried putting on his new shoes .
2019-09-17 05:21:52.105564 ... Predict = he tried out on his new shoes .
2019-09-17 05:21:52.105729 ... Epoch 10/10, Sample 49936
2019-09-17 05:21:52.105742 ... Source  = タバコ を 吸 っ て かま い ま せ ん か 。
2019-09-17 05:21:52.105752 ... Target  = do you mind if i smoke ?
2019-09-17 05:21:52.105762 ... Predict = do you mind if i smoke ?
2019-09-17 05:21:52.105928 ... Epoch 10/10, Sample 49937
2019-09-17 05:21:52.105941 ... Source  = 思い出 そ う と し て い る ところ で す 。
2019-09-17 05:21:52.105951 ... Target  = i 'm trying to remember .
2019-09-17 05:21:52.105960 ... Predict = i 'm sure to see .
2019-09-17 05:21:52.106123 ... Epoch 10/10, Sample 49938
2019-09-17 05:21:52.106136 ... Source  = 私 は 外出 し な けれ ば な ら な い 。
2019-09-17 05:21:52.106146 ... Target  = i must go out .
2019-09-17 05:21:52.106156 ... Predict = i must go out .
2019-09-17 05:21:52.106320 ... Epoch 10/10, Sample 49939
2019-09-17 05:21:52.106333 ... Source  = 彼女 は 人形 を 作 っ て い ま す か 。
2019-09-17 05:21:52.106343 ... Target  = is she making a doll ?
2019-09-17 05:21:52.106353 ... Predict = is she making a doll ?
2019-09-17 05:21:52.106519 ... Epoch 10/10, Sample 49940
2019-09-17 05:21:52.106536 ... Source  = 道路 を 横断 する とき に は 注意 し なさ い 。
2019-09-17 05:21:52.106548 ... Target  = you must be careful in crossing the road .
2019-09-17 05:21:52.106557 ... Predict = you must be careful in crossing the street .
2019-09-17 05:21:52.106725 ... Epoch 10/10, Sample 49941
2019-09-17 05:21:52.106738 ... Source  = 彼 の 運転 は 交通 規則 違反 で あ っ た 。
2019-09-17 05:21:52.106749 ... Target  = his driving was against traffic rules .
2019-09-17 05:21:52.106758 ... Predict = his driver was against traffic rules .
2019-09-17 05:21:52.106924 ... Epoch 10/10, Sample 49942
2019-09-17 05:21:52.106937 ... Source  = 誰 も が 世界 平和 を 望 ん で い る 。
2019-09-17 05:21:52.106947 ... Target  = we are all longing for peace in the world .
2019-09-17 05:21:52.106956 ... Predict = everyone are longing longing for world peace the world .
2019-09-17 05:21:52.107120 ... Epoch 10/10, Sample 49943
2019-09-17 05:21:52.107133 ... Source  = あなた は 父親 を 尊敬 し て い ま し た 。
2019-09-17 05:21:52.107143 ... Target  = you used to look up to your father .
2019-09-17 05:21:52.107152 ... Predict = you look to look up to your father .
2019-09-17 05:21:52.107318 ... Epoch 10/10, Sample 49944
2019-09-17 05:21:52.107331 ... Source  = 彼 は 夏 は 花 に 毎日 水 を や る 。
2019-09-17 05:21:52.107341 ... Target  = he waters the flowers every day in summer .
2019-09-17 05:21:52.107351 ... Predict = he makes flowers flowers every day . summer .
2019-09-17 05:21:52.107514 ... Epoch 10/10, Sample 49945
2019-09-17 05:21:52.107527 ... Source  = 道 に 迷 っ て い る ん で す か 。
2019-09-17 05:21:52.107537 ... Target  = are you lost ?
2019-09-17 05:21:52.107547 ... Predict = are you lost ?
2019-09-17 05:21:52.107711 ... Epoch 10/10, Sample 49946
2019-09-17 05:21:52.107724 ... Source  = 彼女 と お 姉 さん を 間違え て しま っ た 。
2019-09-17 05:21:52.107734 ... Target  = i took her for her sister .
2019-09-17 05:21:52.107744 ... Predict = i made her for her sister .
2019-09-17 05:21:52.107908 ... Epoch 10/10, Sample 49947
2019-09-17 05:21:52.107921 ... Source  = 我々 は その 問題 に つ い て 議論 し た 。
2019-09-17 05:21:52.107931 ... Target  = we discussed the matter .
2019-09-17 05:21:52.107941 ... Predict = we discussed the problem .
2019-09-17 05:21:52.108127 ... Epoch 10/10, Sample 49948
2019-09-17 05:21:52.108140 ... Source  = 彼 は 私 達 の 問題 の 答え を みつけ た 。
2019-09-17 05:21:52.108151 ... Target  = he came up with an answer to our problem .
2019-09-17 05:21:52.108161 ... Predict = he found to with the answer to our problem .
2019-09-17 05:21:52.108346 ... Epoch 10/10, Sample 49949
2019-09-17 05:21:52.108360 ... Source  = パーティー は ５ 月 ２２ 日 に 行 わ れ た 。
2019-09-17 05:21:52.108378 ... Target  = the party was held on may 22 .
2019-09-17 05:21:52.108388 ... Predict = the party was held on 22 22 .
2019-09-17 05:21:52.108553 ... Epoch 10/10, Sample 49950
2019-09-17 05:21:52.108566 ... Source  = 彼 は アフリカ に 行 き たが っ て い る 。
2019-09-17 05:21:52.108576 ... Target  = he wants to go to africa .
2019-09-17 05:21:52.108586 ... Predict = he wants to go to africa .
2019-09-17 05:21:52.108751 ... Epoch 10/10, Sample 49951
2019-09-17 05:21:52.108764 ... Source  = 彼女 は どこ へ 行 っ て しま っ た の ？
2019-09-17 05:21:52.108774 ... Target  = where has she gone ?
2019-09-17 05:21:52.108783 ... Predict = where has she gone ?
2019-09-17 05:21:52.108948 ... Epoch 10/10, Sample 49952
2019-09-17 05:21:52.108961 ... Source  = 彼 は あちこち に 行 っ た こと が あ る 。
2019-09-17 05:21:52.108971 ... Target  = he has been to many places .
2019-09-17 05:21:52.108981 ... Predict = he has been to foot places .
2019-09-17 05:21:52.109145 ... Epoch 10/10, Sample 49953
2019-09-17 05:21:52.109159 ... Source  = 彼 は 庭 で 仕事 を する こと が 好き だ 。
2019-09-17 05:21:52.109169 ... Target  = he likes to work in the garden .
2019-09-17 05:21:52.109178 ... Predict = he likes to work in the garden .
2019-09-17 05:21:52.109343 ... Epoch 10/10, Sample 49954
2019-09-17 05:21:52.109356 ... Source  = もう 少し ゆっくり 話 し て いただけ ま せ ん か 。
2019-09-17 05:21:52.109366 ... Target  = would you mind speaking more slowly ?
2019-09-17 05:21:52.109376 ... Predict = could you mind talking more slowly ?
2019-09-17 05:21:52.109542 ... Epoch 10/10, Sample 49955
2019-09-17 05:21:52.109555 ... Source  = この 単語 に は 二 つ の 意味 が あ る 。
2019-09-17 05:21:52.109565 ... Target  = this word has two meanings .
2019-09-17 05:21:52.109574 ... Predict = this word has two meanings .
2019-09-17 05:21:52.109738 ... Epoch 10/10, Sample 49956
2019-09-17 05:21:52.109751 ... Source  = 先生 は 授業 の 始め に 出席 を と っ た 。
2019-09-17 05:21:52.109761 ... Target  = our teacher called the roll at the beginning of the class .
2019-09-17 05:21:52.109770 ... Predict = the teacher took the roll on the class of class class .
2019-09-17 05:21:52.109935 ... Epoch 10/10, Sample 49957
2019-09-17 05:21:52.109948 ... Source  = 彼女 その 瞬間 に カッと な っ た ん だ ね 。
2019-09-17 05:21:52.109958 ... Target  = she saw red at that moment .
2019-09-17 05:21:52.109967 ... Predict = she stood her at the moment .
2019-09-17 05:21:52.110132 ... Epoch 10/10, Sample 49958
2019-09-17 05:21:52.110145 ... Source  = 昨日 、 彼 は 基地 に 帰 ら な かっ た 。
2019-09-17 05:21:52.110155 ... Target  = he didn 't come back to the base yesterday .
2019-09-17 05:21:52.110164 ... Predict = he didn 't go back to the base yesterday .
2019-09-17 05:21:52.110329 ... Epoch 10/10, Sample 49959
2019-09-17 05:21:52.110342 ... Source  = 私 達 は 正午 に 働 く の を やめ た 。
2019-09-17 05:21:52.110352 ... Target  = we stopped working at noon .
2019-09-17 05:21:52.110361 ... Predict = we stopped working at noon .
2019-09-17 05:21:52.110526 ... Epoch 10/10, Sample 49960
2019-09-17 05:21:52.110544 ... Source  = 彼女 は 彼 の 安否 を 心配 し て い る 。
2019-09-17 05:21:52.110554 ... Target  = she is anxious about his safety .
2019-09-17 05:21:52.110564 ... Predict = she is anxious about his safety .
2019-09-17 05:21:52.110730 ... Epoch 10/10, Sample 49961
2019-09-17 05:21:52.110743 ... Source  = 私 は その ニュース を 聞 い て 驚 い た 。
2019-09-17 05:21:52.110753 ... Target  = i was surprised at the news .
2019-09-17 05:21:52.110763 ... Predict = i was surprised to the news .
2019-09-17 05:21:52.110927 ... Epoch 10/10, Sample 49962
2019-09-17 05:21:52.110940 ... Source  = 彼 は これ 以上 働け な い と 分か っ た 。
2019-09-17 05:21:52.110950 ... Target  = he found it impossible to work any longer .
2019-09-17 05:21:52.110959 ... Predict = he found it to to work any longer .
2019-09-17 05:21:52.111126 ... Epoch 10/10, Sample 49963
2019-09-17 05:21:52.111139 ... Source  = 彼 ら が や れ ば １ 週間 で 終わ る 。
2019-09-17 05:21:52.111149 ... Target  = between them they can finish in a week .
2019-09-17 05:21:52.111158 ... Predict = it them , finish finish in a week .
2019-09-17 05:21:52.111325 ... Epoch 10/10, Sample 49964
2019-09-17 05:21:52.111338 ... Source  = 彼女 が その 成功 を 喜 ぶ もの も もっとも だ 。
2019-09-17 05:21:52.111348 ... Target  = she may well be pleased with the success .
2019-09-17 05:21:52.111358 ... Predict = she may well be happy with the success .
2019-09-17 05:21:52.111523 ... Epoch 10/10, Sample 49965
2019-09-17 05:21:52.111536 ... Source  = もう 私 に 手紙 を 書 か な い で くれ 。
2019-09-17 05:21:52.111546 ... Target  = please don 't write letters to me anymore .
2019-09-17 05:21:52.111556 ... Predict = don don 't write to to me any .
2019-09-17 05:21:52.111720 ... Epoch 10/10, Sample 49966
2019-09-17 05:21:52.111733 ... Source  = 子供 たち は 毎日 この 公園 で 遊 び ま す 。
2019-09-17 05:21:52.111743 ... Target  = children play in this park every day .
2019-09-17 05:21:52.111753 ... Predict = children play in this park every day .
2019-09-17 05:21:52.111917 ... Epoch 10/10, Sample 49967
2019-09-17 05:21:52.111930 ... Source  = 彼 ら は この 町 に 住 ん で い る 。
2019-09-17 05:21:52.111940 ... Target  = they live in this town .
2019-09-17 05:21:52.111950 ... Predict = they live in this town .
2019-09-17 05:21:52.112113 ... Epoch 10/10, Sample 49968
2019-09-17 05:21:52.112126 ... Source  = 彼 は いつ も 喜 ん で 他人 を 手伝 う 。
2019-09-17 05:21:52.112136 ... Target  = he is always willing to help others .
2019-09-17 05:21:52.112146 ... Predict = he always always kind to help others .
2019-09-17 05:21:52.112311 ... Epoch 10/10, Sample 49969
2019-09-17 05:21:52.112324 ... Source  = 私 は 兄 を 見送り に 駅 に い っ た 。
2019-09-17 05:21:52.112334 ... Target  = i went to the station to see my brother off .
2019-09-17 05:21:52.112343 ... Predict = i went to the station to see my brother off .
2019-09-17 05:21:52.112508 ... Epoch 10/10, Sample 49970
2019-09-17 05:21:52.112521 ... Source  = 我々 は 平和 の ため に 働 い て い る 。
2019-09-17 05:21:52.112531 ... Target  = we are working in the interest of peace .
2019-09-17 05:21:52.112541 ... Predict = we are working on the sake of peace .
2019-09-17 05:21:52.112706 ... Epoch 10/10, Sample 49971
2019-09-17 05:21:52.112719 ... Source  = 私 は もう 子供 で は あ り ま せ ん 。
2019-09-17 05:21:52.112729 ... Target  = i am no longer a child .
2019-09-17 05:21:52.112739 ... Predict = i am not longer a child .
2019-09-17 05:21:52.112901 ... Epoch 10/10, Sample 49972
2019-09-17 05:21:52.112914 ... Source  = 誰 が 彼女 を 手伝 っ て くれ ま す か 。
2019-09-17 05:21:52.112924 ... Target  = who helps her ?
2019-09-17 05:21:52.112933 ... Predict = who helps her ?
2019-09-17 05:21:52.113101 ... Epoch 10/10, Sample 49973
2019-09-17 05:21:52.113114 ... Source  = ところ で あなた は どんな スポーツ が 好き で す か 。
2019-09-17 05:21:52.113124 ... Target  = well , what sports do you like ?
2019-09-17 05:21:52.113133 ... Predict = what , what sport do you like ?
2019-09-17 05:21:52.113314 ... Epoch 10/10, Sample 49974
2019-09-17 05:21:52.113328 ... Source  = 彼 は よ い 医者 に な る だ ろ う 。
2019-09-17 05:21:52.113338 ... Target  = he will become a good doctor .
2019-09-17 05:21:52.113348 ... Predict = he will become a good doctor .
2019-09-17 05:21:52.113533 ... Epoch 10/10, Sample 49975
2019-09-17 05:21:52.113546 ... Source  = 私 は 友人 の 家 に 滞在 し て い る 。
2019-09-17 05:21:52.113556 ... Target  = i am staying with a friend .
2019-09-17 05:21:52.113565 ... Predict = i am staying with my friend .
2019-09-17 05:21:52.113730 ... Epoch 10/10, Sample 49976
2019-09-17 05:21:52.113743 ... Source  = 愛 さえ あ れ ば 僕 ら は 大丈夫 だ よ 。
2019-09-17 05:21:52.113753 ... Target  = as long as we love each other , we 'll be all right .
2019-09-17 05:21:52.113763 ... Predict = all i as many love , other , you 'll be all right .
2019-09-17 05:21:52.113929 ... Epoch 10/10, Sample 49977
2019-09-17 05:21:52.113942 ... Source  = 明日 、 朝 の 仕事 は 休 む つもり で す 。
2019-09-17 05:21:52.113952 ... Target  = i 'm taking tomorrow morning off from work .
2019-09-17 05:21:52.113962 ... Predict = i am absent a morning , tomorrow the .
2019-09-17 05:21:52.114125 ... Epoch 10/10, Sample 49978
2019-09-17 05:21:52.114138 ... Source  = 誰 が 知 っ て い る だ ろ う か 。
2019-09-17 05:21:52.114148 ... Target  = who knows that ?
2019-09-17 05:21:52.114158 ... Predict = who knows that ?
2019-09-17 05:21:52.114324 ... Epoch 10/10, Sample 49979
2019-09-17 05:21:52.114336 ... Source  = この 道 を ゆ け ば 駅 に 出 ま す 。
2019-09-17 05:21:52.114346 ... Target  = this road will lead you to the station .
2019-09-17 05:21:52.114356 ... Predict = this road leads lead you to the station .
2019-09-17 05:21:52.114520 ... Epoch 10/10, Sample 49980
2019-09-17 05:21:52.114538 ... Source  = 宿題 を 忘れ る なんて 君 は 不 注意 だっ た 。
2019-09-17 05:21:52.114549 ... Target  = it was careless of you to forget your homework .
2019-09-17 05:21:52.114559 ... Predict = it was careless of you to forget your homework .
2019-09-17 05:21:52.114725 ... Epoch 10/10, Sample 49981
2019-09-17 05:21:52.114738 ... Source  = 私 に お 金 を せび ら な い で くれ 。
2019-09-17 05:21:52.114748 ... Target  = don 't ask me for money .
2019-09-17 05:21:52.114758 ... Predict = don 't ask me to money .
2019-09-17 05:21:52.114922 ... Epoch 10/10, Sample 49982
2019-09-17 05:21:52.114935 ... Source  = 彼 は その 問題 を 解 く こと が でき た 。
2019-09-17 05:21:52.114945 ... Target  = he was able to solve the problem .
2019-09-17 05:21:52.114954 ... Predict = he could able to solve the problem .
2019-09-17 05:21:52.115120 ... Epoch 10/10, Sample 49983
2019-09-17 05:21:52.115133 ... Source  = 彼 に 会 っ た こと を 覚え て い る 。
2019-09-17 05:21:52.115143 ... Target  = i remember seeing him .
2019-09-17 05:21:52.115152 ... Predict = i remember seeing him .
2019-09-17 05:21:52.115315 ... Epoch 10/10, Sample 49984
2019-09-17 05:21:52.115328 ... Source  = 結局 、 君 の ほう が 間違 っ て い た 。
2019-09-17 05:21:52.115338 ... Target  = you were wrong after all .
2019-09-17 05:21:52.115348 ... Predict = in were wrong in all .
2019-09-17 05:21:52.115513 ... Epoch 10/10, Sample 49985
2019-09-17 05:21:52.115526 ... Source  = 乾電池 を 探 し て い る の で す が 。
2019-09-17 05:21:52.115536 ... Target  = i 'm looking for batteries .
2019-09-17 05:21:52.115545 ... Predict = i 'm looking for batteries .
2019-09-17 05:21:52.115711 ... Epoch 10/10, Sample 49986
2019-09-17 05:21:52.115724 ... Source  = 心配 ごと の な い 人 は ほとんど い な い 。
2019-09-17 05:21:52.115734 ... Target  = few people are free from cares .
2019-09-17 05:21:52.115743 ... Predict = few people are not from care .
2019-09-17 05:21:52.115908 ... Epoch 10/10, Sample 49987
2019-09-17 05:21:52.115921 ... Source  = お 金 は ほとんど 残 っ て い な かっ た 。
2019-09-17 05:21:52.115931 ... Target  = there was scarcely any money left .
2019-09-17 05:21:52.115941 ... Predict = there was not little money left .
2019-09-17 05:21:52.116106 ... Epoch 10/10, Sample 49988
2019-09-17 05:21:52.116120 ... Source  = 彼 は 一行 から 離れ て 一人 で 旅行 し た 。
2019-09-17 05:21:52.116129 ... Target  = he made the trip independent of his company .
2019-09-17 05:21:52.116139 ... Predict = he went a trip from of his party .
2019-09-17 05:21:52.116303 ... Epoch 10/10, Sample 49989
2019-09-17 05:21:52.116316 ... Source  = 時間 で す 。 答案 を 出 し て 下さ い 。
2019-09-17 05:21:52.116326 ... Target  = time is up . hand in your papers .
2019-09-17 05:21:52.116335 ... Predict = you is the . i your the papers .
2019-09-17 05:21:52.116501 ... Epoch 10/10, Sample 49990
2019-09-17 05:21:52.116514 ... Source  = 君 が 間違 っ て い る こと は 明白 だ 。
2019-09-17 05:21:52.116524 ... Target  = it 's clear that you 're wrong .
2019-09-17 05:21:52.116533 ... Predict = it 's evident that you are wrong .
2019-09-17 05:21:52.116698 ... Epoch 10/10, Sample 49991
2019-09-17 05:21:52.116711 ... Source  = 彼 は 死 ん だ の も 同様 で あ る 。
2019-09-17 05:21:52.116721 ... Target  = he is as good as dead .
2019-09-17 05:21:52.116731 ... Predict = he is as if as dead .
2019-09-17 05:21:52.116896 ... Epoch 10/10, Sample 49992
2019-09-17 05:21:52.116909 ... Source  = スミス 氏 は 彼女 を 自分 の 秘書 に し た 。
2019-09-17 05:21:52.116919 ... Target  = mr smith made her his secretary .
2019-09-17 05:21:52.116929 ... Predict = mr smith made her his secretary .
2019-09-17 05:21:52.117093 ... Epoch 10/10, Sample 49993
2019-09-17 05:21:52.117106 ... Source  = 私 は すぐ に あなた に 追いつ く だ ろ う 。
2019-09-17 05:21:52.117116 ... Target  = i will soon come up with you .
2019-09-17 05:21:52.117125 ... Predict = i will soon catch up with you .
2019-09-17 05:21:52.117290 ... Epoch 10/10, Sample 49994
2019-09-17 05:21:52.117303 ... Source  = 問題 は 彼女 が 来る か どう か で あ る 。
2019-09-17 05:21:52.117313 ... Target  = the point is whether she will come or not .
2019-09-17 05:21:52.117322 ... Predict = the question is whether she will come or not .
2019-09-17 05:21:52.117486 ... Epoch 10/10, Sample 49995
2019-09-17 05:21:52.117499 ... Source  = 夕方 から 雨 だ って い っ て い た よ 。
2019-09-17 05:21:52.117509 ... Target  = it said rain from this evening .
2019-09-17 05:21:52.117519 ... Predict = it was it from the evening .
2019-09-17 05:21:52.117684 ... Epoch 10/10, Sample 49996
2019-09-17 05:21:52.117697 ... Source  = 旅 は およそ ５ 時間 くらい かか る で しょ う 。
2019-09-17 05:21:52.117707 ... Target  = the trip will take some five hours .
2019-09-17 05:21:52.117716 ... Predict = the trip will take five five hours .
2019-09-17 05:21:52.117881 ... Epoch 10/10, Sample 49997
2019-09-17 05:21:52.117894 ... Source  = 子供 が 夜 １人 で 外出 する の は 危険 だ 。
2019-09-17 05:21:52.117904 ... Target  = it is dangerous for children to go out alone at night .
2019-09-17 05:21:52.117913 ... Predict = it is dangerous for children to go out alone at night .
2019-09-17 05:21:52.118077 ... Epoch 10/10, Sample 49998
2019-09-17 05:21:52.118091 ... Source  = この 仕事 が 終わ っ たら 帰 っ て よ い 。
2019-09-17 05:21:52.118101 ... Target  = you can go home after you have finished this work .
2019-09-17 05:21:52.118110 ... Predict = i may go home when i have finished this work .
2019-09-17 05:21:52.118295 ... Epoch 10/10, Sample 49999
2019-09-17 05:21:52.118309 ... Source  = もう 行 か な く ちゃ 。 じゃ 、 また ね 。
2019-09-17 05:21:52.118320 ... Target  = it 's high time to go . see you later .
2019-09-17 05:21:52.118330 ... Predict = i 's not , not go .
2019-09-17 05:21:52.118519 ... Epoch 10/10, Sample 50000
2019-09-17 05:21:52.118536 ... Source  = 君 が 僕 の 場合 だっ たら どう する つもり だい 。
2019-09-17 05:21:52.118547 ... Target  = what would you do in my place ?
2019-09-17 05:21:52.118557 ... Predict = what will you do when my place ?
2019-09-17 05:21:52.134321 ... accum_loss = 3063.806049346924
2019-09-17 05:21:52.134364 ... Saving Model ...
2019-09-17 05:21:57.401544 ... Finished.
